# 导入模块

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
from jsonpath import jsonpath
import datetime 
from datetime import timedelta ,time, datetime 
import re
from openpyxl import load_workbook
# from msoffcrypto_tool import OfficeFile
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import os
import glob
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean
import io
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

In [3]:
# 黑名单
sql_black = '''
select cause, count(id_card) from db_credit.blacklist group by cause order by count(id_card) desc
'''
df_black = clean.query(sql_black)
# df_black.to_excel('F:/需求/总需求/黑名单.xlsx', index=False)

In [4]:
t_date = datetime.now().strftime('%Y%m%d')

In [5]:
# df_order = pd.read_excel('F:/需求/七月需求/订单明细.xlsx')
df_order = pd.read_excel('F:/需求/七月需求/无标题(4).xlsx')
# df_order = pd.read_excel('F:/需求/七月需求/无标题(3).xlsx')
df_order.rename(columns={'order_number': '订单号', 'create_time': '下单时间', 'status':'订单状态', 'total_freeze_fund_amount': '冻结只有资金'}, inplace=True)
df_order['下单日期']=df_order['下单时间'].dt.date
df_order


,订单号,下单时间,订单状态,order_method,冻结只有资金,product_name,name,下单日期
0,A2024090100060613,2024-09-01 00:06:07,10,0,0.00,准新iPhone15 赠送壳膜配件 顺丰包邮,iPhone15,2024-09-01
1,A2024090100085321,2024-09-01 00:08:53,10,0,0.00,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,iPhone15Pro,2024-09-01
2,A2024090100103928,2024-09-01 00:10:39,10,0,0.00,iPhone15ProMax 降价了 全新国行 顺丰包邮 低租金高通过,iPhone15ProMax,2024-09-01
3,A2024090100115333,2024-09-01 00:11:54,10,0,1799.81,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,iPhone15Pro,2024-09-01
4,A2024090100154743,2024-09-01 00:15:47,10,0,1599.81,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,iPhone15Pro,2024-09-01
...,...,...,...,...,...,...,...,...
43612,A20241105143049151,2024-11-05 14:30:49,10,0,0.00,iPhone 16 Pro Max 全新未激活 国行正品 寄出包邮,iPhone16ProMax,2024-11-05
43613,A20241105143058152,2024-11-05 14:30:58,10,1,0.00,iPhone16Plus 全新未激活 赠送壳膜 顺丰包邮【芝麻活动专享3】,iPhone16Plus,2024-11-05
43614,A20241105143148158,2024-11-05 14:31:48,10,1,0.00,iPhone15Pro 全新无锁 非监管机 顺丰包邮,iPhone15Pro,2024-11-05
43615,A20241105143712183,2024-11-05 14:37:13,10,0,0.00,iPhone15ProMax 全新国行 顺丰包邮 低租金高通过,iPhone15ProMax,2024-11-05


In [6]:
df_order['全免押'] = np.where(df_order['冻结只有资金']==0,1,0)
df_order['全不免押'] = np.where(df_order['冻结只有资金']==-1,1,0)
df_order['部分免押'] = np.where((df_order['冻结只有资金']!=-1)&(df_order['冻结只有资金']!=0),1,0)
df_order

,订单号,下单时间,订单状态,order_method,冻结只有资金,product_name,name,下单日期,全免押,全不免押,部分免押
0,A2024090100060613,2024-09-01 00:06:07,10,0,0.00,准新iPhone15 赠送壳膜配件 顺丰包邮,iPhone15,2024-09-01,1,0,0
1,A2024090100085321,2024-09-01 00:08:53,10,0,0.00,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,iPhone15Pro,2024-09-01,1,0,0
2,A2024090100103928,2024-09-01 00:10:39,10,0,0.00,iPhone15ProMax 降价了 全新国行 顺丰包邮 低租金高通过,iPhone15ProMax,2024-09-01,1,0,0
3,A2024090100115333,2024-09-01 00:11:54,10,0,1799.81,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,iPhone15Pro,2024-09-01,0,0,1
4,A2024090100154743,2024-09-01 00:15:47,10,0,1599.81,iPhone15Pro 租金全网可比 全新 无锁 非监管机 顺丰包邮,iPhone15Pro,2024-09-01,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
43612,A20241105143049151,2024-11-05 14:30:49,10,0,0.00,iPhone 16 Pro Max 全新未激活 国行正品 寄出包邮,iPhone16ProMax,2024-11-05,1,0,0
43613,A20241105143058152,2024-11-05 14:30:58,10,1,0.00,iPhone16Plus 全新未激活 赠送壳膜 顺丰包邮【芝麻活动专享3】,iPhone16Plus,2024-11-05,1,0,0
43614,A20241105143148158,2024-11-05 14:31:48,10,1,0.00,iPhone15Pro 全新无锁 非监管机 顺丰包邮,iPhone15Pro,2024-11-05,1,0,0
43615,A20241105143712183,2024-11-05 14:37:13,10,0,0.00,iPhone15ProMax 全新国行 顺丰包邮 低租金高通过,iPhone15ProMax,2024-11-05,1,0,0


In [7]:
# 快捷下单
df_kj = df_order[df_order['order_method']==1]
df_kj_group = df_kj.groupby('下单日期').agg({'订单号': 'count', '全免押': 'sum', '全不免押': 'sum', '部分免押':'sum'})
# df_kj

In [8]:
# 非快捷下单
df_fkj = df_order[df_order['order_method']==0]
df_fkj_group = df_fkj.groupby('下单日期').agg({'订单号': 'count', '全免押': 'sum', '全不免押': 'sum', '部分免押':'sum'})

In [9]:
# iPhone16系列
df_order['name'] = df_order['name'].fillna('未知')
df_16 = df_order[df_order['name'].str.contains(pat='iPhone16', regex=True)]
df_16_group = df_16.groupby('下单日期').agg({'订单号': 'count', '全免押': 'sum', '全不免押': 'sum', '部分免押':'sum'})

In [10]:
# 非iPhone16系列
df_f16 = df_order[~df_order['name'].str.contains(pat='iPhone16', regex=True)]
df_f16_group = df_f16.groupby('下单日期').agg({'订单号': 'count', '全免押': 'sum', '全不免押': 'sum', '部分免押':'sum'})

In [11]:
# Today1 = datetime.now().strftime('%Y%m%d%H')
# with pd.ExcelWriter(f'F:/需求/七月需求/订单明细汇总_机型_{Today1}.xlsx', engine='openpyxl') as writer:
#     df_16_group.to_excel(writer, sheet_name='iPhone16')
#     df_f16_group.to_excel(writer, sheet_name='非iPhone16')

In [12]:
# with pd.ExcelWriter(f'F:/需求/七月需求/订单明细汇总_快捷下单_{Today1}.xlsx', engine='openpyxl') as writer:
#     df_kj_group.to_excel(writer, sheet_name='快捷下单')
#     df_fkj_group.to_excel(writer, sheet_name='非快捷下单')

In [13]:
# df_data = pd.read_excel('F:/日报/data.xlsx')
# df_data['日期'] = pd.to_datetime(df_data['日期'])
# df_data = df_data.set_index('日期')
# df_data = df_data[['去重订单', '前置拦截', '进件', '机审强拒', '电审拒绝', '客户取消', '无法联系', '出库前风控强拒']]

In [14]:
# weekly_sum = W_group(custom_weekly_resampler(df_data)).drop(columns=['week_day'])
# group(weekly_sum)[['week_group', '去重订单','前置拦截','前置拦截率','进件','进件通过率','机审强拒','机审强拒比例','电审拒绝','电审拒绝比例','客户取消','客户取消率','无法联系','无法联系比例','出库前风控强拒','出库前风控强拒比例']]

In [15]:
# df_ck3 = pd.read_excel('F:/租后表分析数据_ld/出库单三方数据表现-1114.xlsx')


In [16]:
sql_3 = '''
SELECT om.order_number, tor.overdue_day
FROM db_digua_business.t_overdue_record tor 
left join db_digua_business.t_order om on tor.order_id=om.id
'''
df_3 = clean.query(sql_3)

In [17]:
# df_ck31 = df_ck3[['order_number', 'new_overdue_day']].drop_duplicates()
# df_31 = df_3.drop_duplicates()
# df_ck31
# df_ck3113 = pd.merge(df_31, df_ck31, on='order_number', how='right')
# df_ck3113[(df_ck3113['overdue_day'].notna())&(df_ck3113['new_overdue_day'].notna())&(df_ck3113['overdue_day']!=df_ck3113['new_overdue_day'])]

# 截止到当前时间首逾

## 查询订单

In [24]:
# 截止当前时间的首逾逾期订单(已出库，以租后表为标准)的数量，逾期天数，应收租金new_actual_money，实收租金（已收）rent_received，未收租金new_actual_money-rent_received，分配人
sql = '''
select 
om.create_time, om.order_number, tprm.order_id, tprm.rent_received 实收租金
, ymos.refund_date, ymos.reality_refund_date
, om.status, tod.new_actual_money 应收租金
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
-- 资方订单分期表
left join (select order_id, refund_date, money, reality_refund_date from db_rent.ya_merchant_order_stages where sort=2) ymos on ymos.order_id=om.id
where date_format(om.create_time, '%Y-%m-%d')>='2025-05-12'
and date_format(om.create_time, '%Y-%m-%d')<='2025-06-12'  -- 设置时间范围
'''
df = clean.query(sql)
df

,create_time,order_number,order_id,实收租金,refund_date,reality_refund_date,status,应收租金
0,2025-05-12 00:19:51,A2025051200195067,2620453,4000.74,2025-06-15,2025-06-11 09:13:16,4,15596.3
1,2025-05-12 03:35:25,A2025051203352480,2620961,2244.38,2025-06-15,2025-06-11 18:28:11,4,12996.7
2,2025-05-12 06:47:33,A2025051206473362,2621212,1335.00,2025-06-15,2025-06-11 09:13:21,4,12996.7
3,2025-05-12 07:57:09,A2025051207570893,2621325,2115.88,2025-06-15,2025-06-12 04:30:31,4,12996.7
4,2025-05-12 08:02:39,A202505120802384,2621330,1987.38,2025-06-15,2025-06-11 09:13:34,4,12996.7
...,...,...,...,...,...,...,...,...
5395,2025-06-12 16:54:09,A20250612165409931,2887736,249.00,NaT,NaT,8,7182.0
5396,2025-06-11 14:07:33,A2025061114072394,2875109,2095.78,2025-07-14,2025-06-30 17:20:45,4,11608.7
5397,2025-06-12 17:29:22,A20250612172921436,2888249,2001.32,2025-07-15,2025-07-15 21:08:43,4,12996.7
5398,2025-06-10 14:51:48,A20250610145148898,2865833,0.00,NaT,NaT,8,12996.7


## 逾期天数

In [25]:
# 截止当前时间的首逾逾期订单(已出库，以租后表为标准)的数量，逾期天数，应收租金，实收租金，未收租金，分配人
# status=4:租赁中
t_date = datetime.now().date()
df.loc[:, 'now_day'] = pd.to_datetime(datetime.now().date())
df.loc[:, '下单日期'] = pd.to_datetime(df.create_time.dt.date)
df.loc[:, 'reality_refund_date'] = df.reality_refund_date.dt.date
# 首逾：sort=2，refund_date<now_day|reality_refund_date>refund_date
df.loc[:, 'overdue_day'] = np.where(df.reality_refund_date.isna(), np.where(df.refund_date<df.now_day, df.now_day-df.refund_date, 0), 
                                    np.where(df.reality_refund_date>df.refund_date, df.reality_refund_date-df.refund_date, 0))
df.loc[:, 'overdue_day'] = df.overdue_day.apply(lambda x: x.days)
df.loc[:, '未收租金'] = df.应收租金-df.实收租金
df_new = df[(df.overdue_day>0)&(df.reality_refund_date.isna())&(df.status==4)]
df_new

,create_time,order_number,order_id,实收租金,refund_date,reality_refund_date,status,应收租金,now_day,下单日期,overdue_day,未收租金
55,2025-05-12 11:51:59,A20250512115159694,2623104,1301.00,2025-06-15,NaT,4,7798.1,2025-09-10,2025-05-12,87,6497.10
133,2025-05-12 16:03:00,A2025051216030045,2626274,533.07,2025-06-15,NaT,4,12996.7,2025-09-10,2025-05-12,87,12463.63
191,2025-05-12 19:18:05,A20250512191804162,2628640,562.91,2025-06-15,NaT,4,12996.7,2025-09-10,2025-05-12,87,12433.79
314,2025-05-13 15:34:25,A20250513153425609,2635418,688.00,2025-06-16,NaT,4,8549.0,2025-09-10,2025-05-13,86,7861.00
326,2025-05-13 16:08:40,A20250513160839120,2635946,468.44,2025-06-16,NaT,4,12796.7,2025-09-10,2025-05-13,86,12328.26
...,...,...,...,...,...,...,...,...,...,...,...,...
5293,2025-06-12 15:41:53,A20250612154152741,2886549,520.93,2025-07-15,NaT,4,12996.7,2025-09-10,2025-06-12,57,12475.77
5302,2025-06-12 16:13:16,A20250612161316245,2887061,468.44,2025-07-15,NaT,4,12796.7,2025-09-10,2025-06-12,57,12328.26
5319,2025-06-12 17:04:35,A2025061217043599,2887914,501.89,2025-07-15,NaT,4,12996.7,2025-09-10,2025-06-12,57,12494.81
5331,2025-06-12 17:48:50,A20250612174850680,2888492,2168.30,2025-07-15,NaT,4,12997.7,2025-09-10,2025-06-12,57,10829.40


## 分配人

In [26]:
sql_name = '''
        SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
        FROM db_digua_business.t_user_verify_order_record tuvor
        left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
        where tuvor.del_flag = 0 ORDER BY tuvor.update_time
        '''
df_name = clean.query(sql_name)
df_name


,order_id,分配人,update_time
0,703280,李巧凤,2023-10-16 16:02:04
1,703287,小谢,2023-10-16 16:02:04
2,703288,小张,2023-10-16 16:02:04
3,699105,李巧凤,2023-10-16 16:09:36
4,699143,李巧凤,2023-10-16 16:09:36
...,...,...,...
281640,3896320,周汉鸿,2025-09-10 11:48:06
281641,3896336,胡春燕,2025-09-10 11:48:08
281642,3896346,冯二洋,2025-09-10 11:48:10
281643,3896022,冯二洋,2025-09-10 11:48:59


In [27]:
df_name = df_name.sort_values(by='update_time', ascending=False).groupby('order_id').head(1)
# 创建分配人映射关系 
name_dict = {
        '小张': '李巧玲',
        '小周': '李巧凤',
        '小南': '刘三妹',
        '小何': '何静',
        '小谢': '谢金凤',
        '小咪': '杨健',
        '小慧': '林思慧',
        '小滢': '胡彩滢',
        '小星': '廖丽敏',
        '小兰': '黄兰娟',
        '小晚': '周莹',
        '小芳': '罗芳'
        }
name_list = ['李巧玲','李巧凤','刘三妹','何静','谢金凤','廖丽敏','李楠','黄兰娟','杨健','林思慧','胡彩滢','周莹','罗芳', '周念慈', '周汉鸿']
# 匹配映射字典
df_name.loc[:, '信审员'] = df_name.分配人.apply(lambda x: name_dict[x] if str(x).startswith('小') and str(x) in name_dict.keys() else x)
# df_name = df_name[df_name.信审员.isin(name_list)]
df_name

,order_id,分配人,update_time,信审员
281644,3896022,廖丽敏,2025-09-10 11:49:00,廖丽敏
281642,3896346,冯二洋,2025-09-10 11:48:10,冯二洋
281641,3896336,胡春燕,2025-09-10 11:48:08,胡春燕
281640,3896320,周汉鸿,2025-09-10 11:48:06,周汉鸿
281639,3896203,冯二洋,2025-09-10 11:47:25,冯二洋
...,...,...,...,...
37,702113,李巧凤,2023-10-16 16:09:36,李巧凤
36,702045,李巧凤,2023-10-16 16:09:36,李巧凤
2,703288,小张,2023-10-16 16:02:04,李巧玲
1,703287,小谢,2023-10-16 16:02:04,谢金凤


In [28]:
# 关联分配人
df_all = df_new.merge(df_name[['order_id', '信审员']], on='order_id', how='left')
df_all

,create_time,order_number,order_id,实收租金,refund_date,reality_refund_date,status,应收租金,now_day,下单日期,overdue_day,未收租金,信审员
0,2025-05-12 11:51:59,A20250512115159694,2623104,1301.00,2025-06-15,NaT,4,7798.1,2025-09-10,2025-05-12,87,6497.10,钟桂洪
1,2025-05-12 16:03:00,A2025051216030045,2626274,533.07,2025-06-15,NaT,4,12996.7,2025-09-10,2025-05-12,87,12463.63,林婉婷
2,2025-05-12 19:18:05,A20250512191804162,2628640,562.91,2025-06-15,NaT,4,12996.7,2025-09-10,2025-05-12,87,12433.79,林婉婷
3,2025-05-13 15:34:25,A20250513153425609,2635418,688.00,2025-06-16,NaT,4,8549.0,2025-09-10,2025-05-13,86,7861.00,蔡斯静
4,2025-05-13 16:08:40,A20250513160839120,2635946,468.44,2025-06-16,NaT,4,12796.7,2025-09-10,2025-05-13,86,12328.26,蔡斯静
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,2025-06-12 15:41:53,A20250612154152741,2886549,520.93,2025-07-15,NaT,4,12996.7,2025-09-10,2025-06-12,57,12475.77,何静
150,2025-06-12 16:13:16,A20250612161316245,2887061,468.44,2025-07-15,NaT,4,12796.7,2025-09-10,2025-06-12,57,12328.26,胡彩滢
151,2025-06-12 17:04:35,A2025061217043599,2887914,501.89,2025-07-15,NaT,4,12996.7,2025-09-10,2025-06-12,57,12494.81,蔡斯静
152,2025-06-12 17:48:50,A20250612174850680,2888492,2168.30,2025-07-15,NaT,4,12997.7,2025-09-10,2025-06-12,57,10829.40,杨闽红


# 六月份买断

## 查询订单

In [29]:
# 统计六月份买断（买断表t_order_out，买断时间pay_date筛选）的订单数、使用了多少买断优惠劵、优惠前买断金额、优惠后买断金额、采购金额、毛利率（优惠前/后买断金额-采购金额）/采购金额
# ybt.`status`'任务状态，0为待处理，1为跟进中，2已挂起，3已完成，4已取消'，too.status=2为已买断
sql_md= f'''
select
too.order_id 订单ID, tod.new_actual_money 优惠前买断金额, too.discount_money 优惠金额, (tod.new_actual_money-too.discount_money) 优惠后买断金额,
tprm.purchase_amount 采购金额, om.has_actual, date_format(too.pay_date, '%Y-%m') 月份
from db_digua_business.t_order_out too
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=too.order_id
left join db_digua_business.t_order_details tod on tod.order_id=too.order_id
left join db_digua_business.t_order om on om.id=too.order_id
where date_format(too.pay_date, '%Y-%m')='2025-06'
and too.status=2
'''
df_md = clean.query(sql_md)
df_md

,订单ID,优惠前买断金额,优惠金额,优惠后买断金额,采购金额,has_actual,月份
0,1205846,5197.90,0.0,5197.90,3649.0,1,2025-06
1,850763,5197.57,0.0,5197.57,2250.0,1,2025-06
2,1500435,12416.80,0.0,12416.80,8903.0,1,2025-06
3,1349599,12996.70,0.0,12996.70,8108.0,1,2025-06
4,1182116,12996.70,0.0,12996.70,7970.0,1,2025-06
...,...,...,...,...,...,...,...
697,497019,14135.96,0.0,14135.96,8280.0,1,2025-06
698,1055232,10397.10,0.0,10397.10,6898.0,1,2025-06
699,932474,9097.30,0.0,9097.30,5867.0,1,2025-06
700,1401884,12996.70,0.0,12996.70,8108.0,1,2025-06


In [30]:

df_md.loc[:, '实际买断订单数'] = np.where(df_md.has_actual == 1, 1, 0)
# df_md.loc[:, '使用优惠劵订单数'] = np.where((df_md.status.isin([3, 4])) & (df_md.has_actual == 1) & (df_md.discount_money > 0), 1, 0)
df_md.loc[:, '使用优惠劵订单数'] = np.where(df_md.优惠金额 > 0, 1, 0)
# df_md.loc[:, '优惠后买断金额'] = np.where((df_md.status.isin([3, 4])) & (df_md.has_actual == 1),df_md.优惠后买断金额, 0)
df_md.groupby('月份').agg({'订单ID': 'count','使用优惠劵订单数': 'sum', '优惠前买断金额': 'sum', '优惠后买断金额': 'sum', '采购金额': 'sum'}).rename(columns={'订单ID': '买断订单数'})
df_md.loc[:, '发放优惠前的毛利率'] = ((df_md.优惠前买断金额 - df_md.采购金额) / df_md.采购金额).map(
        lambda x: format(x, '.2%'))
df_md.loc[:, '发放优惠后的毛利率'] = ((df_md.优惠后买断金额 - df_md.采购金额) / df_md.采购金额).map(
    lambda x: format(x, '.2%'))
# df_md
    

,买断订单数,使用优惠劵订单数,优惠前买断金额,优惠后买断金额,采购金额
月份,,,,,
2025-06,702,147,7937068.24,7844411.4,4862847.0


# 查询手机号对应的订单

In [31]:
# 查询excel文件，sheet2页面中的手机号列，与数据库订单表中7月份订单手机号列进行匹配，并返回匹配结果的订单号和手机号
df_phone_number = pd.read_excel('D:/需求/进件记录.xlsx', sheet_name='Sheet2')
# 转str类型
df_phone_number['手机号'] = df_phone_number['手机号'].astype(str)
# 取出第一列数据去重，并转换为列表
# phone_numbers = df_phone_number.iloc[:, 0].drop_duplicates().tolist()
# phone_numbers
phone_number_list = tuple(df_phone_number.iloc[:, 0].drop_duplicates().to_list())
phone_number_list


FileNotFoundError: [Errno 2] No such file or directory: 'D:/需求/进件记录.xlsx'

In [ ]:
df = df[df.user_mobile.str.contains(r'\d+')]
# 在df中筛选出手机号在phone_numbers列表中的行
df_filtered = df[df['user_mobile'].isin(phone_numbers)]
# 返回匹配结果的订单号和手机号
df_ans = df_filtered[['order_number', 'user_mobile']]
df_ans



In [ ]:

# 查询订单
sql = f'''
select order_number,user_mobile from db_digua_business.t_order where create_time between '2023-07-01' and '2023-07-15' and user_mobile in {phone_number_list}
'''
df = query(sql)
df


In [ ]:
sql = '''
        SELECT om.order_number,tprm.overdue_type,ymos.reality_refund_date,ymos.refund_date

        FROM db_digua_business.t_postlease_receivables_monitoring  tprm
        LEFT JOIN db_digua_business.t_order om on tprm.order_id = om.id 
        LEFT JOIN db_rent.ya_merchant_order_stages ymos on om.id = ymos.order_id
        where ymos.refund_date >= DATE_ADD(CURRENT_DATE,INTERVAL +0 day )
        and ymos.refund_date <= DATE_ADD(CURRENT_DATE,INTERVAL +3 day )

        '''
        
df = clean.query(sql)

In [ ]:
# from datetime import datetime, timedelta
# # 读取excel文件清单
# # 跑出清单：
# # 1.当前无逾期
# # 2.当期未还款
# # 3.还款日为观察点及后3天，比如8.1跑的，那么还款日为8.1-8.4的同时满足 1和2的清单
# df_list = pd.read_excel('F:/需求/倩姐需求/zzyj_list.xlsx')

# # 拼接df_list与df,以订单号进行匹配
# df_list_merge = df_list.merge(df, on='order_number', how='inner')
# # 选择按照reality_refund_date降序排序，取第一条数据
# # df_list_merge = df_list_merge.sort_values(by='reality_refund_date', ascending=False).groupby('order_number').head(1)
# # # 获取当前无逾期的清单
# # # df_list_merge[df_list_merge.overdue_type.notna()].head()
# # df_list_merge_no_overdue = df_list_merge[df_list_merge.overdue_type=='']
# # # 获取当前未还款的清单
# # df_list_merge_no_repay = df_list_merge[df_list_merge.实付日期new.isna()]
# df_list_merge['实付日期new'] = pd.to_datetime(df_list_merge['实付日期new'])
# df_list_merge['实付日期new'] = np.where(df_list_merge['reality_refund_date'].notna(), df_list_merge['reality_refund_date'], df_list_merge['实付日期new'])

# # 获取今天的日期
# today = pd.Timestamp(datetime.now().date())
# # 获取当前需要还款的清单,应付日期为今天到后3天的范围，还款日为观察点及后3天，比如8.1跑的，那么还款日为8.1-8.4的同时满足 1和2的清单
# df_list_merge_repay = df_list_merge[(df_list_merge.应付日期 >= today) & (df_list_merge.应付日期 <= today + timedelta(days=3))& (df_list_merge.overdue_type=='') & (df_list_merge.实付日期new.isna()) ]
# df_list_merge_repay = df_list_merge_repay.drop_duplicates(subset=['order_number'])

In [ ]:
# df_list_merge[df_list_merge['order_number']=='A20250428224312197']

# 二手汇总

In [ ]:
excel_file = pd.ExcelFile('F:/需求/七月需求/24年1月-10月机型分析.xlsx')
# 从 excel_file 中解析名为 '1月' 的工作表
df = excel_file.parse('1月')[:-1]
for sheet in excel_file.sheet_names[1:]:
    df = pd.concat([df,excel_file.parse(sheet)[:-1]])
df

In [ ]:

# df['型号'] = df['型号'].str.replace(' ', '', regex=False)
# df
# df_group = df.groupby('型号').agg({'数量': 'sum', '总回收价格':'sum', '总采购金额': 'sum', '总官网金额': 'sum'})
# df_group['总回收价格均值'] = df_group['总回收价格']/df_group['数量']
# df_group['总采购金额均值'] = df_group['总采购金额']/df_group['数量']
# df_group['总官网金额均值'] = df_group['总官网金额']/df_group['数量']
# df_group = df_group.reset_index()
# df_group['型号_new'] = df_group['型号'].str.split('/').str[0]
# df_group['内存'] = df_group['型号'].str.split('/').str[1]
# df_group.sort_values(['数量'], ascending=[False]).to_excel('F:/需求/七月需求/24年1月-10月机型分析汇总.xlsx')

# 惜仪需求

In [ ]:
df_es = pd.read_excel('F:/需求/二手手机/二手机型.xlsx')


df_es['月份'] = df_es['入库日期'].dt.month
df_es['型号内存'] = df_es['型号_new'] + '_' + df_es['内存'] + '_' + df_es['颜色']

In [ ]:
month_list = df_es['月份'].drop_duplicates().to_list()
# 使用 groupby 方法按 '型号内存' 对数据进行分组，然后使用 agg 方法对每组数据进行聚合，计算 '颜色' 的数量、'回收价格'、'采购价' 和 '官网价' 的总和。
for ml in month_list:
    df_month = df_es[df_es['月份']==ml]
    df_group = df_month.groupby('型号内存').agg({'颜色': 'count', '回收价格': 'sum', '采购价':'sum', '官网价':'sum'})
    df_group.rename(columns={'颜色': '数量', '回收价格': '总回收价格', '采购价': '总采购价', '官网价': '总官网价'}, inplace=True)
    df_group.loc['合计'] = {'数量': df_group['数量'].sum(), '总回收价格': df_group['总回收价格'].sum(), '总采购价': df_group['总采购价'].sum(),
                                        '总官网价': df_group['总官网价'].sum()}
    df_group['总采购价比值'] = df_group['总回收价格']/df_group['总采购价']
    df_group['总采购价比值'] = df_group['总采购价比值'].apply(lambda x: format(x, '.2%'))

    df_group['总官网价比值'] = df_group['总回收价格']/df_group['总官网价']
    df_group['总官网价比值'] = df_group['总官网价比值'].apply(lambda x: format(x, '.2%'))
    # if ml==1:
    #     with pd.ExcelWriter('F:/需求/二手手机/二手机型（月份）.xlsx', engine='openpyxl') as writer:
    #         df_group.to_excel(writer, sheet_name=f'{ml}月')
    # else:
    #     with pd.ExcelWriter('F:/需求/二手手机/二手机型（月份）.xlsx', engine='openpyxl', mode='a') as writer:
    #         df_group.to_excel(writer, sheet_name=f'{ml}月')

In [ ]:
sheets = pd.ExcelFile('F:/需求/二手手机/二手机型（月份）.xlsx')
df = sheets.parse('9月')[:-1]
for sheet in sheets.sheet_names[9:]:
    df = pd.concat([df,sheets.parse(sheet)[:-1]])
df


In [ ]:
def model_price(df):
    df_group = df.groupby('型号内存').agg({'数量': 'sum', '总回收价格':'sum', '总采购价': 'sum', '总官网价': 'sum'})
    df_group['总回收价格均值'] = df_group['总回收价格']/df_group['数量']
    df_group['总采购价均值'] = df_group['总采购价']/df_group['数量']
    df_group['总官网价均值'] = df_group['总官网价']/df_group['数量']
    df_group = df_group.reset_index()
    df_group['型号'] = df_group['型号内存'].str.split('_').str[0]
    df_group['内存'] = df_group['型号内存'].str.split('_').str[1]
    df_group['颜色'] = df_group['型号内存'].str.split('_').str[2]
    return df_group

In [ ]:
df['型号内存'] = df['型号内存'].str.replace(' ', '', regex=False)
df = df[df['型号内存'].str.contains((r'iPhone14|iPhone15'))]
df = df[df['型号内存'].str.contains((r'99新|95新|准新|90新'))]
df_ds = df[df['型号内存'].str.contains((r'带锁|有锁'))]
df_wds = df[~df['型号内存'].str.contains((r'带锁|有锁'))]
# model_price(df_ds).to_excel(f'F:/需求/七月需求/24年近3个月机型分析汇总_{t_date}_带锁.xlsx')
# model_price(df_wds).to_excel(f'F:/需求/七月需求/24年近3个月机型分析汇总_{t_date}_未带锁.xlsx')
# df_group.sort_values(['数量'], ascending=[False]).to_excel(f'F:/需求/七月需求/24年1月-10月机型分析汇总_{t_date}.xlsx')
# df_group.sort_values(['数量'], ascending=[False])#.to_excel(f'F:/需求/七月需求/24年近3个月机型分析汇总_{t_date}.xlsx')

# 获取优惠5折数据

In [ ]:
df_5 = pd.read_excel('F:/需求/瑞彤需求/总赠送5折优惠卷.xlsx')
df_5['订单号'] = df_5['订单号'].apply(lambda x: x.split('\\')[0].strip())
order_list = tuple(df_5['订单号'].to_list())
df_5['订单号'].shape

In [ ]:
sql=f'''
SELECT tprm.order_number 订单号, tprm.contract_price 买断价, tprm.purchase_amount 采购价,  tos.money 月租金, (tprm.contract_price-tprm.all_rental) 买断尾款
, tprm.all_deposit 商品押金, tprm.all_rental 总租金, tod.product_name 产品名称
FROM db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order_stages tos on tos.order_id=tprm.order_id
left join db_digua_business.t_order_details tod on tod.order_id=tprm.order_id
WHERE tprm.order_number in {order_list} and tos.sort=2
'''
df = query(sql)

In [ ]:
# df.to_excel('F:/需求/瑞彤需求/总赠送5折优惠卷_new.xlsx', index=False)

In [ ]:
df_es = pd.read_excel('F:/需求/二手手机/22年-23年二手机型.xlsx')
df_es = df_es[df_es['订单号'].str.contains('A2023')]


In [ ]:
df_es_group = df_es.groupby(['型号', '内存']).agg({'订单号': 'count', '售价': 'sum', '采购成本': 'sum', '官网价': 'sum'})
df_es_group['回收价'] = df_es_group['售价'] / df_es_group['订单号']
df_es_group['采购价'] = df_es_group['采购成本'] / df_es_group['订单号']
df_es_group['官网价均值'] = df_es_group['官网价'] / df_es_group['订单号']
# df_es_group

# 商家

In [ ]:
# 黑名单
sql4 = '''
SELECT
id_card, mobile
FROM db_credit.blacklist b
WHERE b.end_time is NULL or b.end_time>SYSDATE() -- 结束时间为空且大于当前时间，则为黑名单
'''
df4 = query(sql4)

In [ ]:
# 处理mobile字段并修改字段类型
df4 = df4[df4.mobile.str.contains(r'\d+')]
df4['id_card'] = df4['id_card'].astype(str)
df4['mobile'] = df4['mobile'].astype(int)

In [ ]:
# 导入Excel文件
df_customer = pd.ExcelFile('F:/需求/总需求/商家/客户名单.xlsx')
df_customer1 = df_customer.parse(sheet_name='三要素')
df_customer2 = df_customer.parse(sheet_name='二要素')

In [ ]:
df_customer1['now_date'] = pd.to_datetime(datetime.now().strftime('%Y-%m-%d'))
df_customer1

In [ ]:
# 三要素 匹配身份证号和手机号
df_customer1['身份证'] = df_customer1['身份证'].astype(str)
df_customer_list = df_customer1.身份证.to_list()
df4[df4.id_card.isin(df_customer_list)]
# 匹配手机号
df_customer_mobile_list = df_customer1.手机号.to_list()
df4[df4.mobile.isin(df_customer_mobile_list)]

In [ ]:
# 匹配手机号
df_customer_mobile_list2 = df_customer2.mobile.to_list()
df4[df4.mobile.isin(df_customer_mobile_list2)]

## 年龄分布

In [ ]:
# 定义一个函数来计算年龄
def calculate_age(id_card, order_time):
    birth_year = int(id_card[6:10])
    birth_month = int(id_card[10:12])
    birth_day = int(id_card[12:14])
    birth_date = datetime(birth_year, birth_month, birth_day)
    age = order_time.year - birth_date.year - ((order_time.month, order_time.day) < (birth_date.month, birth_date.day))
    return age

# 应用函数到 DataFrame
df_customer1['age'] = df_customer1.apply(lambda row: calculate_age(row['身份证'], row['now_date']), axis=1)


In [ ]:
# 定义年龄区间
age_bins = [-999, 18, 20, 23, 25, 30, 35, 40, 45, float('inf')]
age_labels = ['[-999,18)','[18,20)',  '[20,23)', '[23,25)', '[25,30)', '[30,35)', '[35,40)', '[40,45)', '[45+)']

# 使用 pd.cut 设置年龄区间
df_customer1['age_group'] = pd.cut(df_customer1['age'], bins=age_bins, labels=age_labels, right=True, include_lowest=False)


In [ ]:
df_customer1[df_customer1.age<=26].shape

In [ ]:
# 三要素
df_age = df_customer1.groupby('age_group').agg(数量=('身份证', 'count'))
df_age.to_excel('F:/需求/总需求/商家/年龄分布1.xlsx')

# 交易流水汇总

In [ ]:
df_ls = pd.read_excel('F:/需求/七月需求/无标题(14).xlsx')

In [ ]:
df_ls.loc[:, '交易发起日期'] = df_ls.交易发起时间.dt.date

In [ ]:
# df_ls.groupby('交易发起日期').agg(总应付金额=('应付金额', 'sum'), 总实付金额=('实付金额（如果为空则应付=实付）', 'sum')).to_excel('F:/需求/七月需求/交易流水汇总.xlsx')

In [ ]:
df_ls2 = pd.read_excel('F:/需求/七月需求/无标题(15).xlsx')

In [ ]:
def pay_way(way):
    if way == 'WX':
        return '微信'
    elif way == 'WXXCX':
        return  "微信小程序"
    elif way == 'ZFB':
        return '支付宝'
    elif way == 'ZFBYSQ':
        return '支付宝预授权'
    elif way == 'ZFBXCX':
        return '支付宝小程序'
    elif way == 'DYXCX':
        return '抖音小程序'
    elif way == 'DY_HX_PAY':
        return '抖音小程序核销'
    elif way == 'YE':
        return '余额'
    elif way == 'WXZFF':
        return '微信支付分'
    elif way == 'ZFBZQQ':
        return '支付宝周期扣款'
    elif way == 'XX':
        return '线下'
    elif way == 'UNIONXX':
        return '银联线下'
    elif way == 'DEPOSIT':
        return '押金抵扣'
    elif way == 'RY_PAY':
        return '第三方代扣'
    elif way == 'ANT_PAY':
        return '蚂蚁链代扣'
    elif way == 'ZYX_PAY':
        return '智赢行支付'
    elif way == 'ZFB_WAP_PAY':
        return '支付宝h5'
    elif way == 'ZFBYWD':
        return '支付宝业务单代扣'
    elif way == 'ZFXXSK':
        return '线下（对资方）支付'
    elif way == 'XXX':
        return '线下（对维客）支付'
    elif way == 'XX_MERCHANT':
        return '线下（对商家）支付'
    elif way == 'UNICOM_PAY':
        return '联通号卡支付'

In [ ]:
df_ls2.loc[:, '支付方式_中文'] = df_ls2.apply(lambda x: pay_way(x['支付方式']), axis=1)
# df_ls2.to_excel('F:/需求/七月需求/支付宝交易流水.xlsx')

In [ ]:
df_ls2.loc[:, '交易发起日期'] = df_ls2.交易发起时间.dt.date
df_ls2_3 = df_ls2[df_ls2.支付方式_中文.isin(['支付宝小程序', '支付宝业务单代扣', '支付宝预授权'])]
df_ls2_3_group = df_ls2_3.groupby('交易发起日期').agg(总应付金额_前三=('应付', 'sum'), 总实付金额_前三=('实付', 'sum'))
df_ls2_4 = df_ls2[df_ls2.支付方式_中文.isin(['支付宝小程序', '支付宝业务单代扣', '支付宝预授权', '智赢行支付'])]
df_ls2_4_group = df_ls2_4.groupby('交易发起日期').agg(总应付金额_后四=('应付', 'sum'), 总实付金额_后四=('实付', 'sum'))
# df_ls2_group = df_ls2_3_group.merge(df_ls2_4_group, on='交易发起日期', how='inner').to_excel('F:/需求/七月需求/支付方式交易汇总.xlsx')

In [ ]:
df_ls2_3_group


# 匹配云商分

In [ ]:
# trace关联
sql_trace='''-- trace关联表 
SELECT order_trace_id, trace_id, order_id, order_number, risk_trace_id
from db_credit.order_association
;
'''
trace_df = query(sql_trace)
# 云商分
sql1='''-- 风控-百融贷前-云商分
-- 101331	101331	101331
SELECT trace_id, query_rs
from db_credit.risk_bairong_scoreysstd
;
'''
br_ys_df =query(sql1)
br_ys_df['云商分'] = br_ys_df['query_rs'].apply(lambda x: json.loads(x).get('Score', {}).get('scoreysstd', None))
br_ys_df['云商分'] = br_ys_df['云商分'].astype(int)
df_rule = trace_df.merge(br_ys_df[['trace_id', '云商分']], on='trace_id', how='outer')


In [ ]:
df_y = pd.read_excel('F:/租后表分析数据_ld/tmp(1).xlsx')

In [ ]:
# df_y[['order_id', 'order_number', '拒绝理由']].merge(df_rule[['order_id', '云商分']], on='order_id', how='left').to_excel('F:/需求/总需求/强拒云商分.xlsx', index=False)

In [ ]:
# （26）年龄、性别、总租金
# 定义一个函数来计算年龄
def calculate_age1(id_card, order_time):
    birth_year = int(id_card[6:10])
    birth_month = int(id_card[10:12])
    birth_day = int(id_card[12:14])
    birth_date = datetime(birth_year, birth_month, birth_day)
    age = order_time.year - birth_date.year - ((order_time.month, order_time.day) < (birth_date.month, birth_date.day))
    return age

# 定义一个函数来判断性别
def get_gender(id_card):
    gender_digit = int(id_card[-2])  # 第17位数字
    return '男' if gender_digit % 2 != 0 else '女'

# 取基本信息
sql = '''
SELECT  om.order_number, om.create_time
,om.all_money as all_rental
,tmu.id_card_num
,tod.actual_money

from db_digua_business.t_order om
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on om.id = tod.order_id
-- where date_format(om.create_time, '%Y-%m-%d') >= '2024-11-01'
;
'''

df_zhys = query(sql)

# df_zhys.dtypes
df_zhys['下单日期'] = df_zhys['create_time'].dt.date
df_zhys['order_number'] = df_zhys['order_number'].astype(str)
# 取样本
path = 'F:/租后表分析数据_ld/'
data_model = pd.read_excel(path + 'tmp(2).xlsx')
data_model['order_number'] = data_model['order_number'].astype(str)
# data_model


In [ ]:
# 关联
data_model = data_model.merge(df_zhys, on = 'order_number', how = 'left')
data_model.shape
data_model[:2]

In [ ]:

data_model1 = data_model[data_model.id_card_num.isna()]
data_model2 = data_model[data_model.id_card_num.notna()]
# data_model = data_model.apply(lambda x: get_gender(x.id_card_num), axis = 1)
data_model2['gender'] = data_model2['id_card_num'].apply(get_gender)
data_model2['age'] = data_model2.apply(lambda row: calculate_age1(row['id_card_num'], row['下单日期']), axis=1)

data_model2[:2]

data_model3 = pd.concat([data_model2, data_model1])
data_model3.to_excel('F:/需求/总需求/性别年龄.xlsx', index=False)

# 拒绝理由

In [ ]:
df_reject = pd.read_excel('F:/租后表分析数据_ld/riskdemo2.xlsx')
df_rejected = df_reject[df_reject.强拒结果==1]
# 前置风控
df_rejected_qz = df_rejected[df_rejected.前置风控.notna()]
df_rejected_qz.loc[:, 'rejected'] = df_rejected_qz.前置风控.apply(lambda x: jsonpath(json.loads(x), '$..rejected')[0])
df_rejected_qz = df_rejected_qz[df_rejected_qz.rejected.astype(str)!='[]']
# 前置风控
df_rejected_hz = df_rejected[df_rejected.后置风控.notna()]
df_rejected_hz.loc[:, 'rejected'] = df_rejected_hz.后置风控.apply(lambda x: jsonpath(json.loads(x), '$..rejected')[0])
df_rejected_hz = df_rejected_hz[df_rejected_hz.rejected.astype(str)!='[]']

# 出库前风控
df_rejected_ck = df_rejected[df_rejected.出库前风控.notna()]
df_rejected_ck.loc[:, 'rejected'] = df_rejected_ck.出库前风控.apply(lambda x: jsonpath(json.loads(x), '$..rejected')[0])
df_rejected_ck = df_rejected_ck[df_rejected_ck.rejected.astype(str)!='[]']
df_rejected_ck.rejected.drop_duplicates()
# df_concat = pd.concat([df_rejected_qz[['手机号', 'rejected']], df_rejected_hz[['手机号', 'rejected']], df_rejected_ck[['手机号', 'rejected']]])
# df_concat = df_concat.explode('rejected')
# pd.crosstab('2024-01-21', df_concat['rejected'], margins=True).to_excel('F:/需求/总需求/拒绝理由.xlsx')

In [ ]:

df_rejected_qz.rejected.drop_duplicates()

In [ ]:
df_rejected_hz.rejected.drop_duplicates()

# 仕程需求

In [ ]:
# df_fp = pd.read_excel('F:/无标题(2).xlsx')
# df_fp = df_fp[~df_fp.agent_user_name.isna()]
# df_fp.loc[:, 'sort'] = df_fp.sort.str.strip('[]')
# df_fp.loc[:, '还款'] = np.where(df_fp.status==3, 1, 0)
# df_fp.loc[:, '未还款'] = np.where(df_fp.status!=3, 1, 0)
# df_fp_g = df_fp.groupby(['agent_user_name', 'sort']).agg({'order_id': 'size', '还款': 'sum', '未还款': 'sum'}).reset_index()
# df_fp_g = df_fp_g.rename(columns={'agent_user_name': '名称', 'sort': '期数', 'order_id': '总任务'})
# df_fp_g.loc[:, '催回率'] = (df_fp_g.还款/df_fp_g.总任务).map(lambda x: format(x, '.2%'))
# # with pd.ExcelWriter('F:/需求/总需求/还款情况.xlsx', engine='xlsxwriter') as writer:
#     # df_fp_g.to_excel(writer, sheet_name='还款情况', index=False)


In [ ]:
# df_6month = pd.read_excel('F:/需求/仕程需求/6月起失联名单.xlsx')
# order_list = tuple(df_6month.订单号.to_list())
# sql_6month = f'''
# select
# om.order_number, ymos.refund_date, ymos.reality_refund_date, ymos.sort, tprm.rembursement_status, om.status
# from db_digua_business.t_order om 
# left join db_rent.ya_merchant_order_stages ymos on om.id=ymos.order_id
# left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
# where om.order_number in {order_list}
# '''
# df_month = query(sql_6month)
# now_day = datetime.now().strftime('%Y-%m-%d')
# df_month = df_month[((df_month.refund_date<now_day)&(df_month.reality_refund_date.isna()))|((df_month.rembursement_status=='已逾期')&(df_month.status==5))]
# df_month.drop_duplicates(subset=['order_number'], keep='last', inplace=True)
# month_list = df_month.order_number.to_list()
# df_6month.loc[:, '是否逾期'] = np.where(df_6month.订单号.isin(month_list), 1, 0)
# df_6month = df_6month.merge(df_month[['sort', 'order_number']], left_on='订单号', right_on='order_number', how='left')
# df_6month.iloc[:, :-1].to_excel('F:/需求/仕程需求/6月起失联名单_new.xlsx', index=False)

# 力哥

In [ ]:
# df  = pd.read_excel('F:/需求/力哥需求/4月份呼入.xlsx', sheet_name='Sheet1')
# df.loc[:, '分钟'] = df.通话时长.str[3:5]
# df.loc[:, '秒'] = df.通话时长.str[-2:]
# df.to_excel('F:/需求/力哥需求/4月份呼入_新.xlsx', index=False)

# 资方数据

In [ ]:
sql_zf = '''
SELECT tprm.order_number, om.merchant_name,tod.sku_attributes, tso.price as 碎屏险价格,
tod.new_actual_money '应收金额', tprm.purchase_amount '采购成本', (tprm.rent_received) '已收租金', tprm.overdue_type, top.residue_money '押金'
,case when ym.`name`='维客自营账户' then '维客自营账户' 
			when ym.`name`='海南鸿丰商业保理有限公司' then '海南鸿丰商业保理有限公司'
			when ym.`name`='成都越蓉金科科技有限公司' then '成都越蓉金科科技有限公司'
			when ym.`name`='杭州金锤网络信息服务有限公司' then '杭州金锤网络信息服务有限公司'
			else '其他' end '资方'
FROM db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
left join (SELECT order_id, residue_money from db_digua_business.t_order_pledge where residue_money>=0) top on top.order_id=om.id
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id
where om.`status`=4
'''
df_zf = query(sql_zf)

In [ ]:
df_zf = df_zf[df_zf.采购成本>0]
df_zf = clean.drop_merchant(df_zf)
df_zf = clean.drop_rejected_merchant(df_zf)
df_zf.loc[:, '押金'] = df_zf.押金.fillna(0)
df_zf.loc[:, '已收租金'] = df_zf.已收租金+df_zf.押金
df_zf.loc[:, '应收未收金额'] = df_zf.应收金额-df_zf.已收租金
df_zf.loc[:, '应收未收采购成本'] = df_zf.采购成本-df_zf.已收租金
df_zf.loc[:, '应收未收中未逾期的金额'] = np.where(df_zf.overdue_type=='', df_zf.应收未收金额, 0)
df_zf.loc[:, '应收未收中DPD1-30的金额'] = np.where(df_zf.overdue_type=='M0', df_zf.应收未收金额, 0)
df_zf.loc[:, '应收未收中DPD31-60的金额'] = np.where(df_zf.overdue_type=='M1', df_zf.应收未收金额, 0)
df_zf.loc[:, '应收未收中DPD61+的金额'] = np.where(df_zf.overdue_type.isin(['M2', 'M3', 'M4', 'M5', 'M6']), df_zf.应收未收金额, 0)
df_zf.loc[:, '应收未收中未逾期的采购成本'] = np.where(df_zf.overdue_type=='', df_zf.应收未收采购成本, 0)
df_zf.loc[:, '应收未收中DPD1-30的采购成本'] = np.where(df_zf.overdue_type=='M0', df_zf.应收未收采购成本, 0)
df_zf.loc[:, '应收未收中DPD31-60的采购成本'] = np.where(df_zf.overdue_type=='M1', df_zf.应收未收采购成本, 0)
df_zf.loc[:, '应收未收中DPD61+的采购成本'] = np.where(df_zf.overdue_type.isin(['M2', 'M3', 'M4', 'M5', 'M6']), df_zf.应收未收采购成本, 0)
df_zf_group = df_zf.groupby('资方').agg({'应收金额': 'sum', '应收未收金额': 'sum', '应收未收中未逾期的金额': 'sum', '应收未收中DPD1-30的金额': 'sum', '应收未收中DPD31-60的金额': 'sum', 
                            '应收未收中DPD61+的金额': 'sum', '采购成本': 'sum', '应收未收采购成本': 'sum', '应收未收中未逾期的采购成本': 'sum', '应收未收中DPD1-30的采购成本' : 'sum',
                            '应收未收中DPD31-60的采购成本': 'sum', '应收未收中DPD61+的采购成本': 'sum', '碎屏险价格': 'sum'})
df_zf_group.sort_values('资方', ascending=False).to_excel('F:/需求/总需求/资方数据.xlsx')

In [ ]:
df_zf[(df_zf.应收未收中未逾期的采购成本 >= 0)&(df_zf.资方 == '维客自营账户')].应收未收中未逾期的采购成本.sum()
df_zf[(df_zf.应收未收中未逾期的采购成本 < 0)&(df_zf.资方 == '维客自营账户')].应收未收中未逾期的采购成本.sum()

## 资方数据鸿丰

In [93]:
# '海南鸿丰商业保理有限公司', 
sql_zf2 = '''
select
om.id, om.status, ym.name, om.create_time, om.user_mobile
from db_digua_business.t_order om
left join db_rent.ya_merchant ym on ym.shop_id=om.merchant_id
where ym.name in ('杭州金锤网络信息服务有限公司')
# -- and DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2024-09-26'
# -- and DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2024-10-05'
'''
df_zf2 = clean.query(sql_zf2)

In [100]:
sql_zf2 = '''
select
om.id, om.status, tm.name, om.create_time, om.user_mobile, om.id order_id, tmu.id_card_num,om.order_number,
case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2
from db_digua_business.t_order om
left join db_digua_business.t_merchant tm on tm.id=om.merchant_id
left join db_digua_business.t_member_user tmu on tmu.id=om.user_id
where tm.name in ('杭州金锤网络信息服务有限公司', '海南鸿丰商业保理有限公司') 

-- and om.status = 4
'''
df_zf2 = clean.query(sql_zf2)

In [101]:
# 杭州金锤  2024年9月26日至10月5日的订单，租赁中状态 订单号明细  2,3,4,5,6,8,15
df_zf2["下单日期"]=df_zf2["create_time"].dt.date
df_zf2["下单日期"]=pd.to_datetime(df_zf2["下单日期"],errors="coerce")
df_zf2.loc[:, '下单月份'] = df_zf2.create_time.dt.strftime('%Y-%m')
# 按下单日期与手机号码去重
df_zf2.drop_duplicates(subset=['下单日期', 'id_card_num'], keep='last', inplace=True)
df_zf2.drop_duplicates(subset=["order_id"], inplace=True)
df_zf2.loc[:, '进件'] = np.where(df_zf2.status.isin([1,13]), 0, 1)
# 删除订单状态空值行
df_zf2.dropna(subset=["status"], axis=0, inplace=True)
# 删除身份证空值行
df_zf2.dropna(subset=["id_card_num"], axis=0, inplace=True)
df_zf2.loc[:, '出库'] = np.where(df_zf2.status.isin([2,3,4,5,6,8,15]), 1, 0)
df_zf2_group = df_zf2.groupby('下单月份').agg({'进件': 'sum', '出库': 'sum'})
df_zf2_group.loc['合计', :] = df_zf2_group.sum(axis=0)
df_zf2_group#.to_excel('F:/需求/七月需求/资方数据_金锤和鸿丰.xlsx')
#df_zf2.rename(columns={'status2': '订单状态', 'order_number': '订单号'}, inplace=True)
# df_zf2[['下单日期','订单状态', '订单号']].to_excel('F:/需求/瑞彤需求/资方数据金锤订单明细.xlsx', index=False)

,进件,出库
下单月份,,
2024-08,9.0,7.0
2024-09,667.0,625.0
2024-10,239.0,231.0
2024-11,467.0,459.0
2024-12,778.0,740.0
2025-01,121.0,117.0
2025-02,88.0,84.0
2025-03,319.0,297.0
2025-04,401.0,392.0


# 财务数据

In [ ]:
def cw(month):
    sql_cw = f'''
    SELECT
    om.order_number 订单号, ym.name 名称, ymos.sort 期数, ymos.money 租金
    from db_digua_business.t_order om 
    left join db_rent.ya_merchant ym on om.merchant_id=ym.shop_id
    left join db_rent.ya_merchant_order_stages ymos on ymos.order_id=om.id
    where om.status=4
    and ym.shop_id in (14, 36, 148)
    and date_format(ymos.refund_date, '%Y-%m')='{month}'
    '''
    df_cw = query(sql_cw)
    df_cw_g = df_cw.groupby('名称').agg({'租金': 'sum'}).rename(columns={'租金': '应收租金'}).rename_axis('归属')
    df_cw_g.loc['合计',:] = df_cw_g.应收租金.sum()
    return df_cw, df_cw_g

# df_cw.to_excel('F:/需求/总需求/财务数据.xlsx', index=False)

In [ ]:
month = 5
# df_cw1, df_cw1_g = cw('2025-01')
# df_cw2, df_cw2_g = cw('2025-02')
# df_cw3, df_cw3_g = cw('2025-03')
df_cw5, df_cw5_g = cw('2025-05')
with pd.ExcelWriter(f'F:/需求/总需求/财务数据_{month}月.xlsx', engine='xlsxwriter') as writer:
    # df_cw1.to_excel(writer, sheet_name='一月份明细', index=False)
    # df_cw1_g.to_excel(writer, sheet_name='一月份汇总')
    # df_cw2.to_excel(writer, sheet_name='二月份明细', index=False)
    # df_cw2_g.to_excel(writer, sheet_name='二月份汇总')
    # df_cw3.to_excel(writer, sheet_name='三月份明细', index=False)
    # df_cw3_g.to_excel(writer, sheet_name='三月份汇总')
    df_cw5.to_excel(writer, sheet_name='五月份明细', index=False)
    df_cw5_g.to_excel(writer, sheet_name='五月份汇总')

In [ ]:
sql_qg = '''
select 
tprm.relet_periods,tod.new_actual_money,tprm.rent_received,
tprm.overdue_periods,tprm.total_receivable, tm.name
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_merchant tm on tm.id=om.merchant_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
where tm.`name` in ('南宁微美汇', '广州捷瑞环保科技')
'''
df_qg = query(sql_qg)


In [ ]:
df_qg.loc[:, '续租剩余总未收'] = np.where(df_qg.relet_periods>0, df_qg.new_actual_money-df_qg.rent_received, 0)
df_qg.loc[:, '逾期总应收'] = np.where(df_qg.overdue_periods>0, df_qg.total_receivable, 0)
df_qg.groupby('name').agg({'续租剩余总未收': 'sum', '逾期总应收': 'sum'}).to_excel('F:/需求/奇哥需求/商家数据.xlsx')

# 廖志彧需求

## 促买断

In [ ]:
# 按结束时间买断订单数
sql_lzy = '''
select
ybt.order_id 订单ID, tod.new_actual_money 买断价, too.discount_money 优惠金额, (tod.new_actual_money-too.discount_money) 实收买断金额,
tprm.purchase_amount 采购金额, ybt.`status`, ybt.follow_log_tag, ybt.type, om.has_actual, date_format(ybt.end_time, '%Y-%m') 月份
from db_rent.ya_buyout_task ybt
left join (select order_id, discount_money from db_digua_business.t_order_out where status=2) too on too.order_id=ybt.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=too.order_id
left join db_digua_business.t_order_details tod on tod.order_id=too.order_id
left join db_digua_business.t_order om on om.id=ybt.order_id
where date_format(ybt.end_time, '%Y-%m')='2025-05'
'''
# 提前买断订单对应期数
sql_stages = '''
select order_id 订单ID, sort 期数 from db_rent.ya_merchant_order_stages where date_format(refund_date, '%Y-%m')='2025-05'
'''
# 按发布时间买断订单数
sql_amd = '''
select ybt.order_id 发布时间订单号, tod.new_actual_money 发布时间买断价, date_format(ybt.add_time, '%Y-%m') 月份 from db_rent.ya_buyout_task ybt 
left join db_digua_business.t_order_details tod on tod.order_id=ybt.order_id
where date_format(ybt.add_time, '%Y-%m')='2025-05'
'''
df_amd = query(sql_amd)

df_amd_g = df_amd.groupby('月份').agg({'发布时间订单号': 'count', '发布时间买断价': 'sum'})
df_lzy = query(sql_lzy)
df_stages = query(sql_stages)

# 发布买断订单数、买断订单数、发布买断金额（采购金额）、发布买断金额（买断价）、实收买断金额、实际买断订单数、买断优惠券总额、发放优惠前的毛利率、发放优惠后的毛利率

In [ ]:
df_lzy.loc[:, '实际买断订单数'] = np.where((df_lzy.status.isin([3, 4]))&(df_lzy.has_actual==1), 1, 0)
df_lzy.loc[:, '实收买断金额'] = np.where((df_lzy.status.isin([3, 4]))&(df_lzy.has_actual==1), df_lzy.实收买断金额, 0)
def md(df):
    df_g = df.groupby('月份').agg({'订单ID': 'count', '采购金额': 'sum', '买断价': 'sum', '实际买断订单数': 'sum', '实收买断金额': 'sum', '优惠金额': 'sum'}).rename(columns={'订单ID': '发布买断订单数'})
    df_g.loc[:, '优惠比例'] = (df_g.优惠金额/df_g.采购金额).map(lambda x: format(x, '.2%'))
    df_g.loc[:, '发放优惠前的毛利率'] = ((df_g.买断价-df_g.采购金额)/df_g.采购金额).map(lambda x: format(x, '.2%'))
    df_g.loc[:, '发放优惠后的毛利率'] = ((df_g.实收买断金额-df_g.采购金额)/df_g.采购金额).map(lambda x: format(x, '.2%'))
    return df_g
df_lzy_ts = md(df_lzy[(df_lzy.status.isin([3, 4]))&(df_lzy.follow_log_tag==4)&(df_lzy.has_actual==1)])
# '任务类型ybt.type，0：自动发布，1手动创建',
df_lzy_tq = df_lzy[(df_lzy.status.isin([3, 4]))&(df_lzy.follow_log_tag!=4)&(df_lzy.type==1)&(df_lzy.has_actual==1)]
df_lzy_tq2 = md(df_lzy_tq)
df_lzy_tq3 = df_lzy_tq.merge(df_stages, on='订单ID', how='left')
df_lzy_dq = df_lzy[(df_lzy.status.isin([3, 4]))&(df_lzy.follow_log_tag!=4)&(df_lzy.type==0)&(df_lzy.has_actual==1)]
# df_lzy_dq = md(df_lzy_dq)
# df_lzy_dq2 = df_lzy_dq.merge(df_amd_g, on='月份', how='left')
# with pd.ExcelWriter('F:/需求/廖志彧需求/5月买断订单.xlsx', engine='xlsxwriter') as writer:
#     df_lzy_ts.to_excel(writer, sheet_name='特殊申请')
#     df_lzy_tq2.to_excel(writer, sheet_name='提前买断')
#     df_lzy_tq3.to_excel(writer, sheet_name='提前买断明细', index=False)
#     df_lzy_dq.to_excel(writer, sheet_name='到期买断')
# df_lzy.to_excel('F:/需求/廖志彧需求/5月买断订单.xlsx', index=False)&(df_lzy.has_actual==1)

## 优召

In [47]:
sql_yz = '''
select
om.id order_id, om.create_time 下单时间,turt.discount_status, turt.id_card_num,tdi.money tdi_money,om.order_number ,turt.create_time 优召时间,turt.complete_time
,tdi.use_type, tdi.type, tdi.create_time 优惠订单创建时间
, tprm.all_rental, tprm.all_deposit, tprm.purchase_amount, om.status, tod.new_actual_money, date_format(turt.complete_time, '%Y-%m') 月份
, tos.money 分期金额
from db_digua_business.t_user_recall_task turt 

left join db_digua_business.t_order om on om.order_number=turt.order_number
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_number=turt.order_number
left join db_digua_business.t_discount_item tdi on tdi.user_id=om.user_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
left join db_digua_business.t_order_stages tos on tos.order_id=om.id
where date_format(turt.complete_time, '%Y-%m-%d')>='2025-09-01'
and date_format(turt.complete_time, '%Y-%m-%d')<='2025-09-30'

'''
# and date_format(om.create_time, '%Y-%m-%d') >= date_format(turt.create_time, '%Y-%m-%d')
# and date_format(om.create_time, '%Y-%m-%d') <= date_format(turt.complete_time, '%Y-%m-%d')
# and turt.order_number is not null
df_yz = clean.query(sql_yz)
df_yz.shape

(5953, 18)

In [ ]:
df_yz_n = df_yz.sort_values('优惠订单创建时间', ascending=False).groupby('id_card_num').head(1)
df_yz_n.shape

(80, 18)

In [ ]:
# 检查订单重复
df_yz_n = df_yz.drop_duplicates(subset=['order_id'])
df_yz_n.shape

(24, 18)

In [48]:
# 剔除status=10的订单
# df_yz = df_yz[df_yz.status!=10]
# df_yz = df_yz.drop_duplicates(subset=['id_card_num'])
# 按id_card_num分组创建时间降序排序取第一个
df_yz.loc[:,'最大分期金额'] = df_yz.groupby('order_id')['分期金额'].transform('max').fillna(0)
df_yz = df_yz.sort_values('优惠订单创建时间', ascending=False).groupby('id_card_num').head(1)
# df_yz = df_yz.drop_duplicates(subset=['id_card_num'])
df_yz["下单日期"]=df_yz["下单时间"].dt.date
df_yz["下单日期"]=pd.to_datetime(df_yz["下单日期"],errors="coerce")
df_yz["优召日期"]=df_yz["优召时间"].dt.date
df_yz["优召日期"]=pd.to_datetime(df_yz["下单日期"],errors="coerce")
df_yz["完成日期"]=df_yz["complete_time"].dt.date
df_yz["完成日期"]=pd.to_datetime(df_yz["下单日期"],errors="coerce")

df_yz.loc[:, '进件订单数'] = np.where(((df_yz.status==1)|(df_yz.status==13))&((df_yz.下单日期>=df_yz.优召日期)&(df_yz.下单日期<=df_yz.完成日期)), 0, 1)
df_yz.loc[:, '出库订单数'] = np.where((df_yz.order_number.notna()), 1, 0)# (df_yz.status.isin([2, 3, 4, 5, 6, 8, 15]))&
# '优惠卷类型type：1、商品满减券；2、首期优惠券；3、商品延期券；4、商品买断券', 5、租中优惠券
# 仅考虑type=5的情况当前优惠券状态为Y，并且优惠券类型为商品满减券（use_type=1）优惠金额等于优惠券金额（tdi_money），优惠券类型为租中优惠券（use_type=2）时，优惠金额等于订单金额乘以（1-tdi_money/100）
# 

df_yz.loc[:, '优惠券金额'] = np.where(
    (df_yz.discount_status == 'Y') & (df_yz.use_type == 1) & (df_yz.type==5), df_yz.tdi_money,
    np.where((df_yz.discount_status == 'Y') & (df_yz.use_type == 2) & (df_yz.type==5), df_yz.最大分期金额 * (1 - df_yz.tdi_money / 100), 0)
)
df_yz.loc[:, '买断价'] = np.where(df_yz.出库订单数==1, df_yz.new_actual_money, 0)
df_yz.loc[:, '采购成本'] = np.where(df_yz.出库订单数==1, df_yz.purchase_amount, 0)
df_yz_g = df_yz.groupby('月份').agg({'进件订单数': 'sum', '出库订单数': 'sum', '优惠券金额': 'sum', '采购成本': 'sum', 'all_rental': 'mean', 'all_deposit': 'mean', '买断价': 'sum'}).rename(columns={'all_rental': '总租金均值', 'all_deposit': '总押金均值'})
df_yz_g.loc[:, '租售比'] = (df_yz_g.总租金均值/df_yz_g.总押金均值).map(lambda x: format(x, '.2%'))
df_yz_g.loc[:, '优惠比例'] = (df_yz_g.优惠券金额/df_yz_g.采购成本).map(lambda x: format(x, '.2%'))
df_yz_g.loc[:, '优惠前毛利率'] = ((df_yz_g.买断价-df_yz_g.采购成本)/df_yz_g.采购成本).map(lambda x: format(x, '.2%'))
df_yz_g.loc[:, '优惠后毛利率'] = ((df_yz_g.买断价-df_yz_g.采购成本-df_yz_g.优惠券金额)/df_yz_g.采购成本).map(lambda x: format(x, '.2%'))
# 发布优召订单数、进件订单数、出库订单数、优惠券金额、采购成本、租售比
# with pd.ExcelWriter('F:/需求/廖志彧需求/5月优召订单.xlsx', engine='xlsxwriter') as writer:
#     df_yz_g.to_excel(writer, sheet_name='优召')
df_yz_g

,进件订单数,出库订单数,优惠券金额,采购成本,总租金均值,总押金均值,买断价,租售比,优惠比例,优惠前毛利率,优惠后毛利率
月份,,,,,,,,,,,
2025-09,146,27,3079.90,244119.0,7271.831481,10036.037037,350860.42,72.46%,1.26%,43.73%,42.46%
2025-10,90,16,2640.34,132130.0,8265.252500,9355.250000,197723.98,88.35%,2.00%,49.64%,47.65%


In [ ]:
df_yz_g

,进件订单数,出库订单数,优惠券金额,采购成本,总租金均值,总押金均值,买断价,租售比,优惠比例,优惠前毛利率,优惠后毛利率
月份,,,,,,,,,,,
2025-07,80,23,4143.42,180998.0,7352.185652,9681.36087,278700.8,75.94%,2.29%,53.98%,51.69%


In [ ]:
# 80	57	23	5481.13	180998.0	7352.185652	9681.36087	277083.59	75.94%	3.03%	53.09%	50.06%
df_yz[(df_yz.discount_status=='Y')&(df_yz.type.isin([1, 5]))].shape#25	

(16, 25)

In [ ]:
df_yz[(df_yz.order_number.notna())].shape#.采购成本.sum()

(23, 25)

In [ ]:
df_yz[(df_yz.优惠券金额>0)].type.value_counts()#320684200110287411

type
5.0    30
Name: count, dtype: int64

## 挽留

In [39]:
sql_wl = '''
select  
tort.order_id, tprm.purchase_amount, tod.new_actual_money, tdi.money tdi_money, tdi.use_type, om.status, 
date_format(tort.create_time, '%Y-%m') 月份, tort.task_status,tdi.create_time 优惠订单创建时间,
tos.money 分期金额
from db_digua_business.t_order_retention_task tort
left join db_digua_business.t_order om on om.id=tort.order_id
left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
left join db_digua_business.t_discount_item tdi on tdi.discount_id=tort.discount_id
left join db_digua_business.t_order_details tod on tod.order_id=om.id
left join db_digua_business.t_order_stages tos on tos.id=tort.discount_stage_id
where tdi.type = 5
-- and date_format(tort.create_time, '%Y-%m-%d')>='2025-07-01'
-- and  DATE_FORMAT(tort.create_time, '%Y-%m-%d')>='2025-06-01'
-- 仅有租中优惠劵
-- and tdi.type = 5
'''
df_wl = clean.query(sql_wl)
df_wl.shape

(1022, 10)

In [40]:

df_wl.月份.unique()

array(['2025-03', '2025-04', '2025-05'], dtype=object)

In [ ]:
# df_wl = df_wl.sort_values('优惠订单创建时间', ascending=False).groupby('order_id').head(1)
df_wl.loc[:, '挽留成功订单数'] = np.where(df_wl.task_status=='SUCCESS', 1, 0)
df_wl.loc[:, '出库'] = np.where(df_wl.status.isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)
# use_type:'优惠券类型：1、满减；2、折扣；
df_wl.loc[:, '折扣优惠券'] = np.where(df_wl.use_type==2, df_wl.tdi_money, 0)
df_wl.loc[:, '满减优惠券'] = np.where(df_wl.use_type==1, df_wl.分期金额 * (1 - df_wl.tdi_money / 100), 0)
df_wl.loc[:, '优惠券总金额'] = df_wl.loc[:, '折扣优惠券'] + df_wl.loc[:, '满减优惠券']
df_wl.loc[:, '采购金额'] = np.where(df_wl.status.isin([2, 3, 4, 5, 6, 8, 15]), df_wl.purchase_amount, 0)
df_wl.loc[:, '买断价'] = np.where(df_wl.status.isin([2, 3, 4, 5, 6, 8, 15]), df_wl.new_actual_money, 0)
df_wl_g = df_wl.groupby('月份').agg({'order_id': 'count', '挽留成功订单数': 'sum', '出库': 'sum', '采购金额': 'sum', '买断价': 'sum', '优惠券总金额': 'sum', '折扣优惠券': 'sum', '满减优惠券': 'sum'}).rename(columns={'order_id': '挽留发布订单数'})
df_wl_g.loc[:, '优惠比例'] = (df_wl_g.优惠券总金额/df_wl_g.采购金额).map(lambda x: format(x, '.2%'))
df_wl_g.loc[:, '满减优惠券比例'] = (df_wl_g.满减优惠券/df_wl_g.采购金额).map(lambda x: format(x, '.2%'))
df_wl_g.loc[:, '折扣优惠券比例'] = (df_wl_g.折扣优惠券/df_wl_g.采购金额).map(lambda x: format(x, '.2%'))
df_wl_g.loc[:, '发放优惠前的毛利率'] = ((df_wl_g.买断价-df_wl_g.采购金额)/df_wl_g.采购金额).map(lambda x: format(x, '.2%'))
df_wl_g.loc[:, '发放优惠后的毛利率'] = ((df_wl_g.买断价-df_wl_g.优惠券总金额-df_wl_g.采购金额)/df_wl_g.采购金额).map(lambda x: format(x, '.2%'))
df_wl_g
# 优惠券（买断优惠券、租中优惠券）总金额、挽留发布任务数、采购金额、买断价、优惠比例（优惠券/采购金额）、发放优惠前的毛利率、发放优惠后的毛利率
# with pd.ExcelWriter('F:/需求/廖志彧需求/5月挽留订单.xlsx', engine='xlsxwriter') as writer:
#     df_wl_g.to_excel(writer, sheet_name='挽留')

In [ ]:
# 检查订单重复
df_wl[df_wl.order_id.duplicated()]
# df_wl

,order_id,purchase_amount,new_actual_money,money,type,status,月份,task_status,优惠订单创建时间,挽留成功订单数,出库,买断优惠券,租中优惠券,优惠券总金额,采购金额,买断价


## 抖音电脑p站数据

In [ ]:
# 仅含代码逻辑
classify_id_list = [125,126,127,185]
df_dy = df[df['归属渠道']=='抖音渠道']
print(df_dy.shape)
df_dy_compu = df_dy[(df_dy.classify_id.isin(classify_id_list))&(df_dy.status == 4)]
print(df_dy_compu.shape)
# 获取型号
def getModel(s):
    color_list = json.loads(s)
    for j in range(0, len(color_list)):
        if color_list[j]["key"] == "型号":
            return color_list[j]["value"]

def getScreen(s):
    color_list = json.loads(s)
    for j in range(0, len(color_list)):
        if color_list[j]["key"] == "显示屏":
            return color_list[j]["value"]

def getTime(s):
    color_list = json.loads(s)
    for j in range(0, len(color_list)):
        if color_list[j]["key"] == "租赁时长":
            return color_list[j]["value"]
        
# df.loc[:,"颜色"]=df.apply(lambda x:clean.getcolor(x["sku_attributes"]),axis=1)
df_dy_compu.loc[:, '型号'] = df_dy_compu['sku_attributes'].apply(lambda x: getModel(x))
df_dy_compu.loc[:, '显示屏'] = df_dy_compu['sku_attributes'].apply(lambda x: getScreen(x))
df_dy_compu.loc[:, '租赁时长'] = df_dy_compu['sku_attributes'].apply(lambda x: getTime(x))


df_dy_compu[['order_id', '型号', '显示屏', '租赁时长']].to_excel('F:/需求/总需求/抖音电脑出库订单明细.xlsx', index=False)





## 挽留、促买断、拒量数

In [48]:
# 挽留
sql_wl = '''
select
date_format(tort.create_time, '%Y-%m') 任务月份, tort.order_id, tort.task_status, om.status, executor_user_name 执行人
from db_digua_business.t_order_retention_task tort
left join db_digua_business.t_order om on om.id=tort.order_id
where date_format(tort.create_time, '%Y-%m-%d')>='2025-07-01'
'''
df_wl = clean.query(sql_wl)
df_wl.shape

(861, 5)

In [ ]:
name_list = ['蔡斯静', '林婉婷', '胡春燕', '何薪华', '刘慧琪', '王伟鸿']
# # 筛选出库单分配人
# df_name = df_name[df_name['分配人'].isin(name_list)]
# 连接挽留表与分配人
# df_wl_merge = df_wl.merge(df_name, on='order_id', how='left')
df_wl = df_wl[df_wl['执行人'].isin(name_list)]
df_wl.loc[:, '挽留成功'] = np.where(df_wl.task_status=='SUCCESS', 1, 0)
df_wl.loc[:, '出库'] = np.where(df_wl.status.isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)
df_wl_g = df_wl.groupby(['任务月份', '执行人']).agg({'order_id': 'count', '挽留成功': 'sum', '出库': 'sum'}).rename(columns={'order_id': '任务数'})
df_wl_g

In [54]:
# 促买断
sql_lzy = '''
select
ybt.order_id , ybt.status, om.has_actual, date_format(ybt.add_time, '%Y-%m') 月份, ybt.assign_user_name, om.relet_status
from db_rent.ya_buyout_task ybt
left join db_digua_business.t_order om on om.id=ybt.order_id
where date_format(ybt.add_time, '%Y-%m-%d')>='2025-07-01'
'''
df_lzy = clean.query(sql_lzy)
df_lzy.shape

(6109, 6)

In [56]:
name_list = ['蔡斯静', '林婉婷', '胡春燕', '何薪华', '刘慧琪', '王伟鸿']
df_lzy = df_lzy[df_lzy.assign_user_name.isin(name_list)]
df_lzy.loc[:, '买断订单数'] = np.where((df_lzy.status.isin([3, 4]))&(df_lzy.has_actual==1), 1, 0)
df_lzy.loc[:, '续租数'] = np.where(df_lzy.status.isin([3, 4])&(df_lzy.relet_status==2), 1, 0)
df_lzy_g = df_lzy.groupby(['月份', 'assign_user_name']).agg({'order_id': 'count', '买断订单数': 'sum', '续租数': 'sum'}).rename(columns={'order_id': '任务数'})
df_lzy_g

任务数  买断订单数  续租数
月份      assign_user_name                 
2025-07 何薪华               306     47   82
        刘慧琪               216     28   58
        林婉婷               204     32   62
        胡春燕               204     34   50
        蔡斯静               205     37   57
2025-08 何薪华               289     52   26
        刘慧琪               251     26   27
        林婉婷               249     21   29
        王伟鸿               222     21   25
        胡春燕               257     28   26
        蔡斯静               251     23   28
2025-09 何薪华               398     27   10
        刘慧琪               335     20    9
        林婉婷               388     22   14
        王伟鸿               331     23   12
        胡春燕               330      9   11
        蔡斯静               430     19   12
2025-10 何薪华                99      7    0
        刘慧琪                93      3    1
        林婉婷                93      2    0
        王伟鸿                94      3    0
        胡春燕                92      0    0
        蔡斯静                95      4    0

In [43]:
# 分配人
sql_name = '''
SELECT tuvor.order_id, tu.nick_name 分配人, tuvor.update_time 
FROM db_digua_business.t_user_verify_order_record tuvor
left join db_digua_business.t_user tu on tuvor.user_id = tu.id 
where tuvor.del_flag = 0 ORDER BY tuvor.update_time
'''
df_name = clean.query(sql_name)

任务数  挽留成功  出库
任务月份    执行人               
2025-07 何薪华   50     5   1
        刘慧琪   59     0   0
        林婉婷   99     9   5
        胡春燕   82     6   3
        蔡斯静   83    24   7
2025-08 何薪华   28     5   4
        刘慧琪   31     2   1
        林婉婷   46     0   0
        王伟鸿   35     5   1
        胡春燕   54     6   2
        蔡斯静   86    21   3
2025-09 何薪华    7     2   2
        刘慧琪   15     0   0
        林婉婷    1     0   0
        王伟鸿   40     3   1
        胡春燕   22     2   0
        蔡斯静   25    13   1
2025-10 何薪华    5     2   1
        刘慧琪    8     0   0
        林婉婷    1     0   0
        王伟鸿   10     1   0
        胡春燕    8     1   1
        蔡斯静    5     2   0

## 匹配订单状态与订单号

In [5]:
sql_h = '''
select order_number, user_mobile,
case status when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
from db_digua_business.t_order 
where date_format(create_time, '%Y-%m-%d')>='2025-10-01'
and date_format(create_time, '%Y-%m-%d')<='2025-10-28'
'''

df_pp = clean.query(sql_h)
df_dpp = pd.read_excel('F:/需求/总需求/匹配数据，10月1号~28号下单记录显示已取消或者风控拒绝订单.xlsx')
# 在merge之前添加数据类型转换
df_dpp['phone_number'] = df_dpp['phone_number'].astype(str)
df_pp['user_mobile'] = df_pp['user_mobile'].astype(str)
df_dpp = df_dpp.merge(df_pp, left_on='phone_number', right_on='user_mobile', how='left')
df_dpp

,phone_number,order_number,user_mobile,status2
0,17631221471,NaN,NaN,NaN
1,17803699975,NaN,NaN,NaN
2,18920910358,NaN,NaN,NaN
3,16620122229,NaN,NaN,NaN
4,19353521555,A20251014215950399,19353521555,订单取消
...,...,...,...,...
250,15031785025,NaN,NaN,NaN
251,18958574663,NaN,NaN,NaN
252,19511324573,NaN,NaN,NaN
253,17382809477,NaN,NaN,NaN


In [6]:
df_dpp.to_excel('F:/需求/总需求/已匹配数据，10月1号~28号下单记录显示已取消或者风控拒绝订单.xlsx', index=False)

# 宏生需求

## 月份毛利

In [ ]:
# 支付宝、京东订单的月份毛利
sql_hs = '''
select
tprm.order_create_time, tprm.order_id 订单号, tprm.all_rental 总租金, tprm.purchase_amount 采购金额, tod.commodity_premium 商品溢价费, tod.new_actual_money 买断金额, tod.product_name, tod.sku_attributes
,om.order_type
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order_details tod on tod.order_id=tprm.order_id
left join db_digua_business.t_order om on om.id=tprm.order_id
where tprm.purchase_amount>0
and date_format(tprm.order_create_time, '%Y-%m-%d')>='2025-05-01'
and date_format(tprm.order_create_time, '%Y-%m-%d')<='2025-08-31'
'''
df_hs = clean.query(sql_hs)
df_hs.loc[:, '下单月份'] = df_hs.order_create_time.dt.strftime('%Y-%m')

In [ ]:
def getrecanzhi(s):
    if s["租赁方案"]=="租完即送":
        return  0
    elif s["残值机型"]=="苹果16":
        return s["采购金额"]*0.6
    elif s["残值机型"]=="苹果15":
        return s["采购金额"]*0.47
    elif s["残值机型"]=="苹果14":
        return s["采购金额"]*0.4
    elif s["残值机型"]=="苹果14以下":
        return s["采购金额"]*0.375
    else:
        return s["采购金额"]*0.425
df_hs.loc[:, '租赁方案'] = np.where(df_hs["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df_hs.loc[:, '残值机型'] = np.where(df_hs["product_name"].str.contains('iPhone'),  
                                np.where(df_hs["product_name"].str.contains('16'), "苹果16",
                                        np.where(df_hs['product_name'].str.contains('15'), "苹果15",
                                                np.where(df_hs['product_name'].str.contains('14'), "苹果14", '苹果14以下'))),"非苹果")
df_hs.loc[:,"残值"]=df_hs.apply(getrecanzhi,axis=1)
def ml(df_hs):
    df_hs.loc[:,"总应收"]=df_hs["总租金"]+df_hs["残值"]+df_hs["商品溢价费"]
    df_hs.loc[:,"原始毛利"]=df_hs["买断金额"]-df_hs["采购金额"]+df_hs["商品溢价费"]
    df_hs.loc[:,"残值估算毛利"]=df_hs["总应收"]-df_hs["采购金额"]
    df_hs_group = df_hs.groupby('下单月份').agg({'订单号': 'count', '总租金': 'sum', '残值': 'sum', '商品溢价费': 'sum', '总应收': 'sum', '采购金额': 'sum', '残值估算毛利': 'mean', '买断金额': 'sum', '原始毛利': 'mean'}).rename(columns={'订单号': '订单数','残值估算毛利': '残值估算毛利均值', '原始毛利': '原始毛利均值'})
    return df_hs_group
# 支付宝订单
df_zfb = df_hs[df_hs.order_type=='ZFB_ORDER']
df_zfb_group = ml(df_zfb)
# 京东订单
df_jd = df_hs[df_hs.order_type=='JD_ORDER']
df_jd_group = ml(df_jd)
# 总体订单
df_hs_group = ml(df_hs)
now_day = datetime.now().strftime('%Y%m%d')
# with pd.ExcelWriter(f'F:/宏生/月份毛利_{now_day}.xlsx', engine='xlsxwriter') as writer:
#     df_zfb_group.to_excel(writer, sheet_name='支付宝订单')
#     df_jd.to_excel(writer, sheet_name='京东订单')

In [ ]:
# df_hs.order_type.unique()
df_hs_group

,订单数,总租金,残值,商品溢价费,总应收,采购金额,残值估算毛利均值,买断金额,原始毛利均值
下单月份,,,,,,,,,
2025-05,4413,30894193.32,1.971442e+07,0.0,5.060861e+07,36630536.1,3167.477447,55656824.40,4311.418151
2025-06,5314,35545552.69,2.391412e+07,0.0,5.945967e+07,43333206.0,3034.712566,67410848.80,4530.982838
2025-07,6189,40432861.93,2.919204e+07,0.0,6.962490e+07,51260235.0,2967.307519,78692582.95,4432.436250
2025-08,4983,32762107.35,2.476535e+07,0.0,5.752746e+07,42327410.0,3050.381569,63886030.00,4326.433875


In [ ]:
# 追加到excel末尾

with pd.ExcelWriter(f'F:/宏生/月份毛利_20250805.xlsx', engine='openpyxl', mode='a') as writer:
    df_hs_group.to_excel(writer, sheet_name='Sheet')


## 续租订单匹配

In [4]:
df_hl = pd.read_excel('F:/宏生/续租分布.xlsx')
df_hl_list = df_hl['order_number'].tolist()
len(df_hl_list)
df_wxz = pd.read_excel('F:\需求\瑞彤需求\预计未续租订单的续租时长数据.xlsx', sheet_name='Sheet1')
df_wxz_list = df_wxz['order_number'].tolist()
len(df_wxz_list)
# 3136-3121=15

28757

25636

In [7]:
# 筛选df_hl中order_number不在df_wxz_list中的数据
df_hl_wxz = df_hl[~df_hl['order_number'].isin(df_wxz_list)]
df_hl_wxz.to_excel('F:/宏生/未续租匹配差异.xlsx')

In [8]:
df_wxz_hl = df_wxz[~df_wxz['order_number'].isin(df_hl_list)]
df_wxz_hl#.to_excel('F:/宏生/未续租匹配差异.xlsx')

,order_number
1330,A2024041914302179
1522,A202405291801474
3512,A2024102710271483
3554,A2024102817095148
3576,A20241029154435362
3632,A2024103117160196
4485,A20241221164845256
4501,A20241222125207210
6443,A2025031218152873
14993,A2025061518093477


# 归还逾期

In [ ]:
# tor.overdue_type = 2归还逾期
sql = '''select
om.order_number, om.status, om.has_actual, om.relet_status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order om on om.id=tprm.order_id
left join db_digua_business.t_overdue_record tor on tor.order_id=om.id
where DATE_FORMAT(om.create_time, '%Y-%m') = '2024-07'
and tor.overdue_type = 2
'''

df_gy = clean.query(sql)

In [ ]:
# path = 'F:/需求/归还逾期/归还逾期.xlsx'
# 订单状态为已完成且买断及为已买断
df_gy.loc[:,'订单状态'] = np.where((df_gy.status2=='已完成')&(df_gy.has_actual==1), '已买断', df_gy.status2)
# 剔除status2为已完成的订单
df_gy_new = df_gy[~(df_gy.status2=='已完成')]
df_gy_new#.to_excel('F:/需求/归还逾期/归还逾期.xlsx', engine='xlsxwriter')

In [ ]:
# 将列名改为中文
df_gy_new#.columns = ['订单号', '订单状态', '是否买断', '归还状态', '订单状态2']
# df_gy.status2.value_counts()
df_gy_new

In [ ]:
df_gy
# 每一天（下单日期）每个渠道的退货订单数，


# 倩姐需求

## 每个月的部分免押和非免押的订单数

In [ ]:
sql = '''
select  om.create_time, om.id as order_id, top.total_freeze_fund_amount, om.order_number
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
from db_digua_business.t_order om
left join db_digua_business.t_order_pay top on om.id=top.order_id
where top.pay_type='ZFBYSQ'
and top.`status` in (2,5)
and  DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2024-01-01'
and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-07-31 '
'''
df_order = clean.query(sql)



In [ ]:
df = df_order.copy()
# df = df[~df.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机","喜卓灵新租机"])]
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单月份"]=df["create_time"].dt.strftime('%Y-%m')

df['年份'] = df["下单日期"].dt.year
# 设置免押
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")

# 去重订单数
df=df.drop_duplicates(subset=['order_id'])

In [ ]:
# dfck = df[df['是否出库']==1]
# 按下单日期看 免押 出库单数
df_my = pd.crosstab(df["下单月份"],df["押金类型"],margins=True)
# 写入excel
# df_my.to_excel('F:/需求/倩姐需求/免押数据.xlsx', engine='xlsxwriter')
df_my

## 按月分组的业务量数据

In [ ]:
sql = '''-- 租后应收监控  
SELECT  tprm.* 
,om.`status` , om.create_time 订单创建时间
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as  status2
,om.order_finish_date as 订单完成时间,om.order_method
,tmu.true_name ,tmu.id_card_num,tmu.mobile 
,tod.product_name, tod.new_actual_money 
-- , tprm.purchase_amount, tprm.rent_received, tprm.all_rental

-- 2025.4.29
,cc.name as channel_name_cc         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,pa.type
,om.order_type
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result

from db_digua_business.t_postlease_receivables_monitoring   tprm 
left join db_digua_business.t_order om on tprm.order_id = om.id
left join  db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join  db_digua_business.t_order_details tod on tprm.order_id = tod.order_id

-- 2025.4.29
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
where DATE_FORMAT(om.create_time, '%Y-%m-%d') <='2025-07-31'
;
'''

df_zhys = clean.query(sql)
# (27787, 62)
df_zhys.shape

(60634, 73)

In [ ]:
# df_zhys["年"] = df_zhys["下单月份"].str[:4]
# pd.crosstab(dfzh1["商家名称"],dfzh1["年"],margins=True).loc[["北京海鸟窝科技有限公司","深圳优优大数据科技有限公司","租着用电脑数码","苏州蚁诺宝"],:]
# 剔除商家数据
def drop_sj(df_zhys,col1,col2):
    df_zhys.drop(df_zhys[df_zhys[col1]=="深圳优优大数据科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="优优2店"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="小豚租（代收）"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="苏州蚁诺宝"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="租着用电脑数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="北京海鸟窝科技有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="星晟数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="蘑菇时间"].index,inplace=True)

    df_zhys.drop(df_zhys[df_zhys[col1]=="汇客好租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="澄心优租"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="CPS渠道合作"].index,inplace=True)
    
    df_zhys.drop(df_zhys[df_zhys[col1]=="趣智数码"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="格木木二奢名品"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="广州康基贸易有限公司"].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys[col1]=="线下小店"].index,inplace=True)  # 计划
    df_zhys.drop(df_zhys[df_zhys[col1]=="小蚂蚁租机"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="人人享租"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="兴鑫兴通讯"].index,inplace=True)  #二手订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="崇胜数码"].index,inplace=True)  #二手订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="喜卓灵租机"].index,inplace=True)  #拒量订单
    df_zhys.drop(df_zhys[df_zhys[col1]=="喜卓灵新租机"].index,inplace=True)  #拒量订单

    df_zhys.drop(df_zhys[df_zhys[col2].str.contains(pat='探路者',regex=False)==True].index,inplace=True)
    df_zhys.drop(df_zhys[df_zhys['merchant_name'] == "乙辉数码"].index, inplace=True)
    return df_zhys

df_zhys = drop_sj(df_zhys,'merchant_name','model_number')
# df_zhys.订单创建时间.unique()

In [ ]:
# 我们业务开始每个月的出库数据进件数据，进件出库率（出库/进件），每个月的采购价purchase_amount汇总，以及签约价new_actual_money汇总吧
# 未完成订单的签约价-已收new_actual_money-rent_received
# 剔除商家拒量的订单
df = df_zhys.copy()
df["下单日期"] = df["订单创建时间"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month
df["下单月份"]=df["订单创建时间"].dt.strftime('%Y-%m')
df['年份'] = df["下单日期"].dt.year
# df['hour'] = df['订单创建时间'].dt.hour
df.下单月份.unique()

array(['2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
       '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06',
       '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12',
       '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
       '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12',
       '2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06',
       '2025-07'], dtype=object)

In [ ]:
df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["是否进件"] = np.where(df["进件"] == '进件', 1, 0)
df["是否出库"] = np.where(df["status"].isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")
df.loc[:,"归属渠道"]=df.apply(lambda x: clean.qudao_type(x["来源渠道"],x["activity_name"],x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

df['归属渠道'] = np.where(df.归属渠道 == '顶部搜索框的搜索结果页', '搜索渠道', df.归属渠道)
# conditions_1 = ['租机宝','八派信息','CPS直播','派金花','富士康租机','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','兔丫租赁','分期乐1','分期乐2','98租超','邦客惠乘车码','推一推','硬派抖音']
conditions_1 = ['八派信息','CPS直播','派金花','九州信息','勉丫租','哈银','租瓜直播','美仑美奂','分期乐1','分期乐2','98租超','推一推','硬派抖音']
# 租机宝 、 富士康租机 、 兔丫租赁 、 邦客惠乘车码  -- 不算S量
df['归属渠道'] = np.where(df.归属渠道.isin(conditions_1), 'S量', df.归属渠道)
df['是否免人审'] = np.where((df.is_vip=='1')&(df.status_result=='0'), 1 , 0)
df['是否拒量'] = np.where(df.tips.str.contains(r'策略2412|命中自有模型回捞策略|回捞策略250330命中')==True, 1, 0)
df['是否号卡'] = np.where(df.type == 4, 1, 0)

# df = df[~df.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机","喜卓灵新租机"])]
df = df[df.是否出库==1]

# 订单去重
dict_status_code = {
            "订单取消": 1,
            "待支付": 2,
            "已退款": 3,
            "待审核": 4,
            "待发货": 5,
            "待收货": 6,
            "租赁中": 7,
            "已完成": 8
        }
df["状态编码"] = df["status2"].map(dict_status_code)
df.sort_values(by=["下单日期", "状态编码"], inplace=True)
# 删除重复单号
df.drop_duplicates(subset=["order_id"], inplace=True)
# 删除身份证空值行
df.dropna(subset=["id_card_num"], axis=0, inplace=True)
# 去刷单订单
# df.drop(df[df['total_describes'].str.contains(pat='panli', regex=False) == True].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单秘密计划"].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单秘密计划-无优惠"].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单曙光计划"].index, inplace=True)
df.drop(df[df['activity_name'] == "线下门店3个月试行"].index, inplace=True)
df.drop_duplicates(subset=["order_id"], inplace=True)
df.drop_duplicates(subset=["true_name", "mobile", "id_card_num", "下单日期"], keep="last", inplace=True)
df.drop(df[df['true_name'].isin(
        ["刘鹏", "谢仕程", "潘立", "洪柳", "陈锦奇", "周杰", "卢腾标", "孔靖", "黄娟", "钟福荣", "邱锐杰", "唐林华"
        , "邓媛斤", "黄子南", "刘莎莎", "赖瑞彤", "孙子文", '淦文豪', '杨明豪', '闫宇龙'])].index, inplace=True)


# 依据订单编号查重，如有重复输出重复的订单记录
duplicate=df[df.duplicated(subset=['order_number'], keep=False)]


duplicate


,id,order_id,order_number,order_time,search_time,end_date,merchant_id,merchant_name,channel_id,channel_name,model_number,specification,purchase_amount,platform_package,all_deposit,all_rental,liquidated_damages,buyout_balance_payment,advance_periods,advance_sum,order_create_time,rembursement_status,paid_periods,begin_overdue_time,overdue_periods,max_overdue_days,now_overdue_days,total_receivable,contract_price,rent_received,unearned_rent_revenue,paid_buyout_balance_payment,total_received,total_unreceived,unrecovered_cost,overdue_type,gross_profit_receivable,overdue_loss,actual_gross_profit,overdue_total_unreceived,relet_periods,relet_days,relet_rent_paid,activity_id,activity_source,order_label,create_time,update_time,order_finish_date,commodity_premium,overdue_fine,pause_reimbursement,create_by_id,update_by_id,del_flag,status,订单创建时间,status2,订单完成时间,order_method,true_name,id_card_num,mobile,product_name,new_actual_money,channel_name_cc,channel_type_id,activity_name,type,order_type,tips,is_vip,status_result,下单日期,月份,下单月份,年份,进件,是否进件,是否出库,来源渠道,归属渠道,状态编码


In [ ]:


df_copy = df[['all_rental', 'purchase_amount' ,'new_actual_money', 'rent_received','status']]
df_copy.loc[:,'剩余应收租金' ] = np.where(df_copy['status'].isin([2, 4, 5]), df_copy['all_rental']-df_copy['rent_received'], 0)
df_copy.loc[:,'剩余总应收' ] = np.where(df_copy['status'].isin([2, 4, 5]), df_copy['new_actual_money']-df_copy['rent_received'], 0)
# 将每一列所有金额汇总
df_copy.rename(columns={'all_rental':'总租金','purchase_amount': '采购金额','new_actual_money': '签约价', 'rent_received': '实收租金'}, inplace=True)

In [ ]:

data_res = df_copy.sum()
# 将数值单位由个转万元
data_res['应收总租金（万元）'] = (data_res['总租金']/10000).round(2)
data_res['采购金额（万元）'] = (data_res['采购金额']/10000).round(2)
data_res['签约价（万元）'] = (data_res['签约价']/10000).round(2)
data_res['实收租金（万元）'] = (data_res['实收租金']/10000).round(2)
data_res['剩余总应收'] = (data_res['剩余总应收']/10000).round(2)
data_res['剩余应收租金（万元）'] = (data_res['剩余应收租金']/10000).round(2)
# 写入excel
data_res[['应收总租金（万元）', '采购金额（万元）', '签约价（万元）', '实收租金（万元）', '剩余总应收','剩余应收租金（万元）']].T#.to_excel('F:/需求/倩姐需求/租金数据.xlsx', sheet_name='租金数据')

应收总租金（万元）     36165.59
采购金额（万元）      43895.88
签约价（万元）       67384.04
实收租金（万元）      22593.35
剩余总应收         40291.13
剩余应收租金（万元）    13852.36
dtype: float64

In [ ]:
df_copy.下单月份.unique()

array(['2022-07', '2022-08', '2022-09', '2022-10', '2022-11', '2022-12',
       '2023-01', '2023-02', '2023-03', '2023-04', '2023-05', '2023-06',
       '2023-07', '2023-08', '2023-09', '2023-10', '2023-11', '2023-12',
       '2024-01', '2024-02', '2024-03', '2024-04', '2024-05', '2024-06',
       '2024-07', '2024-08', '2024-09', '2024-10', '2024-11', '2024-12',
       '2025-01', '2025-02', '2025-03', '2025-04', '2025-05', '2025-06',
       '2025-07', '2025-08'], dtype=object)

In [ ]:
# 应收未收金额：未完成订单（status in [4,5]）的签约价-已收(new_actual_money-rent_received)
df_copy.loc[:,'应收未收金额' ] = np.where(df_copy['status'].isin([2, 4, 5]), df_copy['new_actual_money']-df_copy['rent_received'], 0)
# 按下单月份分组选取出库、进件、采购价、签约价、应收未收金额的汇总
df_res = df_copy[df_copy['purchase_amount']>0].groupby('下单月份').agg({'new_actual_money': 'sum','rent_received': 'sum', '应收未收金额': 'sum'}).rename(columns={'new_actual_money': '应收金额','rent_received': '已收金额'})
# # 计算进件出库率（出库/进件）
# df_res['进件出库率'] = df_res['出库']/df_res['进件']
# # 进件出库率百分比
# df_res['进件出库率'] = df_res['进件出库率'].apply(lambda x: '{:.2%}'.format(x))
# 写入excel
#with pd.ExcelWriter(f'F:/需求/倩姐需求/应收未收金额数据.xlsx', engine='openpyxl') as writer:
    #df_res.to_excel(writer, sheet_name='应收未收金额数据')
# 求和应收未收列
# df_res['应收未收金额'].sum()
# df_res

KeyError: 'new_actual_money'

In [ ]:
df_res['应收未收金额'].sum()

np.float64(410561778.66999996)

## 业务量数据

In [147]:
# 核心逻辑
# # 我们业务开始每个月的出库数据进件数据，进件出库率（出库/进件），每个月的采购价purchase_amount汇总，以及签约价new_actual_money汇总吧
# # 未完成订单的签约价-已收new_actual_money-rent_received
# # 剔除商家拒量的订单
# df = df[~df.merchant_name.isin(["小蚂蚁租机","兴鑫兴通讯","人人享租","崇胜数码","喜卓灵租机","喜卓灵新租机"])]
# df_copy = df[['下单月份', 'status', '是否进件' ,'是否出库' ,'purchase_amount' ,'new_actual_money', '已收租金']]
# df_copy
# # 应收未收金额：未完成订单（status in [4,5]）的签约价-已收(new_actual_money-rent_received)
# df_copy.loc[:,'应收未收金额' ] = np.where(df_copy['status'].isin([4,5]), df_copy['new_actual_money']-df_copy['已收租金'], 0)
# # 按下单月份分组选取出库、进件、采购价、签约价、应收未收金额的汇总
# df_res = df_copy.groupby('下单月份').agg({'是否进件': 'sum', '是否出库': 'sum', 'purchase_amount': 'sum', 'new_actual_money': 'sum', '应收未收金额': 'sum'}).rename(columns={'是否进件': '进件', '是否出库': '出库', 'purchase_amount': '采购价','new_actual_money': '签约价'})
# # 计算进件出库率（出库/进件）
# df_res['进件出库率'] = df_res['出库']/df_res['进件']
# # 进件出库率百分比
# df_res['进件出库率'] = df_res['进件出库率'].apply(lambda x: '{:.2%}'.format(x))
# # 写入excel
# # with pd.ExcelWriter(f'F:/需求/倩姐需求/业务量数据.xlsx', engine='openpyxl') as writer:
# #     df_res.to_excel(writer, sheet_name='业务量数据')

## 核对台账数据

### 核查总订单数，筛选出差异订单

In [66]:
# 读取共享文档台账数据
f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
df_ck = pd.read_excel(f_path_ck,sheet_name="2025") 
# 将序列号转换为日期
base_date = pd.to_datetime('1899-12-30')  # Excel的基准日期
df_ck['日期'] = base_date + pd.to_timedelta(df_ck['日期'], unit='D')
# 筛选日期大于等于'2025-09-11',小于='2025-10-28'的数据
dfck = df_ck[(df_ck['日期'] >= '2025-09-11') & (df_ck['日期'] <= '2025-10-28')]
dfck_order_num_list = dfck['订单号'].tolist()
dfck#.shape

,日期,订单号,类型,型号,内存,颜色,平台套餐,租金,已付租金,期数,订单归属,出租方,备注,快递单号,供应商,成本,碎屏险,设备串码
34247,2025-09-11,A20250909164843968,手机,无锁iPhone16ProMax,256GB,黑色钛金属,租完归还,7331.88,668.68,2,邹巧巧,北京孚林坊商贸,含碎屏险,SF3286119141546,杭州盛亚科技有限公司,8730,NaN,'359959359850884
34248,2025-09-11,A2025091109250871,手机,无锁iPhone16ProMax,256GB,白色钛金属,租完归还,5978.05,544.55,2,周汉鸿,海南鸿丰商业保理有限公司,含碎屏险,SF3232571183552,杭州盛亚科技有限公司,9050,NaN,'354708768855132
34249,2025-09-11,A20250911101158142,手机,无锁iPhone16ProMax,256GB,白色钛金属,租完归还,7993.66,1455.7,3,卢林焕,瑞和嘉信科技五店,京东,SF3232571186389,杭州盛亚科技有限公司,9114,NaN,'357525806611948
34250,2025-09-11,A20250911095634316,手机,无锁iPhone16ProMax,256GB,白色钛金属,租完归还,7331.88,668.68,2,周莹,海南鸿丰商业保理有限公司,含碎屏险,SF3286119141776,杭州盛亚科技有限公司,9050,NaN,'359959358447997
34251,2025-09-11,A20250911102206258,手机,无锁iPhone16ProMax,256GB,白色钛金属,租完归还,7331.88,668.68,2,周莹,海南鸿丰商业保理有限公司,含碎屏险,SF3232571186952,杭州盛亚科技有限公司,9050,NaN,'357573128831000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40683,2025-10-28,A20251028192253187,手机,无锁iPhone16ProMax,256GB,黑色钛金属,租完归还,6997.29,638.19,2,冯二洋,湖北团鑫贸易,含碎屏险,NaN,NaN,NaN,NaN,NaN
40684,2025-10-28,A20251028175826861,手机,无锁iPhone16ProMax,256GB,黑色钛金属,租完归还,5622.40,938,2,冯二洋,驻马店远程铺,含碎屏险,NaN,NaN,NaN,NaN,NaN
40685,2025-10-28,A2025102820063954,手机,无锁iPhone16ProMax,256GB,黑色钛金属,租完归还,7993.66,1455.7,3,廖丽敏,绿能智联科技,含碎屏险，京东,NaN,NaN,NaN,NaN,NaN
40686,2025-10-28,A2025102820055345,手机,无锁iPhone16ProMax,256GB,原色钛金属,租完归还,6997.29,638.19,2,冯二洋,小瓜智链科技,含碎屏险,NaN,NaN,NaN,NaN,NaN


In [67]:
# 读取数据库台账表的数据
sql_tz = '''
select * from db_digua_business.t_statistics_ledger 
WHERE del_flag = 0
and audit_time<='2025-10-28'
'''
df_tz = clean.query(sql_tz)
df_tz_list = df_tz['order_number'].tolist()
df_tz.shape

(6121, 26)

In [69]:
# 筛选出9月11日至10月28日数据库中存在，但共享文档中不存在的订单号
# A20250925213015156， A20251015093336179（台账中该订单号登记多了个换行符，实际订单号为A20251015093336179\n）
df_tz_not_in_ck = df_tz[~df_tz['order_number'].isin(dfck_order_num_list)]
df_tz_not_in_ck.shape
# 筛选出共享文档中存在，但数据库中不存在的订单号
dfck_not_in_tz = dfck[~dfck['订单号'].isin(df_tz_list)]
dfck_not_in_tz.shape

(2, 26)

(322, 18)

In [48]:
# dfck_not_in_tz[['日期', '订单号']].to_excel("F:/需求/倩姐需求/数据库缺失订单.xlsx")

In [49]:
# dfck[dfck.订单号.str.contains('A20251015093336179')]
# 读取数据库台账表中失效的订单数据
sql_tzsx = '''
select * from db_digua_business.t_statistics_ledger 
WHERE del_flag = 1
and audit_time<='2025-10-28'
'''
df_tzsx = clean.query(sql_tzsx)
df_tzsx_list = df_tzsx['order_number'].tolist()
df_tzsx.shape

(308, 26)

In [106]:
# 匹配数据库中失效订单号在共享文档中的数据
df_temp = dfck_not_in_tz.merge(df_tzsx[['order_number', 'del_flag']], left_on='订单号',right_on='order_number', how='left')
# 对订单号进行去重
df_temp = df_temp.drop_duplicates(subset=['订单号'])
df_tt = df_temp[(df_temp['日期']>'2025-09-11')&(df_temp.del_flag==1)][['日期', '订单号', 'del_flag','快递单号']]#.to_excel("F:/需求/倩姐需求/数据库缺失或失效订单.xlsx")
df_tt

,日期,订单号,del_flag,快递单号
71,2025-09-12,A20250912131523108,1.0,已支付100元违约金
73,2025-09-12,A20250912145255680,1.0,已支付100元违约金SF3286557717510
74,2025-09-12,A20250912164553637,1.0,已支付100元违约金SF3286557783360
75,2025-09-13,A20250912171703270,1.0,已支付100元违约金SF3232014229820
79,2025-09-13,A2025091314075083,1.0,取消订单
...,...,...,...,...
408,2025-10-27,A20251027104100518,1.0,已支付100元违约金SF3296238327263
409,2025-10-27,A20251027152238445,1.0,已支付100元违约金
410,2025-10-28,A2025102801034219,1.0,已支付100元违约金
411,2025-10-28,A20251028145111910,1.0,已支付100元违约金


### 核查订单号一致的总租金、已付租金、成本

In [72]:
# 筛选出9月11日至10月28日数据库中存在，共享文档中也存在的订单号
df_tz_in_ck = df_tz[df_tz['order_number'].isin(dfck_order_num_list)]
df_tz_in_ck.shape
df_tz_in_ck.loc['汇总',:] = df_tz_in_ck.select_dtypes(include=['number']).sum()


# 筛选出共享文档中存在，数据库中也存在的订单号
dfck_in_tz = dfck[dfck['订单号'].isin(df_tz_list)]
dfck_in_tz.shape
# 将租金、已付租金、成本列转换为数值类型
dfck_in_tz['租金'] = pd.to_numeric(dfck_in_tz['租金'].astype(str).str.replace(',', ''), errors='coerce')
dfck_in_tz['已付租金'] = pd.to_numeric(dfck_in_tz['已付租金'].astype(str).str.replace(',', ''), errors='coerce')
dfck_in_tz['成本'] = pd.to_numeric(dfck_in_tz['成本'].astype(str).str.replace(',', ''), errors='coerce')
dfck_in_tz.loc['汇总',:] = dfck_in_tz.select_dtypes(include=['number']).sum()



(6119, 26)

(6119, 18)

In [81]:
# df_tz_in_ck[-1:]
# 设置月份列的格式为YYYY-MM，先将audit_time转为日期类型在获取月份
df_tz_in_ck['月份'] = pd.to_datetime(df_tz_in_ck['audit_time']).dt.strftime('%Y-%m')
df_tz_9 = df_tz_in_ck[df_tz_in_ck['月份']=='2025-09']
df_tz_10 = df_tz_in_ck[df_tz_in_ck['月份']=='2025-10']
# 分别计算9月份、10月份所有订单all_money、advance_rent_money、purchase_money的总和
first_month_all_money = df_tz_9['all_money'].sum()
first_month_advance_rent_money = df_tz_9['advance_rent_money'].sum()
first_month_purchase_money = df_tz_9['purchase_money'].sum()
second_month_all_money = df_tz_10['all_money'].sum()
second_month_advance_rent_money = df_tz_10['advance_rent_money'].sum()
second_month_purchase_money = df_tz_10['purchase_money'].sum()
df_tz_9.shape
df_tz_10.shape


(2353, 27)

(3766, 27)

In [82]:
dfck_in_tz[-1:]
# 设置月份列的格式为YYYY-MM，先将日期列转为日期类型在获取月份
dfck_in_tz['月份'] = pd.to_datetime(dfck_in_tz['日期']).dt.strftime('%Y-%m')
dfck_9 = dfck_in_tz[dfck_in_tz['月份']=='2025-09']
dfck_9.shape
dfck_10 = dfck_in_tz[dfck_in_tz['月份']=='2025-10']
# 分别计算9月份、10月份所有订单租金、已付租金、成本的总和
month_9_ck = dfck_9['租金'].sum()
month_9_ck_advance = dfck_9['已付租金'].sum()
month_9_ck_cost = dfck_9['成本'].sum()
month_10_ck = dfck_10['租金'].sum()
month_10_ck_advance = dfck_10['已付租金'].sum()
month_10_ck_cost = dfck_10['成本'].sum()
# 计算差异并输出
print("9月份租金差异：", first_month_all_money - month_9_ck)
print("9月份已付租金差异：", first_month_advance_rent_money - month_9_ck_advance)
print("9月份成本差异：", first_month_purchase_money - month_9_ck_cost)
print("10月份租金差异：", second_month_all_money - month_10_ck)
print("10月份已付租金差异：", second_month_advance_rent_money - month_10_ck_advance)
print("10月份成本差异：", second_month_purchase_money - month_10_ck_cost)
dfck_10.shape


,日期,订单号,类型,型号,内存,颜色,平台套餐,租金,已付租金,期数,订单归属,出租方,备注,快递单号,供应商,成本,碎屏险,设备串码,月份
汇总,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4.343379e+07,4869326.4,11955.0,NaN,NaN,NaN,NaN,NaN,20968829.49,NaN,NaN,NaN


(2353, 19)

9月份租金差异： 1667.0580000001937
9月份已付租金差异： -44137.080000000075
9月份成本差异： 44080.0
10月份租金差异： 3384.960000000894
10月份已付租金差异： -76818.04999999981
10月份成本差异： 33551726.53


(3766, 19)

In [ ]:
# df_tz_9、df_tz_10分别拼接dfck_9、dfck_10的租金、已付租金、成本列
df_merge_9 = df_tz_9.merge(dfck_9[['订单号', '租金', '已付租金', '成本']], left_on='order_number',right_on='订单号', how='left')
df_merge_10 = df_tz_10.merge(dfck_10[['订单号','租金', '已付租金', '成本']], left_on='order_number',right_on='订单号', how='left')
# 输出df_merge_9的all_money-租金不等于0的订单记录
df_merge_10[df_merge_10['purchase_money'] - df_merge_10['成本'] != 0]#.audit_time.value_counts()

筛选范围：2025-09-11至2025-10-28

订单差异：
数据库中存在，但共享文档中不存在的订单号A20250925213015156；
筛选出共享文档中存在，但数据库中不存在的订单有322单，其中209单已失效A20250913123947381, A20250913133506223, A20250919182355264

租金差异：皆为台账人工填写失误，已沟通解决

已收租金差异：数据库预收金额未计入优惠金额但计入了押金，台账已收租金未计入首租一元的租金，故两者存在差异，差异订单均匀分布。

成本差异：台账成本字段存在大量空值或0值

# 临时需求

## 去除商家后自营的租送、租归出库数

In [192]:
# 使用租后监控表做主表（订单号唯一），关联订单详情表、订单表、活动表、用户表
sql ="""
select 
tprm.order_number, tprm.order_time, tprm.merchant_name, tod.sku_attributes, om.status, om.user_mobile, tprm.rembursement_status,om.has_actual,tprm.relet_days,
tprm.order_create_time,pa.name as activity_name ,tmu.id_card_num, tprm.order_id
-- ,tol.delivery_province_name, tol.delivery_city_name, tol.status 物流状态
from db_digua_business.t_postlease_receivables_monitoring tprm
left join db_digua_business.t_order_details tod on tprm.order_id = tod.order_id
left join db_digua_business.t_order om on tprm.order_id = om.id
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- left join db_digua_business.t_order_logistics tol on tprm.order_id = tol.order_id
where DATE_FORMAT(tprm.order_create_time, '%Y-%m-%d')<='2025-09-09'
"""

df_order = clean.query(sql)
df_order.shape

(67668, 13)

In [144]:
df_order.rembursement_status.value_counts()

rembursement_status
还款中        30861
已逾期        17089
已买断         9314
未到首期还款日     4788
续租中         4074
已完成         3509
检测中          114
Name: count, dtype: int64

In [193]:
df = df_order.copy()
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df["是否出库"] = np.where(df["status"].isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)
df["下单日期"]=df["order_create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
# df.loc[:, 'delivery_address'] = df['delivery_province_name'] + df['delivery_city_name']
# 按下单日期与id_card_num去重
df.drop_duplicates(subset=["id_card_num", "下单日期"], keep="last", inplace=True)
df.drop_duplicates(subset=["order_number"], inplace=True)
df.drop_duplicates(subset=["order_id"], inplace=True)
# 删除身份证空值行
df.dropna(subset=["id_card_num"], axis=0, inplace=True)
# 删除订单状态空值行
df.dropna(subset=["status"], axis=0, inplace=True)
df.drop(df[df['activity_name'] == "1000单秘密计划"].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单秘密计划-无优惠"].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单曙光计划"].index, inplace=True)
df.drop(df[df['activity_name'] == "线下门店3个月试行"].index, inplace=True)
# 剔除商家
df = clean.drop_merchant(df)
# 剔除拒量
reject_merchants = ["小蚂蚁租机", "兴鑫兴通讯", "人人享租", "崇胜数码", "喜卓灵租机", "喜卓灵新租机"]
df.drop(df[df['merchant_name'].isin(reject_merchants)].index, inplace=True)
# df.rename(columns={"order_time": "下单月份"}, inplace=True)
# 按delivery_address分组
# df_city_g = df.groupby('delivery_city_name').agg({'是否出库': 'sum'}).rename(columns={'是否出库': '出库数量'})
# df_city_g = df_city_g.sort_values(by='出库数量', ascending=False)
# df_city_g[:50].to_excel('F:/需求/倩姐需求/前50城市出库数.xlsx')

In [ ]:
df.loc[:, '已归还'] = np.where((df.status.isin([8, 15, 16]))&(df.has_actual==0)&(df.relet_days==0), 1, 0)

In [204]:
df_zs = df[(df['status'].isin([8,15,16]))&(df.has_actual==0)&(df.relet_days==0)][['order_number','租赁方案','status','has_actual', 'relet_days']]
df_zs

,order_number,租赁方案,status,has_actual,relet_days
1,A202207131230485,租完归还,8,0,0
8,A202207190744402,租完即送,8,0,0
36,A2022080921122534,租完归还,8,0,0
38,A202208100300451,租完归还,8,0,0
41,A202208110022192,租完即送,8,0,0
...,...,...,...,...,...
67551,A20250909121111181,租完归还,8,0,0
67588,A2025090915014828,租完归还,8,0,0
67611,A20250909162230416,租完归还,8,0,0
67620,A202509091653541091,租完归还,8,0,0


In [206]:
df_db = pd.read_csv('F:\需求\总需求/111.txt', header=None)
df_db_list = df_db[0].tolist()#1292+1151-4=
df_tmp = df_zs[~df_zs.order_number.isin(df_db_list)]
df_tmp['order_number'].to_excel('F:\需求\总需求/112.xlsx')


In [189]:
# df_zs.has_actual.value_counts()
# df_zs.relet_days.value_counts()
# df_zs.status.value_counts()
df.merchant_name.unique()

array(['迪瓜优选', '大龙发十六号店', '大龙发六号店', '大龙发二号店', '大龙发十八号店', '大龙发十号店',
       '大龙发十一号店', '大龙发十九号店', '数字湖科技', '大龙发十三号店', '大龙发五号店', '大龙发三号店',
       '大龙发八号店', '广东易代租机', '大龙发一号店', '宁凡手机店', '智特兴科技', '大龙发十五号店', '义源电子店',
       '大龙发九号店', '大龙发二十号店', '大龙发十七号店', '影贝网络科技', '乾宝通1号店', '大龙发二十二号店',
       '大龙发二十三号店', '华南通信科技', '大龙发二十五号店', '大龙发二十六号店', '成都越蓉金科科技有限公司',
       '永冠兴六号店', '永冠兴三号店', '聚和实业店', '真旺电子铺', '永冠兴九号店', '乾宝通二号店', '乾宝通三号店',
       '永冠兴八号店', '乾宝通四号店', '永冠兴十号店', '永冠兴五号店', '南宁微美汇', '广州捷瑞环保科技',
       '瑞和嘉信科技', '永冠兴二号店', '永冠兴七号店', '永冠兴十一号店', '山东易路人店', '华南通信二号店',
       '域见未来店', '广州康基店', '绿能智联科技', '永冠兴十二号店', 'LK雷铠科技', '桂林野山来店',
       '厦门华宇盛世科技', '萃益仁生物科技', '天翊科技店', '广州蕴发科技', '广西鑫美达家居', '桂林名清店',
       '贵州恒丰泰商务贸易', '贵州益多租店', '贵州中嘉甄选', '北京华瑞荣', '永冠兴十三号店', '海南聚辰科技',
       '杭州金锤网络信息服务有限公司', '上海惠天下', '腾冲腾超店', '林枫科技店', '光宁新能源科技', '海口良泰鑫阳',
       '鑫时通科技', '永冠兴十五号店', '瑞和嘉信科技二店', '中迪晟科技', '广州惠众天下', '瑞和嘉信科技三店',
       '百力一号店', '重庆腾鼎建材', '百力科技二号店', '和鸿润科技', '北京麒麟阁', '泽越科技店',
       '瑞和嘉信科技六店', '科盟信息店

In [119]:
# df_tc = pd.read_excel('F:\需求\倩姐需求\goal_order_number_20250908.xlsx')# 3868
# df_list_tc = df_tc['order_number'].tolist()
# df = df[~df.order_number.isin(df_list_tc)]
# df.shape#62947

(61338, 13)

In [ ]:
df_ck = df[df['是否出库'] == 1]
df_group = pd.crosstab(df_ck["下单月份"], df_ck["租赁方案"], margins=True)
df_group.loc[:, '租完即送占比'] = (df_group.loc[:, '租完即送'] / df_group.loc[:, 'All']).map('{:.2%}'.format)
df_group.loc[:, '租完归还占比'] = (df_group.loc[:, '租完归还'] / df_group.loc[:, 'All']).map('{:.2%}'.format)
df_group#.to_excel('F:/需求/倩姐需求/租赁方案出库数及占比.xlsx')

In [124]:
# df_ck = df[(df['下单月份']=='2025年02月') & (df['是否出库']==1)&(df['租赁方案']=='租完即送')]
# # df_8 = pd.read_excel('F:/需求/倩姐需求/tmp(5).xlsx')
# # df_8_list = df_8['order_number'].tolist()
# # df_ck_r = df_ck[~df_ck.order_number.isin(df_8_list)]
# df_ck['order_number'].to_excel('F:/需求/倩姐需求/2月租完即送出库订单.xlsx', index=False)

## 联合运营

In [107]:
def select_data(hour):
    sql1 = f''' -- 订单&风控信息  近10日数据   
    SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
    ,om.status, om.user_id
    ,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
    when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
    when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
    ,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
    ,tod.sku_attributes,tod.product_name,tod.new_actual_money
    ,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
    ,om.user_mobile,tmu.true_name,tmu.id_card_num
    ,top.total_describes,tor.decision_result,om.cancel_reason
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips 
    ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
    ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result 
    ,cc.name as channel_name         -- 来源渠道
    ,pa.name as activity_name        -- 活动名称
    ,om.merchant_id,om.merchant_name
    ,topay.total_freeze_fund_amount , tod.dy_order_item_json, pa.type
    ,om.buy_service_product,tso.status as service_status 
    ,om.order_method, om.activity_id, om.appid, tprm.max_overdue_days
    , tor.update_time, tomt.reason, cc.channel_type_id, om.order_type, om.union_rent_tag, tojo.app_type
    from  db_digua_business.t_order  om
    left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
    left join db_digua_business.t_order_risk tor on om.id = tor.order_id
    -- 备注信息合并 
    left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
    on om.id = top.order_id 
    -- 服务信息
    left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
    -- 渠道名称
    left join db_digua_business.t_channel cc on om.channel = cc.scene 
    -- 活动名称
    left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
    -- 用户信息 
    left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
    -- 商品信息
    left join db_digua_business.t_order_details tod on om.id = tod.order_id
    -- 免押信息  
    left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
    from db_digua_business.t_order_pay t 

    where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
    on topay.order_id=om.id   and  topay.rn = 1 
    -- 商家订单转移表
    left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
    -- 京东外部订单关联表
    left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=om.id
    where om.user_mobile is not null 
    and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
    ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
    -- and pa.type!=4
    and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day )
    and hour(om.create_time)<'{hour}'
    ;
    '''
    sql3 = ''' -- 拒量拒绝原因
    SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status  
    FROM `db_credit`.risk_examine
    '''

    sql_risk = ''' -- risk等级
    select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 

    -- , replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_tag') end,'"','') as union_rent_tag
    -- , replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.union_rent_rejected') end,'"','') as union_rent_rejected 
    from db_credit.risk
    '''
    df_risk = clean.query(sql_risk)

    sql_ra = ''' -- 996强拒表
    select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  
    from db_credit.risk_alipay_interactive_prod_result
    '''
    df_ra = clean.query(sql_ra)
    df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date, errors="coerce")
    df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)
    # sql_ck = ''' -- 台账出库数据
    # select date, order_number, category, remark from db_digua_business.t_ledger
    # '''
    f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2025年台账.xlsx"
    df_ck = pd.read_excel(f_path_ck, sheet_name="2025")
    df_order = clean.query(sql1)
    df_order = df_order[df_order.type != 4]
    df_risk_examine = clean.query(sql3)
    df_re = df_risk_examine.copy()
    df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date, errors="coerce")
    df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)
    return df_order, df_risk_examine, df_ck, df_risk, df_re, df_ra

# 数据处理
def clean_data(df, df_ck, df_risk, df_re, df_ra):
    # 处理日期
    df["下单日期"] = df["create_time"].dt.date
    df["下单日期"] = pd.to_datetime(df["下单日期"], errors="coerce")
    df["月份"] = df["下单日期"].dt.month
    df["下单月份"] = df["create_time"].dt.strftime('%Y-%m')
    df['hour'] = df['create_time'].dt.hour
    df['minute'] = df['create_time'].dt.minute
    df['拒绝理由'] = df["rejected"].str.replace("[", "").str.replace("]", "").str.replace('"', '')
    # 处理备注信息
    df = df[df['sku_attributes'].notnull()]
    df["取消原因"] = df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
    df["电审拒绝原因"] = df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
    df["无法联系原因"] = df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[
        0].str.strip()
    df.loc[:, "商品类型"] = np.where(
        df["product_name"].str.contains('99新') | df["product_name"].str.contains('95新') | df[
            "product_name"].str.contains('准新'), "二手", "全新")  ##  准新
    df.loc[:, "租赁方案"] = np.where(df["sku_attributes"].str.contains('租完即送'), "租完即送", "租完归还")
    df.loc[:, "押金类型"] = np.where(df["total_freeze_fund_amount"] > 0, "部分免押", "全免押")
    df.loc[:, "优惠券使用否"] = np.where((df["new_actual_money"] - df["all_money"] > 0) & (df["租赁方案"] == '租完即送'), "已使用", "未使用")
    df.loc[:, "是否进行预授权"] = np.where(df["total_freeze_fund_amount"].isnull(), "未预授权", "已预授权")

    df.loc[:, "颜色"] = df.apply(lambda x: clean.getcolor(x["sku_attributes"]), axis=1)
    df.loc[:, "内存"] = df.apply(lambda x: clean.getneicun(x["sku_attributes"]), axis=1)
    df.loc[:, "进件"] = np.where((df["status2"] == "待支付") | (df["status2"] == "订单取消"), "未进件", "进件")
    df.loc[:, "来源渠道"] = df["channel_name"].fillna("未知渠道")
    # 渠道归属
    df.loc[:, "归属渠道"] = df.apply(
        lambda x: clean.qudao_type(x["来源渠道"], x["activity_name"], x["order_method"], x['channel_type_id'], x['order_type']), axis=1)
    # 保留去重前的数据
    df_no_drop = df.copy()
    # 订单去重
    df = clean.order_drop_duplicates(df)
    # 删除重复订单
    df.drop_duplicates(subset=["order_id"], inplace=True)


    # 定义状态 , 'union_rent_tag', 'union_rent_rejected'
    df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(
        df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(
        df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'],
        right_on=['order_id', 'time_ra'], how='left')

    # 判断 前置拦截   机审强拒   出库前风控强拒
    df['是否前置拦截'] = np.where(
        (df.result.str.contains('id_card不得为空')) | (df.result.str.contains('mobile校验不通过')) | (
            df.result.str.contains('name校验不通过'))
        | (df.result.str.contains('年龄超过49岁或低于18岁')) | (df.result == '风控未通过') | (
                df.result == '黑名单用户'), 1, 0)
    df['是否机审强拒'] = np.where(df.status_r == '1', 1, 0)
    df['是否出库前风控强拒'] = np.where((df.status_re == '1') | (df.status_ra == '1'), 1, 0)
    df.loc[:, "审核状态"] = df.apply(
        lambda x: clean.reject_type(x["拒绝理由"], x["进件"], x["电审拒绝原因"], x["取消原因"],
                                            x["status2"], x["无法联系原因"], x["total_describes"],
                                            x['是否前置拦截'], x['是否机审强拒'], x['是否出库前风控强拒']), axis=1)
    df['取消原因2'] = df['cancel_reason'].str.split('：')
    df['取消原因2'] = df['取消原因2'].apply(lambda x: x[-1] if x is not None else x)
    # 保留商家数据
    df_contain = df.copy()
    # 剔除商家数据
    df = clean.drop_merchant(df)
    # 进件数据
    df_j = df[df["进件"] == "进件"]
    # 各个节点状态
    df["待审核"] = np.where(df["审核状态"] == '待审核', 1, 0)
    df["前置拦截"] = np.where(df["审核状态"] == '前置拦截', 1, 0)
    df["人审拒绝"] = np.where(df["审核状态"] == '人审拒绝', 1, 0)
    df["客户取消"] = np.where(df["审核状态"] == '客户取消', 1, 0)
    df["无法联系"] = np.where(df["审核状态"] == '无法联系', 1, 0)
    df["是否进件"] = np.where(df["进件"] == '进件', 1, 0)
    df["是否出库"] = np.where(df["status"].isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)
    df["进件前取消"] = np.where(df["审核状态"] == '进件前取消', 1, 0)
    df['是否出库'] = np.where(
        (df['人审拒绝'] == 0) & (df['客户取消'] == 0) & (df['无法联系'] == 0) & (df['待审核'] == 0) & (
                df['是否出库'] == 1), 1, 0)
    df["出库前风控强拒"] = np.where((df["审核状态"] == '出库前风控强拒') & (df['是否出库'] == 0), 1, 0)
    df["机审强拒"] = np.where((df["审核状态"] == '机审强拒') & (df['是否出库'] == 0), 1, 0)
    df['机审通过件'] = np.where((df['是否进件'] == 1) & (df['机审强拒'] == 0), 1, 0)
    df['风控通过件'] = np.where((df['是否进件'] == 1) & (df['机审强拒'] == 0) & (df['人审拒绝'] == 0), 1, 0)
    df['已退款'] = np.where((df['风控通过件'] == 1) & (df['审核状态'] == '已退款'), 1, 0)
    df['是否二手'] = np.where(df['product_name'].str.contains(r'99新|95新|准新|90新'), 1, 0)
    # 取到没有去重的数据
    df_ll = df.copy()
    # 排序去重
    df.drop_duplicates(subset=["true_name", "user_mobile", "id_card_num", "下单日期"], keep="last", inplace=True)
    # 关联台账数据
    dfck = pd.merge(df_ck, df, left_on="订单号", right_on='order_number')
    dfck.drop_duplicates(subset=["order_number"], inplace=True)
    # 删除已退款订单
    dfck.drop(dfck[dfck["status2"] == "已退款"].index, inplace=True)
    # 删除 露营设备 出库
    try:
        dfck.drop(dfck[dfck["类目"] == "露营设备"].index, inplace=True)
    except:
        dfck.drop(dfck[dfck["类型"] == "露营设备"].index, inplace=True)

    df2 = df.copy()
    df2_ll = df_ll.copy()
    df2 = clean.drop_rejected_merchant(df2)
    df2_ll = clean.drop_rejected_merchant(df2_ll)
    return df_contain, df, df2, dfck, df_j, df_ll, df2_ll, df_no_drop


In [113]:
def get_lhyy( df, df2, df_risk_examine):
    today = datetime.today()
    t_date = today.date().strftime('%Y-%m-%d')
    # 获取联合运营订单的转化数据
    df_lhyy = df[df.union_rent_tag=='Y']
    df2_lhyy = df2[df2.union_rent_tag=='Y']

    df_group = all_models.data_group(df_lhyy, df2_lhyy, df_risk_examine, '下单日期')
    df_group = df_group[["去重订单数", "前置拦截", "拦截率", "进件数", "预授权通过率", "机审强拒", "强拒比例", "机审通过件",
                "人审拒绝", "风控通过件", "风控通过率", "客户取消", "无法联系",
                "出库前风控强拒", "待审核", '出库', '进件出库率', '取消率', '人审拒绝率', '出库前强拒比例',
                '无法联系占比', '订单出库率']]
    # 获取要发送消息的数据
    df_group_new = df_group.iloc[-1, :]
    jj = df_group_new.loc['进件数']
    qx = df_group_new.loc['机审强拒'] + df_group_new.loc['人审拒绝'] + df_group_new.loc['客户取消'] + df_group_new.loc['无法联系'] + df_group_new.loc['出库前风控强拒']
    sh = df_group_new.loc['待审核']
    tg = df_group_new.loc['出库']
    # 各位好，今天进件5单，客户取消0单， 待审核3单，通过2单
    if jj != 0:
        info = f'''各位好，今天进件{jj}单，客户取消{int(qx)}单，待审核{sh}单，通过{tg}单'''
    else:
        info = '各位好，今天没有进件'
    return df_group, info

def run( hour):
    print('正在查询数据...')
    df_order, df_risk_examine, df_ck, df_risk, df_re, df_ra = select_data(hour)
    print('数据查询完毕...\n正在清理数据...')
    df_contain, df, df2, dfck, df_j, df_ll, df2_ll, df_no_drop = clean_data(df_order,df_ck, df_risk, df_re, df_ra)
    print('数据清理完毕...\n正在获取数据...')
    # df_all2, df_ss_group2, df_zm_group2 = get_data_hour(df, df2, df_risk_examine)
    df_lhyy_group, info = get_lhyy(df_ll, df2_ll, df_risk_examine)
    # df_jd_group_new = jd(df_no_drop, df, df2, df_risk_examine)
    # df_jd_group_new2 = jd2(df_no_drop, df, df2, df_risk_examine)
    # df_jd_group_new3 = jd3(df_no_drop, df_risk, df_re, df_ra)
    print('数据获取完毕...')
    return df, df2,df_risk_examine, df_lhyy_group, info#, df_jd_group_new, df_jd_group_new2, df_jd_group_new3

def my_job(hour, minute, hour_date):
    Today = str(datetime.now().strftime('%Y%m%d%H'))
    print(f'执行定时任务：现在是{Today}的{hour}:{minute}')
    df_all2, df_ss_group2, df_zm_group2,df, df2,df_risk_examine, df_lhyy_group, info = run(hour_date)
    # with pd.ExcelWriter(path + f'渠道转化_{Today}.xlsx', engine='xlsxwriter') as writer:
    #     df_all2.to_excel(writer, sheet_name='总体')
    #     df_ss_group2.to_excel(writer, sheet_name='搜索渠道')
    #     df_zm_group2.to_excel(writer, sheet_name='芝麻租物')
    # with pd.ExcelWriter(path2 + f'联合运营_{Today}.xlsx', engine='xlsxwriter') as writer:
    #     df_lhyy_group.to_excel(writer, sheet_name='联合运营转化')
    # 通过钉钉机器人发送消息
    # 要发送的消息内容
    # send_dingtalk_message(webhook, secret, info)
    return df, df2,df_risk_examine, df_lhyy_group, info#, df_jd_group_new, df_jd_group_new2, df_jd_group_new3
    # gc.collect()
    print("回收内存执行完毕！\n")

In [116]:
df_order, df_risk_examine, df_ck, df_risk, df_re, df_ra = select_data(18)
df_contain, df, df2, dfck, df_j, df_ll, df2_ll, df_no_drop = clean_data(df_order,df_ck, df_risk, df_re, df_ra)


In [131]:
df_lhyy = df_ll[df_ll.union_rent_tag=='Y']
df2_lhyy = df2_ll[df2_ll.union_rent_tag=='Y']
df_group = all_models.data_group_contain_hl(df_lhyy, df2_lhyy, df_risk_examine, '下单日期')
df_lhyy_1029 = df_lhyy[df_lhyy['下单日期'] == '2025-10-29']
df2_lhyy_1029 = df2_lhyy[df2_lhyy['下单日期'] == '2025-10-29']

In [138]:
# df_lhyy_1029[df_lhyy_1029['拒绝理由'].str.strip() != '']#.拒绝理由.value_counts()
# df2_lhyy_1029.拒绝理由.value_counts()
df_group = df_group[["去重订单数", "前置拦截", "拦截率", "进件数", "预授权通过率", "机审强拒", "强拒比例", "机审通过件",
                "人审拒绝", "风控通过件", "风控通过率", "客户取消", "无法联系",
                "出库前风控强拒", "待审核", '出库', '进件出库率', '取消率', '人审拒绝率', '出库前强拒比例',
                '无法联系占比', '订单出库率']]
df_group[-1:]

,去重订单数,前置拦截,拦截率,进件数,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,无法联系,出库前风控强拒,待审核,出库,进件出库率,取消率,人审拒绝率,出库前强拒比例,无法联系占比,订单出库率
下单日期,,,,,,,,,,,,,,,,,,,,,,
2025-10-29,44,5,11.36%,33,75.00%,0.0,0.00%,33.0,12.0,21.0,63.64%,2,4,0.0,5,10,30.30%,6.06%,36.36%,0.00%,12.12%,22.73%


# 崔婷玉需求，京东商城、金融买断转化

In [2]:
sql1 = f''' -- 订单&风控信息  近10日数据   
        SELECT om.create_time,om.id as order_id ,om.order_number,om.status, date(om.create_time) as create_date
        ,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
        when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
        when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
        ,tod.sku_attributes,tod.product_name,tod.new_actual_money
        ,om.user_mobile,tmu.true_name,tmu.id_card_num
        ,top.total_describes
        ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
        ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
        ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
        ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
        ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
        ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
        ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
        ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
        ,cc.name as channel_name         -- 来源渠道
        ,cc.channel_type_id              -- 渠道id
        ,pa.name as activity_name        -- 活动名称
        ,om.merchant_id,om.merchant_name,pa.type
        ,om.order_method, om.activity_id
        ,om.order_type, tor.update_time, tomt.reason, tpmn.name 机型, cc.scene, tp.id 商品ID, tp.name phone_name, tojo.app_type
        -- ,tolog.status 物流状态
        from  db_digua_business.t_order  om
        left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
        left join db_digua_business.t_order_risk tor on om.id = tor.order_id
        -- 备注信息合并 
        left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
        on om.id = top.order_id 
        -- 渠道名称
        left join db_digua_business.t_channel cc on om.channel = cc.scene 
        -- 活动名称
        left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
        -- 用户信息 
        left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
        -- 商品信息
        left join db_digua_business.t_order_details tod on om.id = tod.order_id
        -- 商家订单转移表
        left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
        -- 商品表
        left join db_digua_business.t_product tp on tp.id = tod.product_id
        -- 商品型号
        left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id
        -- 物流表
        left join db_digua_business.t_order_logistics tolog on tolog.order_id=om.id
        -- 京东外部订单关联表
        left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=om.id
        where om.user_mobile is not null 
        and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
        ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
        
        and date_format(om.create_time, '%Y-%m-%d')>='2025-10-01'
        and date_format(om.create_time, '%Y-%m-%d')<='2025-10-09'
        ;
        '''
df_order = clean.query(sql1)
df_order.status2.value_counts()

status2
订单取消     111656
已退款       23536
租赁中        1809
待收货         281
已完成         218
待审核         111
待发货          47
待商家收货         5
待归还           2
Name: count, dtype: int64

## select

In [1]:
def select_data():
        sql1 = ''' -- 订单&风控信息  近10日数据   
                SELECT om.create_time,om.id as order_id ,om.order_number,om.status, date(om.create_time) as create_date
                ,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
                when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
                when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
                ,tod.sku_attributes,tod.product_name,tod.new_actual_money
                ,om.user_mobile,tmu.true_name,tmu.id_card_num
                ,top.total_describes
                ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
                ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
                ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
                ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
                ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.is_vip') end,'"','') as is_vip
                ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.status') end,'"','') as status_result
                ,replace(case when JSON_VALID(tor.risk) THEN JSON_EXTRACT(tor.risk, '$.queue_verify_type') end,'"','') as qvt_risk
                ,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.queue_verify_type') end,'"','') as qvt_result
                ,cc.name as channel_name         -- 来源渠道
                ,cc.channel_type_id              -- 渠道id
                ,pa.name as activity_name        -- 活动名称
                ,om.merchant_id,om.merchant_name,pa.type
                ,om.order_method, om.activity_id
                ,om.order_type, tor.update_time, tomt.reason, tpmn.name 机型, cc.scene, tp.id 商品ID, tp.name phone_name, tojo.app_type
                -- ,tolog.status 物流状态
                from  db_digua_business.t_order  om
                left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=om.id
                left join db_digua_business.t_order_risk tor on om.id = tor.order_id
                -- 备注信息合并 
                left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
                on om.id = top.order_id 
                -- 渠道名称
                left join db_digua_business.t_channel cc on om.channel = cc.scene 
                -- 活动名称
                left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
                -- 用户信息 
                left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
                -- 商品信息
                left join db_digua_business.t_order_details tod on om.id = tod.order_id
                -- 商家订单转移表
                left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
                -- 商品表
                left join db_digua_business.t_product tp on tp.id = tod.product_id
                -- 商品型号
                left join db_digua_business.t_product_model_number tpmn on tpmn.id=tp.model_number_id
                -- 物流表
                left join db_digua_business.t_order_logistics tolog on tolog.order_id=om.id
                -- 京东外部订单关联表
                left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=om.id
                where om.user_mobile is not null 
                -- and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
                -- ,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
                
                and date_format(om.create_time, '%Y-%m-%d')>='2025-09-01'
                and date_format(om.create_time, '%Y-%m-%d')<='2025-09-27'
                ;
                '''
        df_order = clean.query(sql1)
        df_order = df_order[df_order.type != 4]
        sql_risk = ''' -- risk等级
                        select trace_id, id_card as id_card_r, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status_r 
                        from db_credit.risk
                        '''
        df_risk = clean.query(sql_risk)
        sql3 = '''
                        SELECT risk_trace_id trace_id, id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
                        '''
        df_risk_examine = clean.query(sql3)
        df_re = df_risk_examine.copy()
        df_re.loc[:, 'time_re'] = pd.to_datetime(df_re.time.dt.date, errors="coerce")
        df_re.rename(columns={'id_card': 'id_card_re', 'status': 'status_re'}, inplace=True)

        sql_ra = ''' -- 996强拒表
                        select order_id, time, replace(case when JSON_VALID(result) THEN JSON_EXTRACT(result, '$.status') end,'"','') as status_ra  from db_credit.risk_alipay_interactive_prod_result
                        '''
        df_ra = clean.query(sql_ra)
        df_ra.loc[:, 'time_ra'] = pd.to_datetime(df_ra.time.dt.date, errors="coerce")
        df_ra = df_ra.sort_values(by='time', ascending=False).groupby('order_id').head(1)

        sql_upv = '''
        select
        tsc.scene, date(tsc.day) c_date, tsc.new_uv_count, tsc.uv_count, tsc.pv_count
        from db_digua_business.t_statistics_channel tsc
        '''
        df_upv = clean.query(sql_upv)
        df_order.loc[:, 'phone_name'] = df_order.phone_name.str.replace(' ', '').str.extract(r'(iPhone\d+(ProMax|Pro|Plus)?)')[0]
        df_order = df_order.merge(df_upv, left_on=['scene', 'create_date'], right_on=['scene', 'c_date'], how='left')
        df_order.loc[:,'商品ID'] = df_order.商品ID.astype(str)+'_'+df_order.phone_name


        return df_order, df_risk, df_risk_examine, df_re, df_ra



## clean

In [2]:
def clean_data(df, df_risk, df_re, df_ra, key=None):
    # 日期处理
    df["下单日期"] = df["create_time"].dt.date
    df["下单日期"] = pd.to_datetime(df["下单日期"], errors="coerce")
    df["月份"] = df["下单日期"].dt.month
    df["下单月份"] = df["create_time"].dt.strftime('%Y-%m')
    df['年份'] = df["下单日期"].dt.year
    df['hour'] = df['create_time'].dt.hour
    # 备注信息处理
    df['拒绝理由'] = df["rejected"].str.replace("[", "").str.replace("]", "").str.replace('"', '')
    df = df[df['sku_attributes'].notnull()]
    df["取消原因"] = df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
    df["电审拒绝原因"] = df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
    df["无法联系原因"] = df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()

    df.loc[:, "内存"] = df.apply(lambda x: clean.getneicun(x["sku_attributes"]), axis=1)
    df.loc[:, '机型内存'] = df.机型+'_'+df.内存
    df.loc[:, "进件"] = np.where((df["status2"] == "待支付") | (df["status2"] == "订单取消"), "未进件", "进件")
    df.loc[:, "来源渠道"] = df["channel_name"].fillna("未知渠道")
    # 渠道归属
    df.loc[:, "归属渠道"] = df.apply(
        lambda x: clean.qudao_type(x["来源渠道"], x["activity_name"], x["order_method"],
                                        x['channel_type_id'], x['order_type']), axis=1)
    # 京东不去重数据，不需要执行订单去重处理
    if key != '京东':
        # 订单去重处理
        df = clean.order_drop_duplicates(df)

    # 定义状态处理
    df = df.merge(df_risk[['trace_id', 'status_r']], on='trace_id', how='left').merge(
        df_re[['trace_id', 'status_re']], on='trace_id', how='left').merge(
        df_ra[['order_id', 'time_ra', 'status_ra']], left_on=['order_id', '下单日期'],
        right_on=['order_id', 'time_ra'], how='left')
    df['是否前置拦截'] = np.where(
        (df.result.str.contains('id_card不得为空')) | (df.result.str.contains('mobile校验不通过')) | (
            df.result.str.contains('name校验不通过'))
        | (df.result.str.contains('年龄超过49岁或低于18岁')) | (df.result == '风控未通过') | (
                df.result == '黑名单用户'), 1, 0)
    df['是否机审强拒'] = np.where(df.status_r == '1', 1, 0)
    df['是否出库前风控强拒'] = np.where((df.status_re == '1') | (df.status_ra == '1'), 1, 0)
    # df.loc[:, "审核状态"] = df.apply(
    #     lambda x: clean.reject_type(x["拒绝理由"], x["进件"], x["电审拒绝原因"], x["取消原因"], x["status2"],
    #                                     x["无法联系原因"], x["total_describes"], x['是否前置拦截'], x['是否机审强拒'],x['是否出库前风控强拒']), axis=1)
    
    df.loc[:, "审核状态"] = df.apply(
    lambda x: clean.reject_type(
        x["拒绝理由"], 
        x["进件"], 
        x["电审拒绝原因"], 
        x["取消原因"], 
        x["status2"] if x["status2"] is not None else "",
        x["无法联系原因"], 
        x["total_describes"], 
        x['是否前置拦截'], 
        x['是否机审强拒'],
        x['是否出库前风控强拒']
    ), 
    axis=1
)
    # 剔除商家数据
    # df = clean.drop_merchant(df)
    # 获取节点状态数据
    df = clean.status_node(df)

    # 剔除拒收数据
    # df = df[df['物流状态'] != 5]
    # 剔除据量数据
    df2 = df.copy()
    # df2 = clean.drop_rejected_merchant(df2)
    df2 = df2[df2.是否拒量 == 0]



    return df, df2

## run

In [3]:
def run():
    df, df_risk, df_risk_examine, df_re, df_ra = select_data()
    print(df.shape, df_risk.shape, df_risk_examine.shape, df_re.shape, df_ra.shape)
    return df, df_risk, df_risk_examine, df_re, df_ra

## 设置下单周范围

In [7]:
def get_week_range_v2(date):
    # 获取该日期所在周的周四
    # weekday(): 周一=0, 周二=1, ..., 周四=3, ..., 周日=6
    days_since_thursday = (date.weekday() - 3) % 7
    thursday = date - pd.Timedelta(days=days_since_thursday)
    # 获取下周的周三（周四+6天）
    next_wednesday = thursday + pd.Timedelta(days=6)
    # 格式化为"年.月.日-年.月.日"格式
    return f"{thursday.year}.{thursday.month}.{thursday.day}-{next_wednesday.year}.{next_wednesday.month}.{next_wednesday.day}"

In [8]:
# def jd_job(self, hour, minute, path, hours):
# date = (datetime.now() - timedelta(days=15)).strftime('%Y-%m-%d')# 15
# today = datetime.now().strftime('%Y%m%d')
# print(f'执行定时任务：现在是{today}的{hour}点{minute}分...')
print('正在查询数据...')
# 如果需要更改时间，还需要改这儿date_add(current_date, interval - 31 day)
sql_jd = ''' -- 京东外部订单关联表
            select  tojo.create_time, tojo.order_id, tojo.status from db_digua_business.t_order_jd_out_no tojo
            where 
            date_format(tojo.create_time, '%Y-%m-%d')>='2025-09-01'
            and date_format(tojo.create_time, '%Y-%m-%d')<='2025-09-27'
            -- and tojo.app_type='JD_APP' -- 先获取京东商城
            -- tojo.create_time>=date_add(current_date, interval - 15 day) and tojo.create_time<current_date
        '''
df_jd = clean.query(sql_jd)

df_jd.loc[:, '下单日期'] = pd.to_datetime(df_jd.create_time.dt.strftime('%Y-%m-%d'))
df_jd["下单月份"] = df_jd["create_time"].dt.strftime('%Y-%m')
df_jd.loc[:, '下单周范围'] = df_jd['下单日期'].apply(get_week_range_v2)
# print(df_jd['下单日期'].unique())
df, df_risk, df_risk_examine, df_re, df_ra = run()

print('数据查询完毕！\n正在清理数据...')
df, df2 = clean_data(df, df_risk, df_re, df_ra, key='京东')
print('数据清理完毕！\n正在计算数据...')
# print(df[(df.order_id==3564937)])
# 关联数据
df_jd_new = df_jd.merge(df[['order_id', 'order_number', 'new_actual_money', 'status2']], on='order_id', how='left')




# 区分京东渠道并计算进件和风控通过的件数--先获取京东商城&(df.app_type=='JD_APP')
df = df[(df.归属渠道=='京东渠道')]
df2 = df2[(df2.归属渠道=='京东渠道')]
df.loc[:, '下单周范围'] = df['下单日期'].apply(get_week_range_v2)
df2.loc[:, '下单周范围'] = df2['下单日期'].apply(get_week_range_v2)
df_group = all_models.data_group(df, df2, df_risk_examine, '下单日期')


# 获取京东的不去重买断价
# 如果需要更改时间，还需要改这儿date_add(current_date, interval - 31 day)
# tojr.request_json like '%IN_THE_LEASE%'表已签收，不含拒收


正在查询数据...
(364274, 41) (652284, 4) (163139, 4) (163139, 5) (174989, 4)
数据查询完毕！
正在清理数据...
数据清理完毕！
正在计算数据...


In [61]:
df_group

,去重订单数,进件数,前置拦截,机审强拒,机审通过件,进件前取消,风控通过件,人审拒绝,客户取消,已退款,无法联系,出库前风控强拒,待审核,出库,总订单数,小蚂蚁机审强拒,小蚂蚁人审拒绝,order_risk_ex,机审强拒_拒量,出库前强拒_拒量,拒量出库,拦截率,进件前取消率,预授权通过率,强拒比例,风控通过率,取消率,无法联系占比,人审拒绝率,出库前强拒比例,进件出库率,总体进件出库率（含拒量）,拒量进件出库率增加,订单出库率,人审转化率
下单日期,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2025-10-10,2120,1054,956,775.0,279.0,55,88.0,191.0,36,36,19,0.0,0,33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.09%,2.59%,49.72%,73.53%,8.35%,3.42%,1.80%,18.12%,0.00%,3.13%,3.13%,0.00%,1.56%,11.83%
2025-10-11,1781,835,827,599.0,236.0,68,91.0,145.0,36,35,17,0.0,0,38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.43%,3.82%,46.88%,71.74%,10.90%,4.31%,2.04%,17.37%,0.00%,4.55%,4.55%,0.00%,2.13%,16.10%
2025-10-12,1807,868,801,631.0,237.0,51,75.0,162.0,38,32,11,0.0,0,26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.33%,2.82%,48.04%,72.70%,8.64%,4.38%,1.27%,18.66%,0.00%,3.00%,3.00%,0.00%,1.44%,10.97%
2025-10-13,1978,965,891,702.0,263.0,48,99.0,164.0,38,35,14,0.0,0,47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.05%,2.43%,48.79%,72.75%,10.26%,3.94%,1.45%,16.99%,0.00%,4.87%,4.87%,0.00%,2.38%,17.87%
2025-10-14,1914,898,900,671.0,227.0,64,104.0,123.0,56,48,17,0.0,0,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.02%,3.34%,46.92%,74.72%,11.58%,6.24%,1.89%,13.70%,0.00%,3.45%,3.45%,0.00%,1.62%,13.66%
2025-10-15,1955,938,874,683.0,255.0,56,107.0,148.0,64,62,12,0.0,0,31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.71%,2.86%,47.98%,72.81%,11.41%,6.82%,1.28%,15.78%,0.00%,3.30%,3.30%,0.00%,1.59%,12.16%
2025-10-16,2091,958,980,679.0,279.0,66,139.0,140.0,78,67,25,0.0,0,36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46.87%,3.16%,45.82%,70.88%,14.51%,8.14%,2.61%,14.61%,0.00%,3.76%,3.76%,0.00%,1.72%,12.90%
2025-10-17,1887,917,859,661.0,256.0,57,114.0,142.0,57,45,27,0.0,2,28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45.52%,3.02%,48.60%,72.08%,12.43%,6.22%,2.94%,15.49%,0.00%,3.05%,3.05%,0.00%,1.48%,10.94%
2025-10-18,1656,843,686,595.0,248.0,57,114.0,134.0,58,51,15,0.0,3,38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.43%,3.44%,50.91%,70.58%,13.52%,6.88%,1.78%,15.90%,0.00%,4.51%,4.51%,0.00%,2.29%,15.32%


In [9]:
sql_jd_md = '''
select distinct o.order_number, od.new_actual_money 买断价, tojr.create_time, o.status, tojr.request_json
from db_digua_business.t_order as o 
left join db_digua_business.t_order_details as od on o.id = od.order_id 
left join db_digua_business.t_order_jd_request tojr on tojr.order_number=o.order_number
left join db_digua_business.t_order_jd_out_no tojo on tojo.order_id=o.id
where 
date_format(tojr.create_time, '%Y-%m-%d')>='2025-09-01'
and date_format(tojr.create_time, '%Y-%m-%d')<='2025-09-27'
-- tojr.create_time>=date_add(current_date, interval -15 day)
and tojr.request_json like '%IN_THE_LEASE%'
-- and tojo.status = 'IN_THE_LEASE'
-- and tojo.app_type='JD_APP' 
'''
df_jd_md = clean.query(sql_jd_md)
df_jd_md = df_jd_md.sort_values(by='create_time', ascending=True).groupby('order_number').head(1)
# df_jd_md.loc[:, '同步日期'] = df_jd_md.create_time.dt.strftime('%Y-%m-%d')转日期对象
df_jd_md.loc[:, '同步日期'] = pd.to_datetime(df_jd_md.create_time.dt.strftime('%Y-%m-%d'))
df_jd_md["下单月份"] = df_jd_md["create_time"].dt.strftime('%Y-%m')
df_jd_md.loc[:, 'orderSignTime'] = df_jd_md['request_json'].apply(
    lambda x: json.loads(x)['orderSignTime'] if pd.notna(x) and x != '' else None
)
# 只需要日期部分
df_jd_md['orderSignTime'] = pd.to_datetime(df_jd_md['orderSignTime'])
df_jd_md['sign_date'] = df_jd_md['orderSignTime'].dt.date
# 筛选sign_date大于等于同步日期之间的数据
df_jd_md = df_jd_md[df_jd_md['sign_date'] >= df_jd_md['同步日期']]

df_jd_md.loc[:, '下单日期'] = df_jd_md['同步日期']
df_jd_md.loc[:, '下单周范围'] = df_jd_md['同步日期'].apply(get_week_range_v2)
df_jd_md.loc[:, '买断价均值'] = df_jd_md.买断价
df_jd_md.loc[:, '合计买断价'] = df_jd_md.买断价
df_jd_md_group = df_jd_md.groupby('下单日期').agg({'order_number': 'count', '买断价均值': 'mean', '合计买断价': 'sum'}).rename(columns={'order_number': '签收数'})
df_jd_md_group.index = pd.to_datetime(df_jd_md_group.index).strftime('%Y-%m-%d')  #-%d
# 计算京东的去重，待发货，已发货和买断价数据


In [41]:
# df_jd_md_group.index = df_jd_md_group.index.strftime('%Y-%m-%d')  
df_jd_md_group#.to_excel(f'F:/需求/瑞彤需求/京东订单物流买断_1.xlsx', index=False)  1712807.9，919488.5

,签收数,买断价均值,合计买断价
下单日期,,,
2025-10-01,47,12456.227660,585442.70
2025-10-02,35,12786.454286,447525.90
2025-10-03,41,12481.856098,511756.10
2025-10-04,27,12748.281481,344203.60
2025-10-05,18,12641.906667,227554.32
2025-10-06,23,13010.873913,299250.10
2025-10-07,31,12527.219355,388343.80
2025-10-08,34,12760.518824,433857.64
2025-10-09,30,12857.440000,385723.20


In [241]:
# df_jd = df[df['归属渠道']=='京东渠道']
# df_jd2 = df2[df2['归属渠道']=='京东渠道']
# df_jd = df_jd[df_jd.下单日期>='2025-05-20']
# df_jd2 = df_jd2[df_jd2.下单日期>='2025-05-20']
# df_jd_group = all_models.data_group(df_jd, df_jd2, df_risk_examine, '下单日期')
# df_jd_group.出库.sum()

np.int64(4486)

In [289]:
# df_jdck_list = df_jd2[df_jd2.是否出库==1]['order_number'].to_list()
df_jd[(df_jd.是否出库==1)&(df_jd.order_id==3564937)]#['order_number'].to_list()
# len(df_jdck_list)

,create_time,order_id,order_number,status,create_date,status2,sku_attributes,product_name,new_actual_money,user_mobile,true_name,id_card_num,total_describes,trace_id,rejected,result,tips,is_vip,status_result,qvt_risk,qvt_result,channel_name,channel_type_id,activity_name,merchant_id,merchant_name,type,order_method,activity_id,order_type,update_time,reason,机型,scene,商品ID,phone_name,app_type,c_date,new_uv_count,uv_count,pv_count,下单日期,月份,下单月份,年份,hour,拒绝理由,取消原因,电审拒绝原因,无法联系原因,内存,机型内存,进件,来源渠道,归属渠道,status_r,status_re,time_ra,status_ra,是否前置拦截,是否机审强拒,是否出库前风控强拒,审核状态,待审核,前置拦截,人审拒绝,客户取消,无法联系,是否进件,是否出库,进件前取消,出库前风控强拒,机审强拒,机审通过件,风控通过件,已退款,是否二手,是否拒量,下单周范围


In [275]:
# df_jd2[(df_jd2.是否出库==1)&(~df_jd2['order_number'].isin(df_jdfh_list))]['order_number'].shape
# # jd方面没有，迪瓜存在出库，原因：京东方面6月9日开始同步订单，之前的数据没有同步过去，订单状态转为已完成，归还或买断

(29,)

In [ ]:
df_jd_md[df_jd_md['下单月份']=='2025-05']#.shape#.to_excel('F:\需求\崔婷玉需求\京东订单物流买断_1.xlsx', index=False)#.shape#.groupby('下单日期').agg({'买断价均值': 'mean', '合计买断价': 'sum'}).shape
# 将df_jd_md与df2左连接
# df_jd_md_merge = df_jd_md.merge(df_jd_new[['订单号','下单日期','订单id', '订单状态']], left_on='order_number',right_on='订单号', how='left')

In [ ]:
# df_jd_new[((df_jd_new['已发货']==1)|(df_jd_new['待发货']==1))&(~df_jd_new['order_number'].isin(df_jdck_list))]#['order_number']
# # 京东存在。迪瓜未找到的订单

In [10]:
# df_jd_new.loc[:, '待发货'] = np.where(df_jd_new.status=='TO_SEND_GOODS', 1, 0)
df_jd_new.loc[:, '待发货'] = np.where((df_jd_new.status=='TO_SEND_GOODS')& (df_jd_new.status2=='待发货'), 1, 0)
df_jd_new.loc[:, '已发货'] = np.where(df_jd_new.status.isin(['IN_DELIVERY', 'IN_THE_LEASE']), 1, 0)

df_jd_new.loc[:, '下单周范围'] = df_jd_new['下单日期'].apply(get_week_range_v2)



df_jd_new_group = df_jd_new.groupby('下单日期').agg({'order_id': 'count', '待发货': 'sum', '已发货': 'sum'}).rename(columns={'order_id': '创建订单数'})
# # 关联各个节点的数据
df_jd_group = df_jd_new_group.merge(df_group[['进件数', '风控通过件']], on='下单日期', how='left')
df_jd_group.loc[:, '审核通过率'] = (df_jd_group.风控通过件/df_jd_group.创建订单数).map(lambda x: format(x, '.2%'))
df_jd_group.loc[:, '风控通过率'] = (df_jd_group.风控通过件/df_jd_group.进件数).map(lambda x: format(x, '.2%'))
df_jd_group.loc[:, '发货率'] = (df_jd_group.已发货/df_jd_group.创建订单数).map(lambda x: format(x, '.2%'))
df_jd_group = df_jd_group[['创建订单数', '进件数', '风控通过件', '审核通过率', '风控通过率', '待发货', '已发货', '发货率']]
df_jd_group.index = df_jd_group.index.astype(str)
df_jd_group = pd.concat([df_jd_group, df_jd_md_group], axis=1)

print('数据计算完毕！\n正在写入数据...')
df_jd_group

数据计算完毕！
正在写入数据...


,创建订单数,进件数,风控通过件,审核通过率,风控通过率,待发货,已发货,发货率,签收数,买断价均值,合计买断价
下单日期,,,,,,,,,,,
2025-09-01,785,737,70.0,8.92%,9.50%,0,33,4.20%,20,12898.700000,257974.0
2025-09-02,846,791,77.0,9.10%,9.73%,0,30,3.55%,14,12180.864286,170532.1
2025-09-03,783,729,83.0,10.60%,11.39%,0,34,4.34%,25,11801.616000,295040.4
2025-09-04,808,757,92.0,11.39%,12.15%,0,41,5.07%,25,12422.916000,310572.9
2025-09-05,724,669,63.0,8.70%,9.42%,0,29,4.01%,25,12795.500000,319887.5
2025-09-06,673,634,54.0,8.02%,8.52%,0,25,3.71%,42,12405.588095,521034.7
2025-09-07,559,521,46.0,8.23%,8.83%,0,23,4.11%,28,12944.771429,362453.6
2025-09-08,853,796,70.0,8.21%,8.79%,0,38,4.45%,37,12700.870270,469932.2
2025-09-09,1017,952,83.0,8.16%,8.72%,0,34,3.34%,26,12577.169231,327006.4


In [43]:
date = (datetime.now() - timedelta(days=15)).strftime('%Y-%m-%d')# 15
today = datetime.now().strftime('%Y%m%d')
df_jd_group.to_excel(f'F:/需求/崔婷玉需求/京东订单租赁或买断10.1-10.27_{today}.xlsx', index=True)

## 买断优召

In [ ]:
sql_lzy = f'''
    select
    ybt.order_id 订单ID, tod.new_actual_money 买断价, too.discount_money 优惠金额, (tod.new_actual_money-too.discount_money) 实收买断金额,
    tprm.purchase_amount 采购金额, ybt.`status`, ybt.follow_log_tag, ybt.type, om.has_actual, date_format(ybt.end_time, '%Y-%m') 月份
    from db_rent.ya_buyout_task ybt
    left join (select order_id, discount_money from db_digua_business.t_order_out where discount_money>0) too on too.order_id=ybt.order_id
    left join db_digua_business.t_postlease_receivables_monitoring tprm on tprm.order_id=ybt.order_id
    left join db_digua_business.t_order_details tod on tod.order_id=ybt.order_id
    left join db_digua_business.t_order om on om.id=ybt.order_id
    where date_format(ybt.end_time, '%Y-%m')='2025-07'
    
    '''

df_lzy = clean.query(sql_lzy)

In [ ]:
df_lzy[df_lzy['订单ID'] == 2881400]

,订单ID,买断价,优惠金额,实收买断金额,采购金额,status,follow_log_tag,type,has_actual,月份
1599,2881400,15596.3,NaN,NaN,10350.0,3,0,1,1,2025-07


In [ ]:
df_lzy_tq = df_lzy[(df_lzy.status.isin([3, 4])) & (df_lzy.follow_log_tag != 4) & (df_lzy.type == 1) & (df_lzy.has_actual == 1)]
# 填充空值为0
# df_lzy_tq.fillna(0, inplace=True)
df_lzy_tq

,订单ID,买断价,优惠金额,实收买断金额,采购金额,status,follow_log_tag,type,has_actual,月份
1257,1697636,12996.7,400.0,12596.7,9700.0,3,0,1,1,2025-07
1434,2290668,12996.7,1200.0,11796.7,8810.0,4,0,1,1,2025-07
1507,2733468,12996.7,1200.0,11796.7,8240.0,3,0,1,1,2025-07
1508,1886094,10397.1,600.0,9797.1,7650.0,4,0,1,1,2025-07
1518,2350912,7797.5,200.0,7597.5,5719.0,4,0,1,1,2025-07
1528,1879510,12996.7,500.0,12496.7,9650.0,3,0,1,1,2025-07
1534,2188686,12996.7,800.0,12196.7,8910.0,4,0,1,1,2025-07
1535,2188686,12996.7,800.0,12196.7,8910.0,3,0,1,1,2025-07
1541,2389809,12997.7,1000.0,11997.7,8770.0,3,0,1,1,2025-07
1549,1275873,6719.0,0.0,6719.0,4930.0,3,0,1,1,2025-07


In [ ]:
# 优惠金额填充空值为0,实收买断金额填充空值为买断价
df_lzy_tq.loc[:, '优惠金额'] = df_lzy_tq.优惠金额.fillna(0)
# # 优惠金额填充空值为0,实收买断金额填充空值为买断价
df_lzy_tq.loc[:, '实收买断金额'] = df_lzy_tq.实收买断金额.fillna(df_lzy_tq.买断价)

df_lzy_tq


,订单ID,买断价,优惠金额,实收买断金额,采购金额,status,follow_log_tag,type,has_actual,月份
1257,1697636,12996.7,400.0,12596.7,9700.0,3,0,1,1,2025-07
1434,2290668,12996.7,1200.0,11796.7,8810.0,4,0,1,1,2025-07
1507,2733468,12996.7,1200.0,11796.7,8240.0,3,0,1,1,2025-07
1508,1886094,10397.1,600.0,9797.1,7650.0,4,0,1,1,2025-07
1518,2350912,7797.5,200.0,7597.5,5719.0,4,0,1,1,2025-07
1528,1879510,12996.7,500.0,12496.7,9650.0,3,0,1,1,2025-07
1534,2188686,12996.7,800.0,12196.7,8910.0,4,0,1,1,2025-07
1535,2188686,12996.7,800.0,12196.7,8910.0,3,0,1,1,2025-07
1541,2389809,12997.7,1000.0,11997.7,8770.0,3,0,1,1,2025-07
1549,1275873,6719.0,0.0,6719.0,4930.0,3,0,1,1,2025-07


# 发货率

In [ ]:
sql1 = '''    
        select o.order_number,od.product_id,od.product_name,o.merchant_id,o.merchant_name,o.`status`,o.create_time,op.pay_date,ol.go_express_date,
        o.alipay_order_id,cc.`name` as channel_name,pa.name as activity_name,o.order_method,cc.channel_type_id, o.order_type, o.day, o.has_order_check, pa.type,
        tpr.min_create_time, op.status as status_o
        from db_digua_business.t_order o 
        left join db_digua_business.t_order_pay op on op.order_id = o.id and op.pay_type = 'ZFBYSQ' and op.sync_mini_order = 'Y'
        left join db_digua_business.t_order_logistics ol on o.id = ol.order_id 
        left join db_digua_business.t_order_details od on od.order_id = o.id
        -- 渠道名称
        left join db_digua_business.t_channel cc on o.channel = cc.scene 
        -- 活动名称
        left join db_digua_business.t_platform_activity pa on o.activity_id = pa.id
        LEFT JOIN db_digua_business.t_merchant tmer on tmer.id = o.merchant_id
        left join (SELECT tpr.order_id, min(tpr.create_time) as min_create_time FROM db_digua_business.t_pay_record tpr GROUP BY tpr.order_id) tpr on tpr.order_id=o.id
        where o.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -15 day ) 
        -- and tmer.shop_type!=2
        and o.user_mobile is not null 
        '''

df_order = clean.query(sql1)
df_order.shape

(382534, 20)

In [ ]:
df = df_order.copy()

df["下单日期"] = pd.to_datetime(df["create_time"]).dt.date
df["下单日期"] = pd.to_datetime(df["下单日期"], errors="coerce")
df["月份"] = df["下单日期"].dt.month
df.loc[:, "来源渠道"] = df["channel_name"].fillna("未知渠道")
df.loc[:, "归属渠道"] = df.apply(lambda x: clean.qudao_type(x["来源渠道"], x["activity_name"], x["order_method"], x['channel_type_id'], x['order_type']),axis=1)

df.drop(df[df['activity_name'] == "1000单秘密计划"].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单秘密计划-无优惠"].index, inplace=True)
df.drop(df[df['activity_name'] == "1000单曙光计划"].index, inplace=True)
df.drop(df[df['activity_name'] == "线下门店3个月试行"].index, inplace=True)
df.drop_duplicates(subset=["order_number"], inplace=True)
# df = df[df.status_o.isin([2, 5])]# , 5
# df = df.reset_index(drop=True)

df.shape

(74325, 24)

In [ ]:
# 原回捞逻辑订单数
# df[df.order_id == 3538653]
df_2412 = df[df.tips.str.contains(r'策略241205|策略241212|命中自有模型回捞策略|回捞策略250330命中', regex=True)==True]
df_2412.shape# 1839
df_2412.loc[:, '机审强拒_拒量'] = np.where(df_2412.qvt_risk=='1', 1, 0)
df_2412[df_2412.机审强拒_拒量==0].head()#.shape


In [ ]:
# 原回捞逻辑订单数
# df[df.order_id == 3538653]
df_2412 = df[df.tips.str.contains(r'策略241205|策略241212|命中自有模型回捞策略|回捞策略250330命中', regex=True)==True]
df_2412.shape# 1839
df_2412.loc[:, '机审强拒_拒量'] = np.where(df_2412.qvt_risk=='1', 1, 0)
df_2412[df_2412.机审强拒_拒量==0].head()#.shape


In [ ]:
def xcx( df):
    # 筛选所需要的字段
    # df = df[df.status_o.isin([2, 5])]
    df_all = df[['order_number', 'product_id', 'product_name', 'merchant_id','merchant_name', 'status', 'create_time', 'pay_date', 'go_express_date',
                'alipay_order_id', 'channel_name','下单日期', '月份', '来源渠道', '归属渠道']]
    df_all = df_all.drop_duplicates(subset=['order_number'])
    # 取到日期并对字段进行重命名
    df_all['create_time'] = df_all['create_time'].dt.date
    df_all['pay_date'] = df_all['pay_date'].dt.date
    df_all['go_express_date'] = df_all['go_express_date'].dt.date
    df_all.rename(columns={'order_number': '订单编号', 'product_id': '商品ID', 'product_name': '商品名称','merchant_id': '店铺ID', 'merchant_name': '店铺名称',
                        'status': '订单状态', 'create_time': '订单生成时间', 'pay_date': '支付时间','go_express_date': '发货时间', 'alipay_order_id': '支付宝流水号'}, inplace=True)
    df_all['发货时效'] = (pd.to_datetime(df_all['发货时间']) - pd.to_datetime(df_all['支付时间'])).dt.days
    # 筛选7天发货时效并计算发货率l
    df_all.loc[:, '发货数'] = np.where(pd.to_datetime(df_all['发货时间'])-pd.to_datetime(df_all['支付时间'])<=pd.Timedelta(hours=168), 1, 0)
    df_all_zh = df_all.groupby('订单生成时间').agg({'订单编号': 'size', '发货数': 'sum'})
    df_all_zh.loc['汇总', :] = df_all_zh.sum(axis=0)
    df_all_zh['发货率'] = (df_all_zh['发货数'] / df_all_zh['订单编号'] * 100).apply(lambda x: f'{x: .2f}%')
    df_all_zh.rename(columns={'订单编号': '进件量'}, inplace=True)

    return df_all_zh

In [ ]:
def get_data(df):
    # 近30天发货率
    # 芝麻租物
    df_zm = df[df['归属渠道'] == '芝麻租物']
    print(len(df_zm))
    df_zm_zh_30 = xcx(df_zm)
    # 搜索渠道
    df_ss = df[df['归属渠道'] == '搜索渠道']
    print(len(df_ss))
    df_ss_zh_30 = xcx(df_ss)

    # # 近7天发货率
    # # 芝麻租物
    # df_zm_7 = df_zm[df_zm.下单日期>=(datetime.now()-timedelta(days=7))]
    # df_zm_zh_7 = xcx(df_zm_7)
    # # 搜索渠道
    # df_ss_7 = df_ss[df_ss.下单日期 >= (datetime.now() - timedelta(days=7))]
    # df_ss_zh_7 = xcx(df_ss_7)

    # # 48小时发货率 总体
    # df_48h = fh48(df)
    # # 芝麻租物
    # df_zm_48h = fh48(df_zm)
    # # 搜索渠道
    # df_ss_48h = fh48(df_ss)
    return  df_zm_zh_30, df_ss_zh_30#, df_zm_zh_7, df_ss_zh_7, df_48h, df_zm_48h, df_ss_48h

df_zm_zh_30, df_ss_zh_30 = get_data(df)

20882
11162


In [ ]:
df_zm_zh_30#[-6:]

In [ ]:
df_ss_zh_30#[-5:]

# 彤姐需求

## 支付宝“品牌”出库数+占比长租 短租出库+占比，不分渠道、不分自营商户。

In [17]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status, om.user_id
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money, tod.product_id
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile
,top.total_describes
,cc.name as channel_name         -- 来源渠道
,cc.channel_type_id              -- 渠道id
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
, tod.dy_order_item_json, pa.type
,om.buy_service_product
,om.order_method, om.activity_id, om.appid
,om.order_type, om.union_rent_tag
,tp.classify_id, tb.name 品牌
,tmu.true_name,tmu.id_card_num, om.day 租赁天数
from  db_digua_business.t_order  om
-- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id

-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id

-- 商品表
left join db_digua_business.t_product tp on tp.id=tod.product_id -- 8942.97MB

-- 品牌表
left join db_digua_business.t_brand tb on tb.id=tp.brand_id


where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静','陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')

# and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -20 day )               -- 近20天数据
# and  om.create_time <= DATE_ADD(CURRENT_DATE,INTERVAL -0 day )
and DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-05-01'
and DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-09-30'
and om.order_type='ZFB_ORDER'

'''

df_order = clean.query(sql1)
df_order.shape

(1310476, 34)

In [28]:
# 支付宝出库，不分渠道、不分自营商户。
# “品牌”出库数+占比
# 长租 短租出库+占比
df = df_order.copy()
df = df[df.type!=4]
df.order_id.notnull().sum()

np.int64(1310476)

### 备注

In [37]:
# df.品牌.unique()


array(['苹果', '华为', '小米', 'vivo', '大疆', '佳能', '富士', '一加', 'oppo', '荣耀',
       '联想', '任天堂', None, 'iQOO', '戴尔', '索尼', '探路者', '真我realme', '华硕',
       '机械革命', '惠普', 'GoPro', 'steam', '三星', '影石instax360', '雷神',
       '宝丽来Polaroid', '漫游超人', 'BOSE', '智云', '理光', '贝瑞克', '致佳'],
      dtype=object)

In [30]:
df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["月份"]=df["下单日期"].dt.month
df['date']=df['create_time'].dt.date
df['hour']=df['create_time'].dt.hour
df["是否出库"] = np.where(df["status"].isin([2,3,4,5,6,8,15]),1,0)
df.loc[:, '租赁类型'] = np.where(df.租赁天数 >= 90, '长租', '短租')

### 去重

In [31]:
df = clean.order_drop_duplicates(df)
df.shape

(794595, 41)

### 剔除商家只保留自营订单

In [33]:
# df = clean.drop_merchant(df)
# reject_merchants = ["小蚂蚁租机", "兴鑫兴通讯", "人人享租", "崇胜数码", "喜卓灵租机", "喜卓灵新租机"]
# df.drop(df[df['merchant_name'].isin(reject_merchants)].index, inplace=True)
# df.shape
df.drop(df[df['merchant_name'] == "线下小店"].index, inplace=True)
df.shape

(789172, 41)

In [49]:
df.columns

Index(['create_date', 'create_time', 'order_id', 'order_number', 'all_money',
       'status', 'user_id', 'status2', 'channel_type', 'sku_attributes',
       'product_name', 'new_actual_money', 'product_id', 'back_type',
       'user_mobile', 'total_describes', 'channel_name', 'channel_type_id',
       'activity_name', 'merchant_id', 'merchant_name', 'dy_order_item_json',
       'type', 'buy_service_product', 'order_method', 'activity_id', 'appid',
       'order_type', 'union_rent_tag', 'classify_id', '品牌', 'true_name',
       'id_card_num', '租赁天数', '是否出库', '租赁类型', '状态编码', '下单日期', '月份', 'date',
       'hour'],
      dtype='object')

In [34]:
# 按“品牌”分组计算出库数以及出库数占比
df_brand_g = df.groupby(['品牌']).agg({'是否出库': 'sum'}).rename(columns={'是否出库': '出库数'}).reset_index()
# 计算出库数占比
df_brand_g['出库数占比'] = (df_brand_g['出库数'] / df_brand_g['出库数'].sum() * 100).apply(lambda x: f'{x: .2f}%')

df_brand_g

,品牌,出库数,出库数占比
0,BOSE,3,0.01%
1,GoPro,22,0.10%
2,iQOO,5,0.02%
3,oppo,9,0.04%
4,steam,0,0.00%
5,vivo,11,0.05%
6,一加,1,0.00%
7,三星,0,0.00%
8,任天堂,28,0.13%
9,佳能,59,0.27%


In [35]:

# 按“租赁类型”分组计算出库数以及出库数占比
df_type_g = df.groupby(['租赁类型']).agg({'是否出库': 'sum'}).rename(columns={'是否出库': '出库数'}).reset_index()
# 计算出库数占比
df_type_g['出库数占比'] = (df_type_g['出库数'] / df_type_g['出库数'].sum() * 100).apply(lambda x: f'{x: .2f}%')
df_type_g

,租赁类型,出库数,出库数占比
0,短租,209,0.95%
1,长租,21828,99.05%


In [36]:
path = 'F:\需求\瑞彤需求/'
with pd.ExcelWriter(path + f'支付宝非线下小店出库订单数.xlsx', engine='openpyxl') as writer:
    df_brand_g.to_excel(writer, sheet_name='品牌出库数', index=False)
    df_type_g.to_excel(writer, sheet_name='租赁类型出库数', index=False) 

## 迪瓜租手机芝麻租赁频道异常交易核对（匹配订单）

In [ ]:
# 仅保留核心逻辑
df_ali_order = pd.read_excel('F:/需求/瑞彤需求/迪瓜租手机芝麻租赁频道异常交易核对.xlsx')
df_ali_orderid_list = df_ali_order['order_id'].astype(str).tolist()
df_ali_new = df[df['alipay_order_id'].isin(df_ali_orderid_list)]

df_ali_merge = df_ali_order.merge(df_ali_new[['order_number','alipay_order_id','下单日期', '来源渠道', '归属渠道', 'status2', 'all_money']], left_on='order_id', right_on='alipay_order_id', how='left')
df_ali_merge.rename(columns={'order_number': '订单号', 'status2': '订单状态', 'all_money': '总租金', 'alipay_order_id': '支付宝订单号'}, inplace=True)
df_ali_merge.to_excel('F:/需求/瑞彤需求/迪瓜租手机芝麻租赁频道异常交易核对结果.xlsx', index=False)



## 支付宝订单匹配扣款时间，发货时间

In [111]:
df_ali_order = pd.read_excel('F:\需求\瑞彤需求\支付宝订单号.xlsx')
df_ali_order_list = df_ali_order['支付宝订单号'].astype(str).tolist()
order_list_placeholder = ','.join(['%s'] * len(df_ali_order_list))

sql_pp = '''
select om.order_number, om.alipay_order_id, tpr.create_time 订单扣款发起时间, tpr.payment_date 实际扣款时间-- ,tol.go_express_date 发货时间
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货\归还中" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2
, om.all_money
from db_digua_business.t_order om
left join db_digua_business.t_pay_record tpr on tpr.order_id=om.id
-- left join db_digua_business.t_order_logistics tol on tol.order_id=om.id

-- left join db_digua_business.t_order_stages tos 
where DATE_FORMAT(om.create_time, '%Y-%m-%d')>='2025-06-01'
and  DATE_FORMAT(om.create_time, '%Y-%m-%d')<='2025-09-31'
-- and om.alipay_order_id in (order_list_placeholder)
'''
df_pp = clean.query(sql_pp)
df_pp.shape

(1509620, 6)

In [112]:
df_pp_ali = df_pp[df_pp['alipay_order_id'].isin(df_ali_order_list)]#.sort_values('实际扣款时间', ascending=True).head(1)
# df_pp_ali.alipay_order_id.value_counts().sort_values(ascending=False)
# # 如果存在多条数据且存在实际扣款时间，则选择实际扣款时间最早的记录，若不存在实际扣款时间则选择订单发起时间最晚的记录
# df_pp_ali_t = df_pp_ali.sort_values('实际扣款时间', ascending=True).drop_duplicates('alipay_order_id', keep='first')
# # df_pp_ali[df_pp_ali['alipay_order_id']=='2025100401502300000004150082652340']
# len(df_pp_ali_t)
# 分离有实际扣款时间和无实际扣款时间的数据
# has_payment_time = df_pp_ali[df_pp_ali['实际扣款时间'].notna()]
# no_payment_time = df_pp_ali[df_pp_ali['实际扣款时间'].isna()]

# # 对有实际扣款时间的数据，按实际扣款时间升序排列，保留最早的一条
# has_payment_dedup = has_payment_time.sort_values('实际扣款时间', ascending=True).drop_duplicates('alipay_order_id', keep='first')

# # 对无实际扣款时间的数据，按订单发起时间降序排列，保留最晚的一条
# no_payment_dedup = no_payment_time.sort_values('订单扣款发起时间', ascending=False).drop_duplicates('alipay_order_id', keep='first')

# # 合并两部分结果
# df_pp_ali_t = pd.concat([has_payment_dedup, no_payment_dedup])
# df_pp_ali

,order_number,alipay_order_id,订单扣款发起时间,实际扣款时间,status2,all_money
103047,A20250610143459599,2025061001502300000001870024855696,2025-06-10 15:08:55,2025-06-10 15:08:56,已完成,5708.63
103048,A20250610143459599,2025061001502300000001870024855696,2025-06-10 15:09:04,2025-06-10 15:09:06,已完成,5708.63
103049,A20250610143459599,2025061001502300000001870024855696,2025-06-10 15:09:13,2025-06-10 15:09:15,已完成,5708.63
103050,A20250610143459599,2025061001502300000001870024855696,2025-06-16 14:40:52,2025-06-16 14:41:37,已完成,5708.63
116019,A20250611145053771,2025061101502300000008370039352545,2025-06-11 15:09:21,2025-06-11 15:33:44,已完成,5622.40
...,...,...,...,...,...,...
969494,A20250812172634651,2025081201502300000003940062235203,2025-08-15 09:08:37,2025-08-15 09:09:27,已完成,5978.05
976318,A20250813102733409,2025081301502300000009530067947532,2025-08-13 11:11:26,2025-08-13 11:11:26,已完成,5978.05
976319,A20250813102733409,2025081301502300000009530067947532,2025-08-13 11:11:34,2025-08-13 11:11:37,已完成,5978.05
976320,A20250813102733409,2025081301502300000009530067947532,2025-08-13 11:11:49,2025-08-13 11:11:51,已完成,5978.05


In [115]:
df_pp_ali.drop_duplicates(subset=["order_number"], inplace=True)
df_pp_ali.shape

(146, 6)

In [116]:

df_pp_ali[['order_number', 'alipay_order_id', 'all_money']].to_excel('F:/需求/瑞彤需求/匹配支付宝订单号.xlsx', index=False)

In [92]:
# df_ali_merge =  df_ali_order.merge(df_pp_ali_t, left_on='订单号', right_on='alipay_order_id', how='left')
# df_ali_merge.to_excel('F:/需求/瑞彤需求/迪瓜匹配订单号.xlsx', index=False)

## 25年1月起 每月，优惠券领取金额+实际使用金额

In [2]:
sql_yh = '''
select
tdi.order_id,tdi.id,too.order_id too_order_id,
tdi.money tdi_money, tdi.status tdi_status
,tdi.use_type, tdi.type, tdi.create_time 优惠订单创建时间

from db_digua_business.t_discount_item tdi 
left join db_digua_business.t_order_out too on too.discount_item_id = tdi.id
where date_format(tdi.create_time, '%Y-%m')>='2025-01'
-- and tdi.order_id is not null
-- and tos.sort = 1
-- and pa.type!=4 
-- and tdi.type = 2
;
'''
# and date_format(om.create_time, '%Y-%m-%d') >= date_format(turt.create_time, '%Y-%m-%d')
# and date_format(om.create_time, '%Y-%m-%d') <= date_format(turt.complete_time, '%Y-%m-%d')
# and turt.order_number is not null
df_yhz = clean.query(sql_yh)
df_yhz.shape
df_yhz.order_id.notnull().sum()
# tdi.order_id 为null可分为两类，未下单或者买断优惠券

(3821158, 8)

np.int64(1197467)

In [3]:
sql_order = '''
select om.status,om.merchant_name, om.create_time, om.id order_id
,pa.name as activity_name 
, tos.money 分期金额, tos.sort, tos.real_pay_money
from db_digua_business.t_order om 
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
left join db_digua_business.t_order_stages tos on tos.order_id=om.id
where date_format(om.create_time, '%Y-%m')>='2025-01'
;
'''
df_order = clean.query(sql_order)
df_order.shape

(27757077, 8)

### 合并数据

In [4]:
df_yhz['order_id'] = np.where(df_yhz.order_id.isna(), df_yhz.too_order_id, df_yhz.order_id)
df_yh_merge = df_yhz.merge(df_order, on='order_id', how='left')
df_yh_merge.shape

(17003184, 15)

### 数据处理

In [5]:
# 删除拒量及商家数据
def drop_merchant(df):
    '''
    删除商家数据
    :param df: 传入带有商家的数据
    :return: 返回剔除了商家的数据
    '''
    # 剔除商家数据只保留自营租机业务数据
    df.drop(df[df['merchant_name'] == "深圳优优大数据科技有限公司"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "优优2店"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "小豚租（代收）"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "苏州蚁诺宝"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "租着用电脑数码"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "北京海鸟窝科技有限公司"].index, inplace=True)

    df.drop(df[df['merchant_name'] == "汇客好租"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "澄心优租"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "CPS渠道合作"].index, inplace=True)
    # df.drop(df[df['sku_attributes'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)
    # 趣智数码  单
    df.drop(df[df['merchant_name'] == "趣智数码"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "格木木二奢名品"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "广州康基贸易有限公司"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "线下小店"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "乙辉数码"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "呱子笔记本电脑"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "南京聚格网络科技"].index, inplace=True)
    
    df.drop(df[df['merchant_name'] == "星晟数码"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "蘑菇时间"].index, inplace=True)
    df.drop(df[df['merchant_name'] == "云启德曜"].index, inplace=True)

    df.drop(df[df['merchant_name'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)
    
    reject_merchants = ["小蚂蚁租机", "兴鑫兴通讯", "人人享租", "崇胜数码", "喜卓灵租机", "喜卓灵新租机"]
    df.drop(df[df['merchant_name'].isin(reject_merchants)].index, inplace=True)
    return df
df_yh = df_yh_merge.copy()
df_yh = drop_merchant(df_yh)
df_yh.shape

(15940463, 15)

In [11]:
df_yh[(df_yh.too_order_id.notna())]

,order_id,id,too_order_id,tdi_money,tdi_status,use_type,type,优惠订单创建时间,status,merchant_name,create_time,activity_name,分期金额,sort,real_pay_money
14902,984005.0,5853402,984005.0,450.0,3,1,4,2025-01-02 10:05:26,NaN,NaN,NaT,NaN,NaN,NaN,NaN
14903,984005.0,5853402,984005.0,450.0,3,1,4,2025-01-02 10:05:26,NaN,NaN,NaT,NaN,NaN,NaN,NaN
16991,864607.0,5854082,864607.0,450.0,3,1,4,2025-01-02 11:03:46,NaN,NaN,NaT,NaN,NaN,NaN,NaN
27232,1024342.0,5857404,1024342.0,866.0,3,1,4,2025-01-02 14:52:55,NaN,NaN,NaT,NaN,NaN,NaN,NaN
27233,1024342.0,5857404,1024342.0,866.0,3,1,4,2025-01-02 14:52:55,NaN,NaN,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16976006,3886740.0,9664097,3886740.0,1100.0,1,1,4,2025-10-20 10:12:16,4.0,晋江玖盛店,2025-09-09 14:01:42,京东活动,726.44,11.0,0.00
16976007,3886740.0,9664097,3886740.0,1100.0,1,1,4,2025-10-20 10:12:16,4.0,晋江玖盛店,2025-09-09 14:01:42,京东活动,728.26,12.0,728.26
16976172,1615911.0,9664130,1615911.0,700.0,3,1,4,2025-10-20 10:14:33,NaN,NaN,NaT,NaN,NaN,NaN,NaN
16989849,1647905.0,9666895,1647905.0,800.0,3,1,4,2025-10-20 12:51:32,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [14]:
def order_drop_duplicates(df):
    
    # 曙光计划和线下小店是同分异构体
    df.drop(df[df['activity_name'] == "1000单秘密计划"].index, inplace=True)
    df.drop(df[df['activity_name'] == "1000单秘密计划-无优惠"].index, inplace=True)
    df.drop(df[df['activity_name'] == "1000单曙光计划"].index, inplace=True)
    df.drop(df[df['activity_name'] == "线下门店3个月试行"].index, inplace=True)
    # 删除订单状态空值行
    df.dropna(subset=["status"], axis=0, inplace=True)
    
    return df
df_yh = order_drop_duplicates(df_yh)
df_yh.shape#(13066296, 13)
df_yh.order_id.notnull().sum()

(13317072, 15)

np.int64(13317072)

In [17]:
df_yh[df_yh.order_id.isna()]
df_yh.sort_values(['order_id', 'sort','use_type'], ascending=[True, True,True])

,order_id,id,too_order_id,tdi_money,tdi_status,use_type,type,优惠订单创建时间,status,merchant_name,create_time,activity_name,分期金额,sort,real_pay_money


,order_id,id,too_order_id,tdi_money,tdi_status,use_type,type,优惠订单创建时间,status,merchant_name,create_time,activity_name,分期金额,sort,real_pay_money
16,1943248.0,5848394,NaN,1.0,3,4,2,2025-01-01 00:15:17,10.0,迪瓜优选,2025-01-01 00:15:17,芝麻租物-苹果16系列,1.00,1.0,0.0
17,1943248.0,5848394,NaN,1.0,3,4,2,2025-01-01 00:15:17,10.0,迪瓜优选,2025-01-01 00:15:17,芝麻租物-苹果16系列,535.86,2.0,0.0
18,1943248.0,5848394,NaN,1.0,3,4,2,2025-01-01 00:15:17,10.0,迪瓜优选,2025-01-01 00:15:17,芝麻租物-苹果16系列,535.86,3.0,0.0
19,1943248.0,5848394,NaN,1.0,3,4,2,2025-01-01 00:15:17,10.0,迪瓜优选,2025-01-01 00:15:17,芝麻租物-苹果16系列,535.86,4.0,0.0
20,1943248.0,5848394,NaN,1.0,3,4,2,2025-01-01 00:15:17,10.0,迪瓜优选,2025-01-01 00:15:17,芝麻租物-苹果16系列,535.86,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17003088,4274024.0,9669226,NaN,1.0,2,4,2,2025-10-20 14:51:00,13.0,迪瓜优选,2025-10-20 14:51:45,京东校园开学季,599.75,8.0,0.0
17003089,4274024.0,9669226,NaN,1.0,2,4,2,2025-10-20 14:51:00,13.0,迪瓜优选,2025-10-20 14:51:45,京东校园开学季,599.75,9.0,0.0
17003090,4274024.0,9669226,NaN,1.0,2,4,2,2025-10-20 14:51:00,13.0,迪瓜优选,2025-10-20 14:51:45,京东校园开学季,599.75,10.0,0.0
17003091,4274024.0,9669226,NaN,1.0,2,4,2,2025-10-20 14:51:00,13.0,迪瓜优选,2025-10-20 14:51:45,京东校园开学季,599.75,11.0,0.0


In [51]:
df_yh_c = df_yh.copy()
# 买断优惠劵没有order_id,反向关联t_order_out表获取order_id填充

duplicates = df_yh_c[df_yh_c.duplicated(subset=['order_id', 'sort'], keep=False)]
if not duplicates.empty:
    print("发现重复数据，数量：", len(duplicates))
    # 可以选择保留最新的记录（根据实际业务逻辑调整）
    # 这里假设reality_refund_date不为空的是更新的记录
    df_yh_c = df_yh_c.sort_values(['order_id', 'sort','use_type'], ascending=[True, True,True])
    
    # 去重，保留每个(order_id, sort)的第一条记录（即最新的）
    # df_yh_c = df_yh_c.drop_duplicates(subset=['order_id','use_type'], keep='first')
# df_yh_c.drop_duplicates(subset=["order_id"], inplace=True)
# df_yh_c.drop_duplicates(subset=["order_number"], inplace=True)
df_yh_c.shape
df_yh_c.order_id.notnull().sum()
df_yh_c

发现重复数据，数量： 5580


(13317072, 15)

np.int64(13317072)

,order_id,id,too_order_id,tdi_money,tdi_status,use_type,type,优惠订单创建时间,status,merchant_name,create_time,activity_name,分期金额,sort,real_pay_money
16,1943248.0,5848394,NaN,1.0,3,4,2,2025-01-01 00:15:17,10.0,迪瓜优选,2025-01-01 00:15:17,芝麻租物-苹果16系列,1.00,1.0,0.0
17,1943248.0,5848394,NaN,1.0,3,4,2,2025-01-01 00:15:17,10.0,迪瓜优选,2025-01-01 00:15:17,芝麻租物-苹果16系列,535.86,2.0,0.0
18,1943248.0,5848394,NaN,1.0,3,4,2,2025-01-01 00:15:17,10.0,迪瓜优选,2025-01-01 00:15:17,芝麻租物-苹果16系列,535.86,3.0,0.0
19,1943248.0,5848394,NaN,1.0,3,4,2,2025-01-01 00:15:17,10.0,迪瓜优选,2025-01-01 00:15:17,芝麻租物-苹果16系列,535.86,4.0,0.0
20,1943248.0,5848394,NaN,1.0,3,4,2,2025-01-01 00:15:17,10.0,迪瓜优选,2025-01-01 00:15:17,芝麻租物-苹果16系列,535.86,5.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17003088,4274024.0,9669226,NaN,1.0,2,4,2,2025-10-20 14:51:00,13.0,迪瓜优选,2025-10-20 14:51:45,京东校园开学季,599.75,8.0,0.0
17003089,4274024.0,9669226,NaN,1.0,2,4,2,2025-10-20 14:51:00,13.0,迪瓜优选,2025-10-20 14:51:45,京东校园开学季,599.75,9.0,0.0
17003090,4274024.0,9669226,NaN,1.0,2,4,2,2025-10-20 14:51:00,13.0,迪瓜优选,2025-10-20 14:51:45,京东校园开学季,599.75,10.0,0.0
17003091,4274024.0,9669226,NaN,1.0,2,4,2,2025-10-20 14:51:00,13.0,迪瓜优选,2025-10-20 14:51:45,京东校园开学季,599.75,11.0,0.0


### 所有优惠券种金额

In [ ]:
# df_yh_c = df_yh_c[df_yh_c.type!=4]
# df_yh_c["下单月份"]=df_yh_c["优惠订单创建时间"].dt.strftime('%Y-%m')
df_yh_c["下单月份"]=df_yh_c["create_time"].dt.strftime('%Y-%m')
# df_yh_c.drop_duplicates(subset=["order_id"], inplace=True)
df_yh_c .loc[:, '是否出库'] = np.where(df_yh_c ["status"].isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)
df_yh_c.loc[:,'最大分期金额'] = df_yh_c.groupby('order_id')['分期金额'].transform('max').fillna(0)
# df_yh_c.loc[:,'出库订单领取优惠金额'] =  np.where(df_yh_c.是否出库==1, np.where(df_yh_c.use_type.isin([2, 4]),df_yh_c.最大分期金额-df_yh_c.分期金额,np.where(df_yh_c.use_type==1, df_yh_c.tdi_money, 0)),0)
df_yh_c.loc[:, '优惠金额'] = np.where(df_yh_c.tdi_status==3, np.where(df_yh_c.use_type.isin([2, 4]),df_yh_c.最大分期金额-df_yh_c.分期金额,np.where(df_yh_c.use_type==1, df_yh_c.tdi_money, 0)),0)
df_yh_c.loc[:, '出库订单优惠金额'] = np.where(df_yh_c.是否出库==1, df_yh_c.优惠金额, 0)

df_yh_c.sort_values(['order_id', 'sort'], ascending=[True, True])
df_yh_c = df_yh_c.drop_duplicates(subset=['order_id'], keep='first')
# 存在优惠券状态为失效但实际使用了优惠券，补全这批订单
df_yh_c.loc[:, '优惠状态'] = np.where((df_yh_c.tdi_status==3)|((df_yh_c.tdi_status==4)&(df_yh_c.是否出库==1)), 1, 0)
df_yh_c.loc[:,'是否使用优惠'] = np.where(df_yh_c.优惠状态==1, 1, 0)
df_yh_c.loc[:,'出库是否使用优惠'] = np.where(df_yh_c.是否出库==1, df_yh_c.是否使用优惠, 0)


In [33]:
df_yh_g = df_yh_c.groupby(['下单月份']).agg({'优惠金额': 'sum','是否使用优惠':'sum','出库订单优惠金额':'sum', '是否出库':'sum','出库是否使用优惠':'sum'}
                                        ).rename(columns={'优惠金额':'总体优惠金额','是否出库':'出库数','是否使用优惠':'优惠单数','出库是否使用优惠':'出库使用优惠单数'})
df_yh_g['平均优惠金额'] = df_yh_g['总体优惠金额']/df_yh_g['优惠单数']
df_yh_g['平均出库使用优惠金额'] = df_yh_g['出库订单优惠金额']/df_yh_g['出库使用优惠单数']
df_yh_g.loc["汇总", :] = df_yh_g.select_dtypes(include=[np.number]).sum()
df_yh_g

,总体优惠金额,优惠单数,出库订单优惠金额,出库数,出库使用优惠单数,平均优惠金额,平均出库使用优惠金额
下单月份,,,,,,,
2025-01,6.796357e+06,11261.0,824074.21,1481.0,1481.0,603.530518,556.430932
2025-02,6.468049e+06,11452.0,840267.06,1659.0,1659.0,564.796495,506.490090
2025-03,7.704650e+06,14173.0,1120157.48,2235.0,2235.0,543.614617,501.189029
2025-04,1.058776e+07,20437.0,1692392.29,3499.0,3499.0,518.068213,483.678848
2025-05,1.353686e+07,24594.0,2090504.39,3930.0,3930.0,550.412907,531.934959
2025-06,1.903425e+07,35567.0,2274267.35,4372.0,4372.0,535.165899,520.189238
2025-07,2.599840e+07,47544.0,2725961.40,5068.0,5068.0,546.828275,537.877151
2025-08,3.077955e+07,52699.0,2624557.60,4536.0,4536.0,584.063269,578.606173
2025-09,2.386341e+07,37395.0,2094402.45,3306.0,3305.0,638.144310,633.707247


In [ ]:
# df_yh_c[(df_yh_c.是否出库==1)&(df_yh_c.是否使用优惠==0)]['order_id']
df_yh[df_yh['order_id']==3886740]

In [36]:
# df_yh_c[df_yh_c.order_id==4233974]#.tail()
df_yh_g.to_excel('F:\需求\瑞彤需求\优惠券.xlsx')


### 买断优惠券

In [46]:

del df_yh_merge

### 首期使用优惠券

In [53]:
df_yh_sy = df_yh_c[df_yh_c.type==2] .copy()
df_yh_sy ["下单月份"]=df_yh_sy ["优惠订单创建时间"].dt.strftime('%Y-%m')
df_yh_sy .loc[:, '是否出库'] = np.where(df_yh_sy ["status"].isin([2, 3, 4, 5, 6, 8, 15]), 1, 0)
df_yh_sy = df_yh_sy[df_yh_sy['是否出库']==1]
df_yh_sy .loc[:,'最大分期金额'] = df_yh_sy .groupby('order_id')['分期金额'].transform('max').fillna(0)
df_yh_sy .loc[:, '优惠金额'] = np.where(df_yh_sy .tdi_status==3, np.where(df_yh_sy .use_type.isin([2, 4]),df_yh_sy .最大分期金额-df_yh_sy .分期金额,np.where(df_yh_sy .use_type==1, df_yh_sy .tdi_money, 0)),0)
df_yh_sy .sort_values(['order_id', 'sort'], ascending=[True, True])
df_yh_sy  = df_yh_sy .drop_duplicates(subset=['order_id'], keep='first')
# df_yh_sy .loc[:, '优惠金额'] = np.where(df_yh_sy .tdi_status==3, np.where((df_yh_sy .use_type == 1), df_yh_sy .tdi_money,
#                                 np.where((df_yh_sy .use_type == 2), df_yh_sy .分期金额 * (1 - df_yh_sy .tdi_money / 100), 0)), 0)
df_yh_sy.loc[:, '优惠状态'] = np.where((df_yh_sy.tdi_status==3)|((df_yh_sy.tdi_status==4)&(df_yh_sy.是否出库==1)), 1, 0)
df_yh_sy.loc[:,'是否使用优惠'] = np.where(df_yh_sy.优惠状态==1, 1, 0)
df_yhsy_g = df_yh_sy .groupby(['下单月份']).agg({'优惠金额': 'sum','是否使用优惠':'sum','是否出库':'sum'}).rename(columns={'优惠金额':'首期优惠金额','是否使用优惠':'优惠单数','是否出库':'出库'})
df_yhsy_g['平均优惠金额'] = df_yhsy_g['首期优惠金额']/df_yhsy_g['优惠单数']
df_yhsy_g.loc['汇总', :] = df_yhsy_g.sum()
df_yhsy_g#1943248

,order_id,id,too_order_id,tdi_money,tdi_status,use_type,type,优惠订单创建时间,status,merchant_name,create_time,activity_name,分期金额,sort,real_pay_money,下单月份,是否出库,最大分期金额,优惠金额
28,1943249.0,5848395,NaN,1.0,3,4,2,2025-01-01 00:16:13,4.0,天翊科技店,2025-01-01 00:16:13,芝麻租物-苹果16系列,1.00,1.0,1.00,2025-01,1,332.06,331.06
29,1943249.0,5848395,NaN,1.0,3,4,2,2025-01-01 00:16:13,4.0,天翊科技店,2025-01-01 00:16:13,芝麻租物-苹果16系列,331.44,2.0,331.44,2025-01,1,332.06,0.62
30,1943249.0,5848395,NaN,1.0,3,4,2,2025-01-01 00:16:13,4.0,天翊科技店,2025-01-01 00:16:13,芝麻租物-苹果16系列,331.44,3.0,331.44,2025-01,1,332.06,0.62
31,1943249.0,5848395,NaN,1.0,3,4,2,2025-01-01 00:16:13,4.0,天翊科技店,2025-01-01 00:16:13,芝麻租物-苹果16系列,331.44,4.0,331.44,2025-01,1,332.06,0.62
32,1943249.0,5848395,NaN,1.0,3,4,2,2025-01-01 00:16:13,4.0,天翊科技店,2025-01-01 00:16:13,芝麻租物-苹果16系列,331.44,5.0,331.44,2025-01,1,332.06,0.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16999726,4273623.0,9668691,NaN,1.0,3,4,2,2025-10-20 14:31:22,2.0,厦门共勋商贸,2025-10-20 14:31:34,京东活动,1082.94,8.0,0.00,2025-10,1,1085.36,2.42
16999727,4273623.0,9668691,NaN,1.0,3,4,2,2025-10-20 14:31:22,2.0,厦门共勋商贸,2025-10-20 14:31:34,京东活动,1082.94,9.0,0.00,2025-10,1,1085.36,2.42
16999728,4273623.0,9668691,NaN,1.0,3,4,2,2025-10-20 14:31:22,2.0,厦门共勋商贸,2025-10-20 14:31:34,京东活动,1082.94,10.0,0.00,2025-10,1,1085.36,2.42
16999729,4273623.0,9668691,NaN,1.0,3,4,2,2025-10-20 14:31:22,2.0,厦门共勋商贸,2025-10-20 14:31:34,京东活动,1082.94,11.0,0.00,2025-10,1,1085.36,2.42


,首期优惠金额,优惠单数,出库,平均优惠金额
下单月份,,,,
2025-01,821224.76,1479.0,1479.0,555.256768
2025-02,833981.40,1658.0,1658.0,503.004463
2025-03,1109431.73,2232.0,2232.0,497.057227
2025-04,1682357.03,3498.0,3498.0,480.948265
2025-05,2080098.64,3929.0,3929.0,529.421899
2025-06,2259674.50,4364.0,4364.0,517.798923
2025-07,2710836.80,5061.0,5061.0,535.632642
2025-08,2621108.73,4533.0,4533.0,578.228266
2025-09,2091565.27,3271.0,3271.0,639.426863


### 首期领取优惠券

In [215]:
# df_yh_g.to_excel('F:\需求\瑞彤需求\首期领取优惠券.xlsx')

## 预计未续租订单的续租时长数据

### 导入订单数据

In [29]:
# 1、未续租订单(om.status = 4 & om.relet_status = 1)中筛选预计续租超过6个月的订单数（月份汇总+订单明细），例如，6个月100单  7个月90单....
# 2、续租中订单中续租订单的月份分布量
# 续租时长：（买断价-首年租金（总租金））*续租系数 = 续租总租金
# 续租总租金/续租日租金 = 续租时长， 
# 续租时长/30 向上取整即为续租期数
# 续租日租金、续租系数默认为tod.relet_price、tod.relet_out_coefficient;可能会修改，如果t_order_relet_setting表中有数据，则取t_order_relet_setting表中的数据
# 手机分类：165（二手），116
sql_xz = '''
select om.order_number, tod.actual_money, om.all_money, tod.relet_price, tod.relet_out_coefficient, tod.sku_attributes, om.id order_id
, om.create_time, tmu.id_card_num, tors.relet_price as relet_price2, tors.relet_out_coefficient as relet_out_coefficient2
, pa.name as activity_name, om.merchant_name

from db_digua_business.t_order om 
left join db_digua_business.t_order_details tod on om.id = tod.order_id
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join db_digua_business.t_order_relet_setting tors on om.id = tors.order_id
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
left join db_digua_business.t_product tp on tod.product_id = tp.id
where om.status = 4 
and om.relet_status = 1
and tp.classify_id in (165, 116)

'''
sql_order_xz = clean.query(sql_xz)
df_xz = sql_order_xz.copy()
df_xz.shape


(51510, 13)

In [34]:
df_xz[df_xz.order_number=='A2023032120300045']

,order_number,actual_money,all_money,relet_price,relet_out_coefficient,sku_attributes,order_id,create_time,id_card_num,relet_price2,relet_out_coefficient2,activity_name,merchant_name,下单日期,租赁方案,续租日租金,续租系数,续租总租金,续租时长,续租期数
4,A2023032120300045,10399.0,5279.27,15.78,1.0,"[{""custom"":false,""index"":0,""key"":""租赁方案"",""value...",186902,2023-03-21 20:30:01,460003199908273232,34.99,1.23,租手机-苹果系列,一小时,2023-03-21,租完归还,34.99,1.23,6297.2679,179.973361,6.0


### 数据处理

In [30]:
def drop_merchant(df):
        '''
        删除商家数据
        :param df: 传入带有商家的数据
        :return: 返回剔除了商家的数据
        '''
        # 剔除商家数据只保留自营租机业务数据
        df.drop(df[df['merchant_name'] == "深圳优优大数据科技有限公司"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "优优2店"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "小豚租（代收）"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "苏州蚁诺宝"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "租着用电脑数码"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "北京海鸟窝科技有限公司"].index, inplace=True)

        df.drop(df[df['merchant_name'] == "汇客好租"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "澄心优租"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "CPS渠道合作"].index, inplace=True)
        df.drop(df[df['sku_attributes'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)
        # 趣智数码  单
        df.drop(df[df['merchant_name'] == "趣智数码"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "格木木二奢名品"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "广州康基贸易有限公司"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "线下小店"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "乙辉数码"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "呱子笔记本电脑"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "南京聚格网络科技"].index, inplace=True)
        
        df.drop(df[df['merchant_name'] == "星晟数码"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "蘑菇时间"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "云启德曜"].index, inplace=True)

        df.drop(df[df['merchant_name'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)
        
        reject_merchants = ["小蚂蚁租机", "兴鑫兴通讯", "人人享租", "崇胜数码", "喜卓灵租机", "喜卓灵新租机"]
        df.drop(df[df['merchant_name'].isin(reject_merchants)].index, inplace=True)
        return df

df_xz = drop_merchant(df_xz)

In [31]:
# 处理下单日期
df_xz = df_xz[df_xz['sku_attributes'].notnull()]
df_xz["下单日期"]=df_xz["create_time"].dt.date
df_xz["下单日期"]=pd.to_datetime(df_xz["下单日期"],errors="coerce")
# 排除df_xz.new_actual_money - df_xz.all_money<100的订单
df_xz = df_xz[df_xz.actual_money - df_xz.all_money>=100]
df_xz.loc[:,"租赁方案"]=np.where(df_xz["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df_xz = df_xz[df_xz["租赁方案"]=="租完归还"]
# 
df_xz.drop(df_xz[df_xz['activity_name'] == "1000单秘密计划"].index, inplace=True) 
df_xz.drop(df_xz[df_xz['activity_name'] == "1000单秘密计划-无优惠"].index, inplace=True)
df_xz.drop(df_xz[df_xz['activity_name'] == "1000单曙光计划"].index, inplace=True)
df_xz.drop(df_xz[df_xz['activity_name'] == "线下门店3个月试行"].index, inplace=True)
df_xz.drop_duplicates(subset=["order_id"], inplace=True)
# 按下单日期、id_card_num去重
df_xz = df_xz.drop_duplicates(subset=['下单日期', 'id_card_num'], keep='first')
df_xz.shape


(43027, 15)

In [32]:
# 更新续租日租金、续租系数
df_xz.loc[:, '续租日租金'] = np.where(df_xz.relet_price2.isnull(), df_xz.relet_price, df_xz.relet_price2)
df_xz.loc[:, '续租系数'] = np.where(df_xz.relet_out_coefficient2.isnull(), df_xz.relet_out_coefficient, df_xz.relet_out_coefficient2)
# 计算续租总租金、续租期数
df_xz.loc[:, '续租总租金'] = (df_xz.actual_money - df_xz.all_money) * df_xz.续租系数
df_xz.loc[:, '续租时长'] = df_xz.续租总租金 / df_xz.续租日租金
# np.ceil() 向上取整
df_xz.loc[:, '续租期数'] = np.ceil(df_xz.续租时长 / 30)

# 筛选续租期数大于等于6个月的订单
df_xz_6 = df_xz[df_xz.续租期数 >= 6]
# 按续租期数分组统计订单数
df_xz_group = df_xz_6.groupby('续租期数').agg({'order_number': 'count'}).rename(columns={'order_number': '订单数'})

df_xz_group

,订单数
续租期数,
6.0,41851
7.0,7


In [28]:
# df_xz_6.sort_values('续租期数', ascending=False)
# df_xz_6[['order_number', '续租时长', '续租期数', '续租总租金', '续租系数']]
df_xz_6[df_xz_6.order_number=='A2025101815054882']

,order_number,actual_money,all_money,relet_price,relet_out_coefficient,sku_attributes,order_id,create_time,id_card_num,relet_price2,relet_out_coefficient2,activity_name,merchant_name,下单日期,租赁方案,续租日租金,续租系数,续租总租金,续租时长,续租期数
50476,A2025101815054882,12898.7,7993.66,36.14,1.33,"[{""key"":""租赁方案"",""value"":""租完归还/可买断"",""type"":""text...",4253355,2025-10-18 15:05:58,320105199001061810,35.98,1.32,京东活动,洛阳津富建设,2025-10-18,租完归还,35.98,1.32,6474.6528,179.95144,6.0


In [33]:
with pd.ExcelWriter('F:/需求/瑞彤需求/未续租订单的续租时长数据_10.27.xlsx') as writer:
    df_xz_group.to_excel(writer, sheet_name='续租期数分布', index=True)
    df_xz_6[['order_number', '续租时长', '续租期数', '续租总租金', '续租系数']].to_excel(writer, sheet_name='续租时长数据', index=True)

## 续租中订单中实际续租月份分布量（无用）
 

In [77]:

sql_xzing = '''
select om.order_number, tod.sku_attributes, om.id order_id, om.merchant_name, tod.actual_money, om.all_money
, tod.relet_price, tod.relet_out_coefficient, tors.relet_price as relet_price2, tors.relet_out_coefficient as relet_out_coefficient2
, om.create_time, tmu.id_card_num
, pa.name as activity_name, om.merchant_name
, tprm.relet_periods, tprm.relet_days
from db_digua_business.t_order om 
left join db_digua_business.t_postlease_receivables_monitoring tprm on om.id = tprm.order_id
left join db_digua_business.t_order_details tod on om.id = tod.order_id
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join db_digua_business.t_order_relet_setting tors on om.id = tors.order_id
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
left join db_digua_business.t_product tp on tod.product_id = tp.id
-- left join db_rent_
where om.status = 4 
and om.relet_status = 2
and tp.classify_id in (165, 116)
;
'''
df_xzing = clean.query(sql_xzing)
df_xzi = df_xzing.copy()
df_xzi.shape


(5805, 16)

### 数据处理

In [78]:
def drop_merchant(df):
        '''
        删除商家数据
        :param df: 传入带有商家的数据
        :return: 返回剔除了商家的数据
        '''
        # 剔除商家数据只保留自营租机业务数据
        df.drop(df[df['merchant_name'] == "深圳优优大数据科技有限公司"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "优优2店"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "小豚租（代收）"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "苏州蚁诺宝"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "租着用电脑数码"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "北京海鸟窝科技有限公司"].index, inplace=True)

        df.drop(df[df['merchant_name'] == "汇客好租"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "澄心优租"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "CPS渠道合作"].index, inplace=True)
        df.drop(df[df['sku_attributes'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)
        # 趣智数码  单
        df.drop(df[df['merchant_name'] == "趣智数码"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "格木木二奢名品"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "广州康基贸易有限公司"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "线下小店"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "乙辉数码"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "呱子笔记本电脑"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "南京聚格网络科技"].index, inplace=True)
        
        df.drop(df[df['merchant_name'] == "星晟数码"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "蘑菇时间"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "云启德曜"].index, inplace=True)

        df.drop(df[df['merchant_name'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)
        
        reject_merchants = ["小蚂蚁租机", "兴鑫兴通讯", "人人享租", "崇胜数码", "喜卓灵租机", "喜卓灵新租机"]
        df.drop(df[df['merchant_name'].isin(reject_merchants)].index, inplace=True)
        return df
    
df_xzi = drop_merchant(df_xzi)

In [79]:
# 处理下单日期
df_xzi = df_xzi[df_xzi['sku_attributes'].notnull()]
df_xzi["下单日期"]=df_xzi["create_time"].dt.date
df_xzi["下单日期"]=pd.to_datetime(df_xzi["下单日期"],errors="coerce")
df_xzi.loc[:,"租赁方案"]=np.where(df_xzi["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df_xzi = df_xzi[df_xzi["租赁方案"]=="租完归还"]
# 
df_xzi.drop(df_xzi[df_xzi['activity_name'] == "1000单秘密计划"].index, inplace=True) 
df_xzi.drop(df_xzi[df_xzi['activity_name'] == "1000单秘密计划-无优惠"].index, inplace=True)
df_xzi.drop(df_xzi[df_xzi['activity_name'] == "1000单曙光计划"].index, inplace=True)
df_xzi.drop(df_xzi[df_xzi['activity_name'] == "线下门店3个月试行"].index, inplace=True)
df_xzi.drop_duplicates(subset=["order_id"], inplace=True)
# 按下单日期、id_card_num去重
df_xzi = df_xzi.drop_duplicates(subset=['下单日期', 'id_card_num'], keep='first')
df_xzi.shape


(5805, 18)

In [81]:
df_xzi.loc[:, '续租系数'] = np.where(df_xzi.relet_out_coefficient2.isnull(), df_xzi.relet_out_coefficient, df_xzi.relet_out_coefficient2)
df_xzi.loc[:, '续租总租金'] = (df_xzi.actual_money - df_xzi.all_money) * df_xzi.续租系数
df_xzi.rename(columns={'relet_periods': '续租期数', 'relet_days': '续租天数'}, inplace=True)
df_xzi_g = df_xzi.groupby('续租期数').agg({'order_number': 'count'}).rename(columns={'order_number': '订单数'})


In [82]:
with pd.ExcelWriter('F:/需求/瑞彤需求/已续租订单的实际续租期数数据.xlsx') as writer:
    df_xzi_g.to_excel(writer, sheet_name='续租期数分布', index=True)
    df_xzi[['order_number', '续租天数', '续租期数', '续租总租金', '续租系数']].to_excel(writer, sheet_name='续租时长数据', index=True)

## 续租中订单中理论续租月份分布量

In [97]:
# 
sql_xzed = '''
select om.order_number, tod.sku_attributes, om.id order_id, om.merchant_name, tod.actual_money, om.all_money
, tod.relet_price, tod.relet_out_coefficient, tors.relet_price as relet_price2, tors.relet_out_coefficient as relet_out_coefficient2
, om.create_time 
-- , tmu.id_card_num
, pa.name as activity_name
, ymos.money, ymos.sort, ymos.is_relet
from db_digua_business.t_order om 
-- left join db_digua_business.t_postlease_receivables_monitored tprm on om.id = tprm.order_id
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
left join db_digua_business.t_order_relet_setting tors on om.id = tors.order_id
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
left join db_digua_business.t_product tp on tod.product_id = tp.id
left join db_rent.ya_merchant_order_stages ymos on om.id = ymos.order_id
where om.status = 4 
and om.relet_status = 2
and tp.classify_id in (165, 116)
;
'''
df_xze = clean.query(sql_xzed)

df_xze.shape


(121797, 15)

### 数据处理

In [98]:
def drop_merchant(df):
        '''
        删除商家数据
        :param df: 传入带有商家的数据
        :return: 返回剔除了商家的数据
        '''
        # 剔除商家数据只保留自营租机业务数据
        df.drop(df[df['merchant_name'] == "深圳优优大数据科技有限公司"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "优优2店"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "小豚租（代收）"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "苏州蚁诺宝"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "租着用电脑数码"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "北京海鸟窝科技有限公司"].index, inplace=True)

        df.drop(df[df['merchant_name'] == "汇客好租"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "澄心优租"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "CPS渠道合作"].index, inplace=True)
        df.drop(df[df['sku_attributes'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)
        # 趣智数码  单
        df.drop(df[df['merchant_name'] == "趣智数码"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "格木木二奢名品"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "广州康基贸易有限公司"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "线下小店"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "乙辉数码"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "呱子笔记本电脑"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "南京聚格网络科技"].index, inplace=True)
        
        df.drop(df[df['merchant_name'] == "星晟数码"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "蘑菇时间"].index, inplace=True)
        df.drop(df[df['merchant_name'] == "云启德曜"].index, inplace=True)

        df.drop(df[df['merchant_name'].str.contains(pat='探路者', regex=False) == True].index, inplace=True)
        
        reject_merchants = ["小蚂蚁租机", "兴鑫兴通讯", "人人享租", "崇胜数码", "喜卓灵租机", "喜卓灵新租机"]
        df.drop(df[df['merchant_name'].isin(reject_merchants)].index, inplace=True)
        return df
df_xzed = df_xze.copy()
df_xzed = drop_merchant(df_xzed)

In [101]:
# 处理下单日期
df_xzed = df_xzed[df_xzed['sku_attributes'].notnull()]
df_xzed["下单日期"]=df_xzed["create_time"].dt.date
df_xzed["下单日期"]=pd.to_datetime(df_xzed["下单日期"],errors="coerce")
# 排除df_xzed.new_actual_money - df_xzed.all_money<100的订单
df_xzed = df_xzed[df_xzed.actual_money - df_xzed.all_money>=100]
df_xzed.loc[:,"租赁方案"]=np.where(df_xzed["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df_xzed = df_xzed[df_xzed["租赁方案"]=="租完归还"]
# 
df_xzed.drop(df_xzed[df_xzed['activity_name'] == "1000单秘密计划"].index, inplace=True) 
df_xzed.drop(df_xzed[df_xzed['activity_name'] == "1000单秘密计划-无优惠"].index, inplace=True)
df_xzed.drop(df_xzed[df_xzed['activity_name'] == "1000单曙光计划"].index, inplace=True)
df_xzed.drop(df_xzed[df_xzed['activity_name'] == "线下门店3个月试行"].index, inplace=True)

# 按'order_number', 'sort'去重
df_xzed = df_xzed.drop_duplicates(subset=['order_number', 'sort'], keep='first')
df_xzed.shape


(121449, 21)

In [102]:
# 更新续租日租金、续租系数
df_xzed.loc[:, '续租日租金'] = np.where(df_xzed.relet_price2.isnull(), df_xzed.relet_price, df_xzed.relet_price2)
df_xzed.loc[:, '续租系数'] = np.where(df_xzed.relet_out_coefficient2.isnull(), df_xzed.relet_out_coefficient, df_xzed.relet_out_coefficient2)
# 计算续租总租金、续租期数
# 续租期数为is_relet=1的最大sort-is_relet=0的最大sort
import pandas as pd

# 1. 计算每个order_number下is_relet=0的最大sort（首租最大）
max_sort_0 = df_xzed[df_xzed['is_relet'] == 0].groupby('order_number')['sort'].max()

# 2. 计算每个order_number下is_relet=1的最大sort（续租最大）
max_sort_1 = df_xzed[df_xzed['is_relet'] == 1].groupby('order_number')['sort'].max()

# 3. 计算续租期数（续租最大 - 首租最大），并通过order_number映射回原DataFrame
df_xzed['续租期数'] = df_xzed['order_number'].map(max_sort_1) - df_xzed['order_number'].map(max_sort_0)

# 1. 计算每个order_number下，is_relet==1的money总和
relet_rent_per_order = df_xzed[df_xzed['is_relet'] == 1].groupby('order_number')['money'].sum()

# 2. 将结果映射回原DataFrame，新增“续租总租金”列
df_xzed['续租总租金'] = df_xzed['order_number'].map(relet_rent_per_order)

# （可选）如果某个订单没有续租（is_relet==1的记录），用0填充NaN
df_xzed['续租总租金'] = df_xzed['续租总租金'].fillna(0)

# 按order_number去重
df_xzed = df_xzed.drop_duplicates(subset=['order_number'], keep='first')


# np.ceil() 向上取整


# # 筛选续租期数大于等于6个月的订单
# df_xzed_6 = df_xzed[df_xzed.续租期数 >= 6]
# 按续租期数分组统计订单数
df_xzed_group = df_xzed.groupby('续租期数').agg({'order_number': 'count'}).rename(columns={'order_number': '订单数'})

df_xzed_group

,订单数
续租期数,
3.0,1
5.0,4
6.0,1042
7.0,109
8.0,46
9.0,271
10.0,1019
11.0,1586
12.0,1380


In [103]:
# # df_xzed.shape(5540, 21)
# with pd.ExcelWriter('F:/需求/瑞彤需求/已续租订单的续租期数数据.xlsx') as writer:
#     df_xzed_group.to_excel(writer, sheet_name='续租期数分布', index=True)
#     df_xzed[['order_number', '续租期数', '续租总租金', '续租系数']].to_excel(writer, sheet_name='续租时长数据', index=True)


# 七月需求

## 23-25年出库数据

In [ ]:
# def year_sum(df, year):
#     df2 = df[df.年份==year]176
#     df2_ck = df2[df2.是否出库==1]
#     df2_group = df2_ck.groupby('下单月份').agg({'是否出库': 'sum', 'purchase_amount': 'sum', 'new_actual_money': 'sum'}).rename(columns={'是否出库': '出库数', 'purchase_amount': '采购金额', 'new_actual_money': '签约价'})
#     return df2_group
# df_group_2023 = year_sum(df, 2023)
# df_group_2024 = year_sum(df, 2024)
# df_group_2025 = year_sum(df, 2025)
# df_group_2025
# with pd.ExcelWriter(f'F:/需求/七月需求/23-25年出库数据1.xlsx', engine='xlsxwriter') as writer:
#     df_group_2023.to_excel(writer, sheet_name='2023年出库数据')
#     df_group_2024.to_excel(writer, sheet_name='2024年出库数据')
#     df_group_2025.to_excel(writer, sheet_name='2025年出库数据')

# df_new = df[~df.merchant_name.isin(['小蚂蚁租机', '兴鑫兴通讯', '人人享租', '崇胜数码', '喜卓灵租机'])]
# df_new_g = df_new[df_new.是否出库==1].groupby('下单月份').agg({'是否出库': 'sum', 'new_actual_money': 'sum'}).rename(columns={'new_actual_money': '签约价', '是否出库': '出库'})
# df_g = df.groupby('下单月份').agg({'是否进件': 'sum'}).rename(columns={'是否进件': '进件'})
# df_group = df_g.merge(df_new_g, on='下单月份', how='left').reset_index()
# df_group[['进件', '出库', '签约价']].set_index('进件')

## 近一年碎屏险单

In [12]:
sql = '''
select tso.begin_date, tso.price, tso.order_id

from db_digua_business.t_service_order tso
where tso.status = 3
-- and tso.begin_date >= DATE_sub(CURRENT_DATE,INTERVAL 1 YEAR )
and DATE_FORMAT(tso.begin_date, '%Y-%m-%d')>='2024-10-01'
and DATE_FORMAT(tso.begin_date, '%Y-%m-%d')<='2025-09-30'
'''

df_order = clean.query(sql)
df_order.shape



(34846, 3)

In [13]:
df = df_order.copy()
df["下单日期"]=df["begin_date"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单月份"]=df["begin_date"].dt.strftime('%Y-%m')
df.下单月份.unique()

array(['2024-10', '2024-11', '2024-12', '2025-01', '2025-02', '2025-03',
       '2025-04', '2025-05', '2025-06', '2025-07', '2025-08', '2025-09'],
      dtype=object)

In [15]:
# 按下单月份分组计算碎屏险单总金额
df_so = df.groupby('下单月份').agg({'price': 'sum', 'order_id': 'count'}).rename(columns={'price': '碎屏险金额', 'order_id': '订单数'})#.reset_index()
df_so.loc['汇总', :] = df_so.select_dtypes(include=[np.number]).sum()
df_so.to_excel('F:/需求/七月需求/近一年碎屏险单.xlsx', index=True)